<a href="https://colab.research.google.com/github/ksw9092ksw2902/DAB/blob/main/%EC%B5%9C%EC%A2%85_%EC%BD%94%EB%93%9C_%EC%A0%95%EB%A6%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pulp

### 기상청 API + 수요 예측

In [ ]:
import pandas as pd
import numpy as np
import joblib
from datetime import datetime, timedelta
import requests
import json
from zoneinfo import ZoneInfo

# --- Google Colab 환경 설정 ---
from google.colab import drive
drive.mount('/content/drive')

# --- 1. 💡 사용자 설정 💡 ---
# 기상자료개방포털에서 발급받은 인증키(authKey)
KMA_AUTH_KEY = "kY2gpTQjR0iNoKU0I9dI5g"

# --- 2. 경로 설정, 대여소 목록 ---
BASE_PATH = '/content/drive/MyDrive/DAB_따릉팡/RF 정리/DAB/'
MODEL_DIR_DEMAND = f'{BASE_PATH}saved_models_all_final_수요'
MODEL_DIR_RETURN = f'{BASE_PATH}saved_models_all_final_반납'

STATIONS_LIST = [
    1302, 1303, 1304, 1305, 1306, 1308, 1314, 1315, 1316, 1318, 1319, 1321, 1322, 1325, 1326, 1327, 1328, 1332, 1333, 1334,
    1336, 1337, 1338, 1339, 1340, 1342, 1343, 1344, 1346, 1347, 1348, 1349, 1351, 1352, 1353, 1354, 1357, 1358, 1360, 1361,
    1362, 1363, 1364, 1365, 1366, 1367, 1368, 1370, 1372, 1373, 1375, 1376, 1377, 1379, 1380, 1381, 1382, 1384, 1385, 1386,
    1388, 1390, 1391, 1392, 1393, 1394, 1395, 1396, 4406, 4408, 4411, 4414, 4415, 4417, 4418, 4419, 4420, 4421, 4423, 4425
]
stations_to_exclude = [4462, 1362, 4408]
stations_to_exclude_final = [s for s in stations_to_exclude if s in STATIONS_LIST]
STATIONS_LIST = [s for s in STATIONS_LIST if s not in stations_to_exclude_final]

# --- 3. 헬퍼 함수  ---
def get_latest_kma_forecast(auth_key, now_kst):
    print("  - 기상자료개방포털 API를 통해 최신 예보를 가져옵니다...")
    announcement_times = [2, 5, 8, 11, 14, 17, 20, 23]
    current_hour = now_kst.hour
    if current_hour < 2:
        base_date_obj = now_kst - timedelta(days=1)
        base_time_hour = 23
    else:
        base_date_obj = now_kst
        base_time_hour = max([t for t in announcement_times if t <= current_hour])
    base_date = base_date_obj.strftime('%Y%m%d')
    base_time = f'{base_time_hour:02d}00'
    url = "https://apihub.kma.go.kr/api/typ02/openApi/VilageFcstInfoService_2.0/getVilageFcst"
    params = {'authKey': auth_key, 'pageNo': '1', 'numOfRows': '1000', 'dataType': 'JSON', 'base_date': base_date, 'base_time': base_time, 'nx': '61', 'ny': '128'}
    try:
        response = requests.get(url, params=params, timeout=15)
        response.raise_for_status()
        data = response.json()
        if data['response']['header']['resultCode'] != '00': return None
        items = data['response']['body']['items']['item']
        forecasts = {}
        for item in items:
            key = item['fcstDate'] + item['fcstTime']
            if key not in forecasts: forecasts[key] = {}
            if item['category'] == 'TMP': forecasts[key]['temp'] = float(item['fcstValue'])
            elif item['category'] == 'PCP':
                val = item['fcstValue'] # 예: '1mm', '5mm', '1 미만', '강수없음', '30.0' 등


                if 'mm' in val:
                    # 'mm' 단위가 포함된 경우 (예: '1mm', '5.0mm')
                    try:
                        # 'mm' 제거 후 float 변환 시도
                        forecasts[key]['rain'] = float(val.replace('mm', ''))
                    except ValueError:
                        # 변환 실패 시 0.0으로 처리 (예: ' - mm')
                        forecasts[key]['rain'] = 0.0
                elif '미만' in val:
                    # '1 미만', '0.1 미만' 등 -> 0.0으로 처리
                    forecasts[key]['rain'] = 0.0
                elif val in ['강수없음', '-']:
                     # '강수없음' 또는 '-' 문자열 -> 0.0으로 처리
                     forecasts[key]['rain'] = 0.0
                else:
                    # 그 외의 경우 (단위 없는 숫자 문자열 등)
                    try:
                        # float 변환 시도 (예: '30.0')
                        forecasts[key]['rain'] = float(val)
                    except ValueError:
                        # 위 모든 경우에 해당하지 않고 변환도 실패하면 0.0으로 처리
                        forecasts[key]['rain'] = 0.0

        print(f"    - 조회 기준: {base_date} {base_time} 발표 자료 수신 완료.")
        return forecasts
    except Exception as e:
        print(f"❗️오류: 기상청 API 호출 중 문제가 발생했습니다: {e}")
        return None

def generate_simple_future_features(target_date, stations, weather_df, station_col):
    hours = range(24)
    future_df = pd.DataFrame([(station, hour) for station in stations for hour in hours], columns=[station_col, '시간대'])
    future_df['기준일자'] = pd.to_datetime(target_date)
    future_df = pd.merge(future_df, weather_df, on='시간대', how='left')
    future_df['월'] = future_df['기준일자'].dt.month
    future_df['요일'] = future_df['기준일자'].dt.dayofweek
    future_df['주중/주말/공휴일'] = np.where(future_df['요일'] >= 5, '주말', '주중')
    future_df['월_sin'] = np.sin(2 * np.pi * (future_df['월'] - 1) / 12)
    future_df['월_cos'] = np.cos(2 * np.pi * (future_df['월'] - 1) / 12)
    future_df['요일_sin'] = np.sin(2 * np.pi * future_df['요일'] / 7)
    future_df['요일_cos'] = np.cos(2 * np.pi * future_df['요일'] / 7)
    return future_df

def aggregate_to_block_simple(df, time_col, station_col):
    df_copy = df.copy()
    df_copy['시간블록'] = np.where((df_copy[time_col] >= 8) & (df_copy[time_col] <= 19), '주간', '야간')
    agg_rules = { '평균기온': 'mean', '총강수량': 'sum', '월_sin': 'first', '월_cos': 'first', '요일_sin': 'first', '요일_cos': 'first' }
    grouping_cols = ['기준일자', '시간블록', station_col, '주중/주말/공휴일']
    return df_copy.groupby(grouping_cols).agg(agg_rules).reset_index()

# --- 4. 메인 실행 로직 ---
if __name__ == "__main__":
    now_kst = datetime.now(ZoneInfo('Asia/Seoul'))

    # 주간/야간 예측에 필요한 날짜들을 미리 계산
    target_dates = {}
    # 주간(08~20시) 예측 대상 날짜 계산
    if now_kst.hour < 20: # 20시 이전이면 '오늘' 주간 블록을 예측
        target_dates['주간'] = now_kst.date()
    else: # 20시 이후면 '내일' 주간 블록을 예측
        target_dates['주간'] = (now_kst + timedelta(days=1)).date()

    # 야간(20~08시) 예측 대상 날짜 계산

    target_dates['야간'] = now_kst.date()

    print(f">>> 현재 시각({now_kst.strftime('%H:%M')}) 기준, 주간/야간 수요 예측을 모두 시작합니다.")
    print(f"    - 주간 예측 대상 날짜: {target_dates['주간']}")
    print(f"    - 야간 예측 대상 날짜: {target_dates['야간']}")

    all_forecasts = get_latest_kma_forecast(KMA_AUTH_KEY, now_kst)

    if all_forecasts is None:
        print("❗️API 호출 실패로 예측을 중단합니다. 잠시 후 다시 시도해주세요.")
        exit()

    try:
        models = {
            '주간': {'수요': joblib.load(f'{MODEL_DIR_DEMAND}/rf_model_global_주간_수요.joblib'), '반납': joblib.load(f'{MODEL_DIR_RETURN}/rf_model_global_주간_반납.joblib')},
            '야간': {'수요': joblib.load(f'{MODEL_DIR_DEMAND}/rf_model_global_야간_수요.joblib'), '반납': joblib.load(f'{MODEL_DIR_RETURN}/rf_model_global_야간_반납.joblib')}
        }
    except Exception as e:
        print(f"❗️오류: 모델 파일 로딩 중 문제가 발생했습니다. 경로를 확인해주세요: {e}")
        exit()

    print("  - 모델 예측 및 최종 데이터셋 생성 중...")
    output_dict = {}
    block_map = {'주간': 1, '야간': 2}

    #  주간, 야간 순서로 루프를 돌며 각각 예측 수행
    for block_type in ['주간', '야간']:
        target_date_for_prediction = target_dates[block_type]

        # 해당 날짜의 날씨 데이터 추출
        temp_list, rain_list = [], []
        for hour in range(24):
            dt_key = (datetime.combine(target_date_for_prediction, datetime.min.time()) + timedelta(hours=hour)).strftime('%Y%m%d%H00')
            forecast_hour = all_forecasts.get(dt_key, {'temp': 12.0, 'rain': 0.0}) # 예보 없으면 기본값
            temp_list.append(forecast_hour.get('temp', 12.0))
            rain_list.append(forecast_hour.get('rain', 0.0))

        weather_forecast_df = pd.DataFrame({'시간대': range(24), '평균기온': temp_list, '총강수량': rain_list})

        # 피처 생성
        future_features_hourly = generate_simple_future_features(target_date_for_prediction, STATIONS_LIST, weather_forecast_df, '대여 대여소번호')
        future_blocks = aggregate_to_block_simple(future_features_hourly, '시간대', '대여 대여소번호')

        merged_block = future_blocks[future_blocks['시간블록'] == block_type]

        if not merged_block.empty:
            demand_model = models[block_type]['수요']
            return_model = models[block_type]['반납']

            block_ohe = pd.get_dummies(merged_block, columns=['주중/주말/공휴일', '대여 대여소번호'], drop_first=True)
            X_demand = block_ohe.reindex(columns=demand_model.feature_names_in_, fill_value=0)
            X_return = block_ohe.reindex(columns=return_model.feature_names_in_, fill_value=0)

            pred_demand = demand_model.predict(X_demand)
            pred_return = return_model.predict(X_return)

            merged_block_with_pred = merged_block.copy()
            merged_block_with_pred['순수요_예측값'] = pred_return - pred_demand

            for _, row in merged_block_with_pred.iterrows():
                station_id = int(row['대여 대여소번호'])
                block_id = block_map[block_type]
                net_demand = round(row['순수요_예측값'], 3)
                output_dict[(station_id, block_id)] = net_demand



Mounted at /content/drive
>>> 현재 시각(20:16) 기준, 주간/야간 수요 예측을 모두 시작합니다.
    - 주간 예측 대상 날짜: 2026-01-14
    - 야간 예측 대상 날짜: 2026-01-13
  - 기상자료개방포털 API를 통해 최신 예보를 가져옵니다...
    - 조회 기준: 20260113 2000 발표 자료 수신 완료.


/usr/local/lib/python3.12/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.7.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator RandomForestRegressor from version 1.7.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


  - 모델 예측 및 최종 데이터셋 생성 중...


In [ ]:
#정류장 번호
station_ID = {1302, 1303, 1304, 1305, 1306, 1308, 1314, 1315, 1316, 1318, 1319, 1321, 1322, 1325, 1326, 1327, 1328, 1332, 1333, 1334, 1336, 1337, 1338, 1339, 1340, 1342, 1343, 1344, 1346, 1347, 1348, 1349, 1351, 1352, 1353, 1354, 1357, 1358, 1360, 1361, 1363, 1364, 1365, 1366, 1367, 1368, 1370, 1372, 1373, 1375, 1376, 1377, 1379, 1380, 1381, 1382, 1384, 1385, 1386, 1388, 1390, 1391, 1392, 1393, 1394, 1395, 1396, 4406, 4411, 4414, 4415, 4417, 4418, 4419, 4420, 4421, 4423, 4425}

### 따릉이 재고 API

In [ ]:
import requests
import json
import re

# 1. 우리 모델이 분석하려는 기준 대여소 목록
my_station_set = set(station_ID)
print(f"분석 대상 대여소 목록: 총 {len(my_station_set)}개")

# 2. stationName에서 대여소 번호를 추출하는 함수
def extract_station_number_from_name(station_name):
    match = re.match(r'(\d+)\.', station_name)
    if match:
        return int(match.group(1))
    return None

# 3. API 정보 설정
URL = "https://www.bikeseoul.com/app/station/getStationRealtimeStatus.do"
PAYLOAD = {"stationGrpSeq": "ALL"}

print("실시간 따릉이 재고 정보를 API로 가져옵니다...")

# 4. API 요청 및 최종 S0 생성
try:
    response = requests.post(URL, data=PAYLOAD, timeout=10)
    response.raise_for_status()
    data = response.json()

    # (총 재고, 총 거치대 수)를 함께 저장
    api_data_by_name = {}
    for station_info in data.get('realtimeList', []):
        station_name = station_info.get("stationName", "")
        station_number = extract_station_number_from_name(station_name)

        if station_number is not None:
            qr_stock = int(station_info.get("parkingQRBikeCnt", "0"))
            lcd_stock = int(station_info.get("parkingBikeTotCnt", "0"))
            total_stock = qr_stock + lcd_stock
            # 총 거치대 수('rackTotCnt') 정보를 함께 가져옵니다.
            rack_count = int(station_info.get("rackTotCnt", "0"))
            api_data_by_name[station_number] = (total_stock, rack_count)


    realtime_initial_stock = {}
    for sid in my_station_set:
        stock_info = api_data_by_name.get(sid) # (재고, 거치대수) 튜플 또는 None

        if stock_info:
            current_stock, rack_count = stock_info

            if current_stock > 0:
                # 실시간 재고가 0보다 크면, 그 값을 그대로 사용
                realtime_initial_stock[sid] = current_stock
            else:

                assumed_stock = max(1, round(rack_count / 2))
                realtime_initial_stock[sid] = assumed_stock
        else:
            # API 응답에 대여소 정보가 아예 없는 경우 0으로 처리
            realtime_initial_stock[sid] = 0

    # 5. 결과 확인
    zero_stock_stations = [sid for sid, stock in realtime_initial_stock.items() if stock == 0]

    print("-" * 50)
    print(f" S0 변수에 초기 재고를 반영했습니다.")
    print(f"최종적으로 재고가 0으로 처리된 대여소(API에 없음): {len(zero_stock_stations)}개")
    print("-" * 50)
    print("S0 변수 샘플:", dict(list(realtime_initial_stock.items())[:10]))

except requests.exceptions.RequestException as e:
    print(f"🔴 API 요청 실패: {e}")
    realtime_initial_stock = None

분석 대상 대여소 목록: 총 78개
실시간 따릉이 재고 정보를 API로 가져옵니다...
--------------------------------------------------
 S0 변수에 초기 재고를 반영했습니다.
최종적으로 재고가 0으로 처리된 대여소(API에 없음): 0개
--------------------------------------------------
S0 변수 샘플: {1302: 3, 1303: 2, 1304: 1, 1305: 20, 1306: 1, 1308: 1, 1314: 6, 1315: 12, 1316: 6, 1318: 5}


In [ ]:
# 대여소 별 t 시점의 순수요량 (-:대여, +:반납) , (i,t) : 순수요량
net_demand = output_dict

In [ ]:
# @title
# 대여소 간 t 시점의 이동 시간, (i,j,t) : 이동시간


travel_time = {(0, 1302, 2): 23.5,
 (0, 1303, 2): 22.0,
 (0, 1304, 2): 25.7,
 (0, 1305, 2): 19.3,
 (0, 1306, 2): 23.2,
 (0, 1308, 2): 12.2,
 (0, 1314, 2): 23.2,
 (0, 1315, 2): 24.2,
 (0, 1316, 2): 19.3,
 (0, 1318, 2): 19.5,
 (0, 1319, 2): 16.6,
 (0, 1321, 2): 16.1,
 (0, 1322, 2): 19.6,
 (0, 1325, 2): 24.6,
 (0, 1326, 2): 22.1,
 (0, 1327, 2): 22.1,
 (0, 1328, 2): 20.1,
 (0, 1332, 2): 17.6,
 (0, 1333, 2): 17.4,
 (0, 1334, 2): 17.9,
 (0, 1336, 2): 20.6,
 (0, 1337, 2): 22.1,
 (0, 1338, 2): 19.5,
 (0, 1339, 2): 22.1,
 (0, 1340, 2): 23.7,
 (0, 1342, 2): 23.1,
 (0, 1343, 2): 23.1,
 (0, 1344, 2): 25.8,
 (0, 1346, 2): 22.2,
 (0, 1347, 2): 21.2,
 (0, 1348, 2): 19.8,
 (0, 1349, 2): 22.9,
 (0, 1351, 2): 13.6,
 (0, 1352, 2): 15.8,
 (0, 1353, 2): 18.6,
 (0, 1354, 2): 18.2,
 (0, 1357, 2): 23.4,
 (0, 1358, 2): 21.0,
 (0, 1360, 2): 22.2,
 (0, 1361, 2): 25.1,
 (0, 1363, 2): 14.8,
 (0, 1364, 2): 24.5,
 (0, 1365, 2): 26.4,
 (0, 1366, 2): 19.5,
 (0, 1367, 2): 20.0,
 (0, 1368, 2): 22.4,
 (0, 1370, 2): 25.2,
 (0, 1372, 2): 15.3,
 (0, 1373, 2): 17.6,
 (0, 1375, 2): 19.6,
 (0, 1376, 2): 19.8,
 (0, 1377, 2): 19.0,
 (0, 1379, 2): 23.9,
 (0, 1380, 2): 16.5,
 (0, 1381, 2): 19.8,
 (0, 1382, 2): 21.3,
 (0, 1384, 2): 23.2,
 (0, 1385, 2): 21.9,
 (0, 1386, 2): 22.6,
 (0, 1388, 2): 24.3,
 (0, 1390, 2): 20.1,
 (0, 1391, 2): 21.1,
 (0, 1392, 2): 25.0,
 (0, 1393, 2): 20.3,
 (0, 1394, 2): 18.3,
 (0, 1395, 2): 24.4,
 (0, 1396, 2): 23.0,
 (0, 4406, 2): 23.0,
 (0, 4411, 2): 20.1,
 (0, 4414, 2): 14.8,
 (0, 4415, 2): 27.4,
 (0, 4417, 2): 26.4,
 (0, 4418, 2): 16.2,
 (0, 4419, 2): 19.8,
 (0, 4420, 2): 19.0,
 (0, 4421, 2): 19.6,
 (0, 4423, 2): 20.7,
 (0, 4425, 2): 24.2,
 (1302, 0, 2): 24.4,
 (1302, 1303, 2): 10.6,
 (1302, 1304, 2): 5.6,
 (1302, 1305, 2): 9.2,
 (1302, 1306, 2): 6.4,
 (1302, 1308, 2): 16.1,
 (1302, 1314, 2): 8.5,
 (1302, 1315, 2): 12.4,
 (1302, 1316, 2): 16.7,
 (1302, 1318, 2): 15.9,
 (1302, 1319, 2): 15.6,
 (1302, 1321, 2): 17.7,
 (1302, 1322, 2): 16.5,
 (1302, 1325, 2): 22.4,
 (1302, 1326, 2): 19.5,
 (1302, 1327, 2): 19.5,
 (1302, 1328, 2): 17.8,
 (1302, 1332, 2): 26.3,
 (1302, 1333, 2): 23.4,
 (1302, 1334, 2): 23.8,
 (1302, 1336, 2): 9.2,
 (1302, 1337, 2): 9.4,
 (1302, 1338, 2): 11.6,
 (1302, 1339, 2): 19.8,
 (1302, 1340, 2): 27.3,
 (1302, 1342, 2): 4.2,
 (1302, 1343, 2): 1.6,
 (1302, 1344, 2): 14.5,
 (1302, 1346, 2): 18.9,
 (1302, 1347, 2): 17.8,
 (1302, 1348, 2): 16.4,
 (1302, 1349, 2): 27.9,
 (1302, 1351, 2): 12.4,
 (1302, 1352, 2): 18.2,
 (1302, 1353, 2): 13.3,
 (1302, 1354, 2): 17.4,
 (1302, 1357, 2): 16.0,
 (1302, 1358, 2): 14.2,
 (1302, 1360, 2): 9.6,
 (1302, 1361, 2): 2.0,
 (1302, 1363, 2): 11.2,
 (1302, 1364, 2): 1.4,
 (1302, 1365, 2): 3.2,
 (1302, 1366, 2): 21.4,
 (1302, 1367, 2): 16.6,
 (1302, 1368, 2): 12.1,
 (1302, 1370, 2): 4.9,
 (1302, 1372, 2): 16.0,
 (1302, 1373, 2): 18.4,
 (1302, 1375, 2): 20.4,
 (1302, 1376, 2): 22.2,
 (1302, 1377, 2): 20.1,
 (1302, 1379, 2): 17.0,
 (1302, 1380, 2): 17.5,
 (1302, 1381, 2): 27.0,
 (1302, 1382, 2): 22.2,
 (1302, 1384, 2): 11.8,
 (1302, 1385, 2): 10.2,
 (1302, 1386, 2): 11.0,
 (1302, 1388, 2): 9.2,
 (1302, 1390, 2): 28.5,
 (1302, 1391, 2): 30.1,
 (1302, 1392, 2): 12.7,
 (1302, 1393, 2): 12.1,
 (1302, 1394, 2): 12.3,
 (1302, 1395, 2): 27.7,
 (1302, 1396, 2): 28.1,
 (1302, 4406, 2): 25.5,
 (1302, 4411, 2): 21.5,
 (1302, 4414, 2): 13.6,
 (1302, 4415, 2): 22.9,
 (1302, 4417, 2): 16.1,
 (1302, 4418, 2): 14.3,
 (1302, 4419, 2): 26.1,
 (1302, 4420, 2): 25.4,
 (1302, 4421, 2): 14.6,
 (1302, 4423, 2): 17.8,
 (1302, 4425, 2): 25.8,
 (1303, 0, 2): 19.5,
 (1303, 1302, 2): 2.8,
 (1303, 1304, 2): 8.8,
 (1303, 1305, 2): 4.6,
 (1303, 1306, 2): 6.6,
 (1303, 1308, 2): 10.5,
 (1303, 1314, 2): 8.2,
 (1303, 1315, 2): 10.4,
 (1303, 1316, 2): 14.0,
 (1303, 1318, 2): 7.4,
 (1303, 1319, 2): 9.3,
 (1303, 1321, 2): 12.6,
 (1303, 1322, 2): 10.1,
 (1303, 1325, 2): 16.4,
 (1303, 1326, 2): 13.2,
 (1303, 1327, 2): 13.2,
 (1303, 1328, 2): 11.5,
 (1303, 1332, 2): 19.9,
 (1303, 1333, 2): 17.0,
 (1303, 1334, 2): 17.4,
 (1303, 1336, 2): 6.1,
 (1303, 1337, 2): 4.8,
 (1303, 1338, 2): 7.0,
 (1303, 1339, 2): 11.2,
 (1303, 1340, 2): 19.0,
 (1303, 1342, 2): 7.8,
 (1303, 1343, 2): 2.5,
 (1303, 1344, 2): 6.1,
 (1303, 1346, 2): 10.3,
 (1303, 1347, 2): 9.3,
 (1303, 1348, 2): 7.9,
 (1303, 1349, 2): 19.6,
 (1303, 1351, 2): 7.4,
 (1303, 1352, 2): 8.9,
 (1303, 1353, 2): 4.9,
 (1303, 1354, 2): 11.2,
 (1303, 1357, 2): 13.9,
 (1303, 1358, 2): 12.1,
 (1303, 1360, 2): 7.6,
 (1303, 1361, 2): 5.1,
 (1303, 1363, 2): 6.2,
 (1303, 1364, 2): 4.6,
 (1303, 1365, 2): 6.4,
 (1303, 1366, 2): 16.2,
 (1303, 1367, 2): 8.1,
 (1303, 1368, 2): 7.0,
 (1303, 1370, 2): 8.1,
 (1303, 1372, 2): 10.8,
 (1303, 1373, 2): 12.1,
 (1303, 1375, 2): 14.0,
 (1303, 1376, 2): 17.1,
 (1303, 1377, 2): 13.8,
 (1303, 1379, 2): 11.9,
 (1303, 1380, 2): 11.2,
 (1303, 1381, 2): 20.6,
 (1303, 1382, 2): 15.8,
 (1303, 1384, 2): 2.3,
 (1303, 1385, 2): 1.9,
 (1303, 1386, 2): 2.6,
 (1303, 1388, 2): 3.6,
 (1303, 1390, 2): 23.6,
 (1303, 1391, 2): 21.7,
 (1303, 1392, 2): 4.1,
 (1303, 1393, 2): 8.4,
 (1303, 1394, 2): 7.9,
 (1303, 1395, 2): 19.1,
 (1303, 1396, 2): 19.4,
 (1303, 4406, 2): 19.2,
 (1303, 4411, 2): 12.9,
 (1303, 4414, 2): 8.5,
 (1303, 4415, 2): 16.7,
 (1303, 4417, 2): 14.0,
 (1303, 4418, 2): 8.1,
 (1303, 4419, 2): 19.9,
 (1303, 4420, 2): 19.1,
 (1303, 4421, 2): 8.6,
 (1303, 4423, 2): 9.2,
 (1303, 4425, 2): 17.2,
 (1304, 0, 2): 27.8,
 (1304, 1302, 2): 7.9,
 (1304, 1303, 2): 14.4,
 (1304, 1305, 2): 12.7,
 (1304, 1306, 2): 10.0,
 (1304, 1308, 2): 19.7,
 (1304, 1314, 2): 11.9,
 (1304, 1315, 2): 12.6,
 (1304, 1316, 2): 15.0,
 (1304, 1318, 2): 19.8,
 (1304, 1319, 2): 21.4,
 (1304, 1321, 2): 21.3,
 (1304, 1322, 2): 22.4,
 (1304, 1325, 2): 28.3,
 (1304, 1326, 2): 25.5,
 (1304, 1327, 2): 25.5,
 (1304, 1328, 2): 23.8,
 (1304, 1332, 2): 32.2,
 (1304, 1333, 2): 29.4,
 (1304, 1334, 2): 29.8,
 (1304, 1336, 2): 12.9,
 (1304, 1337, 2): 12.9,
 (1304, 1338, 2): 15.1,
 (1304, 1339, 2): 23.2,
 (1304, 1340, 2): 30.8,
 (1304, 1342, 2): 2.4,
 (1304, 1343, 2): 8.7,
 (1304, 1344, 2): 18.2,
 (1304, 1346, 2): 22.8,
 (1304, 1347, 2): 21.7,
 (1304, 1348, 2): 20.4,
 (1304, 1349, 2): 31.4,
 (1304, 1351, 2): 15.9,
 (1304, 1352, 2): 17.4,
 (1304, 1353, 2): 17.0,
 (1304, 1354, 2): 23.2,
 (1304, 1357, 2): 15.2,
 (1304, 1358, 2): 12.9,
 (1304, 1360, 2): 13.1,
 (1304, 1361, 2): 6.1,
 (1304, 1363, 2): 14.7,
 (1304, 1364, 2): 5.6,
 (1304, 1365, 2): 5.8,
 (1304, 1366, 2): 24.9,
 (1304, 1367, 2): 20.5,
 (1304, 1368, 2): 15.6,
 (1304, 1370, 2): 0.8,
 (1304, 1372, 2): 19.6,
 (1304, 1373, 2): 24.5,
 (1304, 1375, 2): 26.4,
 (1304, 1376, 2): 25.7,
 (1304, 1377, 2): 26.1,
 (1304, 1379, 2): 19.9,
 (1304, 1380, 2): 23.3,
 (1304, 1381, 2): 32.9,
 (1304, 1382, 2): 28.2,
 (1304, 1384, 2): 15.1,
 (1304, 1385, 2): 14.0,
 (1304, 1386, 2): 14.7,
 (1304, 1388, 2): 13.1,
 (1304, 1390, 2): 32.2,
 (1304, 1391, 2): 33.5,
 (1304, 1392, 2): 16.1,
 (1304, 1393, 2): 15.6,
 (1304, 1394, 2): 15.8,
 (1304, 1395, 2): 31.1,
 (1304, 1396, 2): 31.5,
 (1304, 4406, 2): 31.5,
 (1304, 4411, 2): 24.9,
 (1304, 4414, 2): 17.2,
 (1304, 4415, 2): 23.3,
 (1304, 4417, 2): 16.6,
 (1304, 4418, 2): 20.1,
 (1304, 4419, 2): 32.2,
 (1304, 4420, 2): 31.5,
 (1304, 4421, 2): 21.1,
 (1304, 4423, 2): 21.7,
 (1304, 4425, 2): 29.2,
 (1305, 0, 2): 17.7,
 (1305, 1302, 2): 6.8,
 (1305, 1303, 2): 5.6,
 (1305, 1304, 2): 12.8,
 (1305, 1306, 2): 7.3,
 (1305, 1308, 2): 9.9,
 (1305, 1314, 2): 10.0,
 (1305, 1315, 2): 12.2,
 (1305, 1316, 2): 15.8,
 (1305, 1318, 2): 9.5,
 (1305, 1319, 2): 11.3,
 (1305, 1321, 2): 11.4,
 (1305, 1322, 2): 12.2,
 (1305, 1325, 2): 18.2,
 (1305, 1326, 2): 15.2,
 (1305, 1327, 2): 15.2,
 (1305, 1328, 2): 13.5,
 (1305, 1332, 2): 22.0,
 (1305, 1333, 2): 19.0,
 (1305, 1334, 2): 19.5,
 (1305, 1336, 2): 4.1,
 (1305, 1337, 2): 1.9,
 (1305, 1338, 2): 3.6,
 (1305, 1339, 2): 13.3,
 (1305, 1340, 2): 21.0,
 (1305, 1342, 2): 11.3,
 (1305, 1343, 2): 6.5,
 (1305, 1344, 2): 7.8,
 (1305, 1346, 2): 12.4,
 (1305, 1347, 2): 11.3,
 (1305, 1348, 2): 9.9,
 (1305, 1349, 2): 21.5,
 (1305, 1351, 2): 6.2,
 (1305, 1352, 2): 11.4,
 (1305, 1353, 2): 6.8,
 (1305, 1354, 2): 13.7,
 (1305, 1357, 2): 15.8,
 (1305, 1358, 2): 14.0,
 (1305, 1360, 2): 9.4,
 (1305, 1361, 2): 9.0,
 (1305, 1363, 2): 5.0,
 (1305, 1364, 2): 8.4,
 (1305, 1365, 2): 10.3,
 (1305, 1366, 2): 15.0,
 (1305, 1367, 2): 10.2,
 (1305, 1368, 2): 3.6,
 (1305, 1370, 2): 12.1,
 (1305, 1372, 2): 9.7,
 (1305, 1373, 2): 14.1,
 (1305, 1375, 2): 16.1,
 (1305, 1376, 2): 15.9,
 (1305, 1377, 2): 15.8,
 (1305, 1379, 2): 14.0,
 (1305, 1380, 2): 12.8,
 (1305, 1381, 2): 23.2,
 (1305, 1382, 2): 17.8,
 (1305, 1384, 2): 6.1,
 (1305, 1385, 2): 5.5,
 (1305, 1386, 2): 4.4,
 (1305, 1388, 2): 7.5,
 (1305, 1390, 2): 21.2,
 (1305, 1391, 2): 23.2,
 (1305, 1392, 2): 5.9,
 (1305, 1393, 2): 10.1,
 (1305, 1394, 2): 9.6,
 (1305, 1395, 2): 20.7,
 (1305, 1396, 2): 21.1,
 (1305, 4406, 2): 21.2,
 (1305, 4411, 2): 14.6,
 (1305, 4414, 2): 6.9,
 (1305, 4415, 2): 18.9,
 (1305, 4417, 2): 15.7,
 (1305, 4418, 2): 9.8,
 (1305, 4419, 2): 22.3,
 (1305, 4420, 2): 21.6,
 (1305, 4421, 2): 10.3,
 (1305, 4423, 2): 11.0,
 (1305, 4425, 2): 19.0,
 (1306, 0, 2): 22.7,
 (1306, 1302, 2): 4.9,
 (1306, 1303, 2): 9.4,
 (1306, 1304, 2): 9.8,
 (1306, 1305, 2): 7.6,
 (1306, 1308, 2): 14.4,
 (1306, 1314, 2): 12.5,
 (1306, 1315, 2): 14.5,
 (1306, 1316, 2): 18.1,
 (1306, 1318, 2): 11.8,
 (1306, 1319, 2): 13.4,
 (1306, 1321, 2): 16.1,
 (1306, 1322, 2): 14.1,
 (1306, 1325, 2): 20.2,
 (1306, 1326, 2): 17.8,
 (1306, 1327, 2): 17.8,
 (1306, 1328, 2): 16.1,
 (1306, 1332, 2): 24.9,
 (1306, 1333, 2): 22.0,
 (1306, 1334, 2): 22.4,
 (1306, 1336, 2): 8.0,
 (1306, 1337, 2): 8.3,
 (1306, 1338, 2): 10.2,
 (1306, 1339, 2): 15.4,
 (1306, 1340, 2): 23.0,
 (1306, 1342, 2): 8.3,
 (1306, 1343, 2): 5.7,
 (1306, 1344, 2): 10.4,
 (1306, 1346, 2): 14.7,
 (1306, 1347, 2): 13.6,
 (1306, 1348, 2): 12.2,
 (1306, 1349, 2): 24.0,
 (1306, 1351, 2): 11.2,
 (1306, 1352, 2): 12.7,
 (1306, 1353, 2): 9.2,
 (1306, 1354, 2): 15.5,
 (1306, 1357, 2): 18.0,
 (1306, 1358, 2): 16.2,
 (1306, 1360, 2): 11.8,
 (1306, 1361, 2): 6.0,
 (1306, 1363, 2): 10.0,
 (1306, 1364, 2): 5.5,
 (1306, 1365, 2): 7.3,
 (1306, 1366, 2): 19.6,
 (1306, 1367, 2): 12.5,
 (1306, 1368, 2): 10.3,
 (1306, 1370, 2): 9.1,
 (1306, 1372, 2): 14.5,
 (1306, 1373, 2): 16.8,
 (1306, 1375, 2): 18.8,
 (1306, 1376, 2): 20.8,
 (1306, 1377, 2): 18.7,
 (1306, 1379, 2): 16.4,
 (1306, 1380, 2): 15.2,
 (1306, 1381, 2): 25.6,
 (1306, 1382, 2): 20.4,
 (1306, 1384, 2): 8.7,
 (1306, 1385, 2): 6.2,
 (1306, 1386, 2): 6.9,
 (1306, 1388, 2): 4.4,
 (1306, 1390, 2): 26.6,
 (1306, 1391, 2): 24.0,
 (1306, 1392, 2): 8.4,
 (1306, 1393, 2): 12.7,
 (1306, 1394, 2): 12.2,
 (1306, 1395, 2): 23.0,
 (1306, 1396, 2): 23.3,
 (1306, 4406, 2): 23.7,
 (1306, 4411, 2): 17.0,
 (1306, 4414, 2): 12.1,
 (1306, 4415, 2): 21.3,
 (1306, 4417, 2): 18.3,
 (1306, 4418, 2): 12.4,
 (1306, 4419, 2): 24.7,
 (1306, 4420, 2): 24.0,
 (1306, 4421, 2): 13.0,
 (1306, 4423, 2): 13.6,
 (1306, 4425, 2): 21.4,
 (1308, 0, 2): 13.7,
 (1308, 1302, 2): 11.5,
 (1308, 1303, 2): 10.4,
 (1308, 1304, 2): 17.3,
 (1308, 1305, 2): 7.7,
 (1308, 1306, 2): 11.5,
 (1308, 1314, 2): 15.4,
 (1308, 1315, 2): 17.5,
 (1308, 1316, 2): 21.1,
 (1308, 1318, 2): 13.3,
 (1308, 1319, 2): 15.2,
 (1308, 1321, 2): 7.6,
 (1308, 1322, 2): 10.9,
 (1308, 1325, 2): 16.9,
 (1308, 1326, 2): 14.2,
 (1308, 1327, 2): 14.2,
 (1308, 1328, 2): 12.6,
 (1308, 1332, 2): 21.5,
 (1308, 1333, 2): 18.5,
 (1308, 1334, 2): 19.0,
 (1308, 1336, 2): 8.9,
 (1308, 1337, 2): 7.3,
 (1308, 1338, 2): 4.8,
 (1308, 1339, 2): 16.9,
 (1308, 1340, 2): 24.4,
 (1308, 1342, 2): 15.7,
 (1308, 1343, 2): 11.2,
 (1308, 1344, 2): 13.2,
 (1308, 1346, 2): 16.2,
 (1308, 1347, 2): 15.2,
 (1308, 1348, 2): 13.8,
 (1308, 1349, 2): 25.4,
 (1308, 1351, 2): 1.5,
 (1308, 1352, 2): 7.6,
 (1308, 1353, 2): 11.0,
 (1308, 1354, 2): 9.9,
 (1308, 1357, 2): 21.0,
 (1308, 1358, 2): 19.2,
 (1308, 1360, 2): 14.7,
 (1308, 1361, 2): 13.0,
 (1308, 1363, 2): 2.8,
 (1308, 1364, 2): 12.5,
 (1308, 1365, 2): 14.2,
 (1308, 1366, 2): 11.0,
 (1308, 1367, 2): 14.0,
 (1308, 1368, 2): 7.7,
 (1308, 1370, 2): 16.0,
 (1308, 1372, 2): 6.0,
 (1308, 1373, 2): 13.2,
 (1308, 1375, 2): 15.1,
 (1308, 1376, 2): 12.4,
 (1308, 1377, 2): 15.7,
 (1308, 1379, 2): 17.8,
 (1308, 1380, 2): 12.4,
 (1308, 1381, 2): 22.1,
 (1308, 1382, 2): 16.8,
 (1308, 1384, 2): 11.6,
 (1308, 1385, 2): 9.9,
 (1308, 1386, 2): 10.0,
 (1308, 1388, 2): 11.8,
 (1308, 1390, 2): 22.5,
 (1308, 1391, 2): 20.3,
 (1308, 1392, 2): 10.4,
 (1308, 1393, 2): 14.2,
 (1308, 1394, 2): 13.7,
 (1308, 1395, 2): 21.1,
 (1308, 1396, 2): 21.4,
 (1308, 4406, 2): 19.9,
 (1308, 4411, 2): 18.1,
 (1308, 4414, 2): 3.7,
 (1308, 4415, 2): 23.1,
 (1308, 4417, 2): 21.4,
 (1308, 4418, 2): 14.1,
 (1308, 4419, 2): 25.1,
 (1308, 4420, 2): 24.3,
 (1308, 4421, 2): 14.5,
 (1308, 4423, 2): 15.1,
 (1308, 4425, 2): 21.0,
 (1314, 0, 2): 19.2,
 (1314, 1302, 2): 7.2,
 (1314, 1303, 2): 6.3,
 (1314, 1304, 2): 13.0,
 (1314, 1305, 2): 8.8,
 (1314, 1306, 2): 12.8,
 (1314, 1308, 2): 12.4,
 (1314, 1315, 2): 3.7,
 (1314, 1316, 2): 7.4,
 (1314, 1318, 2): 5.8,
 (1314, 1319, 2): 7.1,
 (1314, 1321, 2): 9.8,
 (1314, 1322, 2): 7.9,
 (1314, 1325, 2): 14.3,
 (1314, 1326, 2): 11.5,
 (1314, 1327, 2): 11.6,
 (1314, 1328, 2): 9.8,
 (1314, 1332, 2): 18.9,
 (1314, 1333, 2): 15.9,
 (1314, 1334, 2): 16.4,
 (1314, 1336, 2): 9.9,
 (1314, 1337, 2): 7.8,
 (1314, 1338, 2): 10.3,
 (1314, 1339, 2): 9.8,
 (1314, 1340, 2): 17.4,
 (1314, 1342, 2): 11.5,
 (1314, 1343, 2): 6.9,
 (1314, 1344, 2): 6.2,
 (1314, 1346, 2): 8.9,
 (1314, 1347, 2): 7.8,
 (1314, 1348, 2): 6.4,
 (1314, 1349, 2): 18.4,
 (1314, 1351, 2): 12.9,
 (1314, 1352, 2): 15.4,
 (1314, 1353, 2): 3.5,
 (1314, 1354, 2): 9.2,
 (1314, 1357, 2): 7.2,
 (1314, 1358, 2): 5.4,
 (1314, 1360, 2): 1.0,
 (1314, 1361, 2): 9.3,
 (1314, 1363, 2): 11.6,
 (1314, 1364, 2): 9.0,
 (1314, 1365, 2): 10.5,
 (1314, 1366, 2): 10.7,
 (1314, 1367, 2): 6.6,
 (1314, 1368, 2): 10.0,
 (1314, 1370, 2): 12.2,
 (1314, 1372, 2): 10.3,
 (1314, 1373, 2): 10.4,
 (1314, 1375, 2): 12.3,
 (1314, 1376, 2): 14.6,
 (1314, 1377, 2): 13.0,
 (1314, 1379, 2): 8.5,
 (1314, 1380, 2): 8.9,
 (1314, 1381, 2): 19.5,
 (1314, 1382, 2): 14.0,
 (1314, 1384, 2): 4.9,
 (1314, 1385, 2): 10.8,
 (1314, 1386, 2): 11.6,
 (1314, 1388, 2): 9.4,
 (1314, 1390, 2): 19.2,
 (1314, 1391, 2): 17.7,
 (1314, 1392, 2): 5.9,
 (1314, 1393, 2): 3.5,
 (1314, 1394, 2): 3.7,
 (1314, 1395, 2): 17.6,
 (1314, 1396, 2): 17.9,
 (1314, 4406, 2): 17.4,
 (1314, 4411, 2): 11.3,
 (1314, 4414, 2): 13.9,
 (1314, 4415, 2): 15.0,
 (1314, 4417, 2): 7.6,
 (1314, 4418, 2): 6.0,
 (1314, 4419, 2): 18.5,
 (1314, 4420, 2): 17.7,
 (1314, 4421, 2): 6.1,
 (1314, 4423, 2): 7.7,
 (1314, 4425, 2): 15.8,
 (1315, 0, 2): 20.4,
 (1315, 1302, 2): 11.7,
 (1315, 1303, 2): 10.7,
 (1315, 1304, 2): 13.2,
 (1315, 1305, 2): 13.4,
 (1315, 1306, 2): 17.2,
 (1315, 1308, 2): 16.6,
 (1315, 1314, 2): 5.9,
 (1315, 1316, 2): 4.8,
 (1315, 1318, 2): 10.5,
 (1315, 1319, 2): 11.5,
 (1315, 1321, 2): 13.9,
 (1315, 1322, 2): 12.2,
 (1315, 1325, 2): 18.3,
 (1315, 1326, 2): 15.9,
 (1315, 1327, 2): 15.9,
 (1315, 1328, 2): 14.2,
 (1315, 1332, 2): 23.0,
 (1315, 1333, 2): 20.0,
 (1315, 1334, 2): 20.5,
 (1315, 1336, 2): 14.5,
 (1315, 1337, 2): 12.4,
 (1315, 1338, 2): 14.8,
 (1315, 1339, 2): 14.1,
 (1315, 1340, 2): 21.7,
 (1315, 1342, 2): 14.5,
 (1315, 1343, 2): 11.4,
 (1315, 1344, 2): 11.2,
 (1315, 1346, 2): 12.7,
 (1315, 1347, 2): 12.4,
 (1315, 1348, 2): 10.9,
 (1315, 1349, 2): 22.6,
 (1315, 1351, 2): 17.3,
 (1315, 1352, 2): 19.7,
 (1315, 1353, 2): 7.9,
 (1315, 1354, 2): 13.6,
 (1315, 1357, 2): 6.0,
 (1315, 1358, 2): 4.2,
 (1315, 1360, 2): 6.4,
 (1315, 1361, 2): 13.9,
 (1315, 1363, 2): 16.1,
 (1315, 1364, 2): 13.4,
 (1315, 1365, 2): 13.5,
 (1315, 1366, 2): 15.1,
 (1315, 1367, 2): 11.1,
 (1315, 1368, 2): 11.2,
 (1315, 1370, 2): 14.0,
 (1315, 1372, 2): 14.7,
 (1315, 1373, 2): 14.9,
 (1315, 1375, 2): 16.8,
 (1315, 1376, 2): 18.9,
 (1315, 1377, 2): 16.7,
 (1315, 1379, 2): 11.5,
 (1315, 1380, 2): 13.3,
 (1315, 1381, 2): 23.7,
 (1315, 1382, 2): 18.5,
 (1315, 1384, 2): 9.4,
 (1315, 1385, 2): 15.6,
 (1315, 1386, 2): 16.3,
 (1315, 1388, 2): 17.6,
 (1315, 1390, 2): 23.5,
 (1315, 1391, 2): 22.1,
 (1315, 1392, 2): 10.5,
 (1315, 1393, 2): 7.9,
 (1315, 1394, 2): 8.1,
 (1315, 1395, 2): 21.6,
 (1315, 1396, 2): 21.9,
 (1315, 4406, 2): 21.9,
 (1315, 4411, 2): 15.6,
 (1315, 4414, 2): 18.5,
 (1315, 4415, 2): 15.6,
 (1315, 4417, 2): 7.5,
 (1315, 4418, 2): 10.4,
 (1315, 4419, 2): 22.9,
 (1315, 4420, 2): 22.2,
 (1315, 4421, 2): 10.5,
 (1315, 4423, 2): 12.2,
 (1315, 4425, 2): 19.9,
 (1316, 0, 2): 21.3,
 (1316, 1302, 2): 9.4,
 (1316, 1303, 2): 8.5,
 (1316, 1304, 2): 14.0,
 (1316, 1305, 2): 11.2,
 (1316, 1306, 2): 15.4,
 (1316, 1308, 2): 14.4,
 (1316, 1314, 2): 3.7,
 (1316, 1315, 2): 4.3,
 (1316, 1318, 2): 8.2,
 (1316, 1319, 2): 9.3,
 (1316, 1321, 2): 11.7,
 (1316, 1322, 2): 10.0,
 (1316, 1325, 2): 16.1,
 (1316, 1326, 2): 13.7,
 (1316, 1327, 2): 13.7,
 (1316, 1328, 2): 12.0,
 (1316, 1332, 2): 20.8,
 (1316, 1333, 2): 17.8,
 (1316, 1334, 2): 18.3,
 (1316, 1336, 2): 12.4,
 (1316, 1337, 2): 10.2,
 (1316, 1338, 2): 12.7,
 (1316, 1339, 2): 11.8,
 (1316, 1340, 2): 19.4,
 (1316, 1342, 2): 13.8,
 (1316, 1343, 2): 9.1,
 (1316, 1344, 2): 9.0,
 (1316, 1346, 2): 10.4,
 (1316, 1347, 2): 10.1,
 (1316, 1348, 2): 8.7,
 (1316, 1349, 2): 20.4,
 (1316, 1351, 2): 15.1,
 (1316, 1352, 2): 17.0,
 (1316, 1353, 2): 5.3,
 (1316, 1354, 2): 11.1,
 (1316, 1357, 2): 7.6,
 (1316, 1358, 2): 1.7,
 (1316, 1360, 2): 3.9,
 (1316, 1361, 2): 11.3,
 (1316, 1363, 2): 12.7,
 (1316, 1364, 2): 10.7,
 (1316, 1365, 2): 12.6,
 (1316, 1366, 2): 12.7,
 (1316, 1367, 2): 8.2,
 (1316, 1368, 2): 8.7,
 (1316, 1370, 2): 14.3,
 (1316, 1372, 2): 12.3,
 (1316, 1373, 2): 12.4,
 (1316, 1375, 2): 14.6,
 (1316, 1376, 2): 16.7,
 (1316, 1377, 2): 14.6,
 (1316, 1379, 2): 9.0,
 (1316, 1380, 2): 11.1,
 (1316, 1381, 2): 21.2,
 (1316, 1382, 2): 16.1,
 (1316, 1384, 2): 6.5,
 (1316, 1385, 2): 12.1,
 (1316, 1386, 2): 12.9,
 (1316, 1388, 2): 14.6,
 (1316, 1390, 2): 20.8,
 (1316, 1391, 2): 20.0,
 (1316, 1392, 2): 7.8,
 (1316, 1393, 2): 5.4,
 (1316, 1394, 2): 5.5,
 (1316, 1395, 2): 19.1,
 (1316, 1396, 2): 19.6,
 (1316, 4406, 2): 19.5,
 (1316, 4411, 2): 12.6,
 (1316, 4414, 2): 15.2,
 (1316, 4415, 2): 16.9,
 (1316, 4417, 2): 9.0,
 (1316, 4418, 2): 8.0,
 (1316, 4419, 2): 20.4,
 (1316, 4420, 2): 19.6,
 (1316, 4421, 2): 7.6,
 (1316, 4423, 2): 9.4,
 (1316, 4425, 2): 17.4,
 (1318, 0, 2): 17.9,
 (1318, 1302, 2): 8.1,
 (1318, 1303, 2): 6.9,
 (1318, 1304, 2): 14.1,
 (1318, 1305, 2): 10.7,
 (1318, 1306, 2): 13.2,
 (1318, 1308, 2): 11.0,
 (1318, 1314, 2): 6.2,
 (1318, 1315, 2): 7.3,
 (1318, 1316, 2): 10.9,
 (1318, 1319, 2): 6.1,
 (1318, 1321, 2): 8.6,
 (1318, 1322, 2): 6.8,
 (1318, 1325, 2): 12.1,
 (1318, 1326, 2): 10.4,
 (1318, 1327, 2): 10.5,
 (1318, 1328, 2): 8.7,
 (1318, 1332, 2): 17.5,
 (1318, 1333, 2): 14.6,
 (1318, 1334, 2): 15.1,
 (1318, 1336, 2): 11.0,
 (1318, 1337, 2): 8.9,
 (1318, 1338, 2): 9.5,
 (1318, 1339, 2): 6.3,
 (1318, 1340, 2): 14.4,
 (1318, 1342, 2): 12.6,
 (1318, 1343, 2): 7.7,
 (1318, 1344, 2): 9.1,
 (1318, 1346, 2): 5.8,
 (1318, 1347, 2): 4.7,
 (1318, 1348, 2): 3.2,
 (1318, 1349, 2): 15.0,
 (1318, 1351, 2): 11.9,
 (1318, 1352, 2): 15.6,
 (1318, 1353, 2): 3.5,
 (1318, 1354, 2): 8.1,
 (1318, 1357, 2): 10.8,
 (1318, 1358, 2): 8.9,
 (1318, 1360, 2): 8.0,
 (1318, 1361, 2): 10.3,
 (1318, 1363, 2): 12.9,
 (1318, 1364, 2): 9.8,
 (1318, 1365, 2): 11.6,
 (1318, 1366, 2): 9.6,
 (1318, 1367, 2): 3.4,
 (1318, 1368, 2): 7.0,
 (1318, 1370, 2): 13.4,
 (1318, 1372, 2): 9.3,
 (1318, 1373, 2): 9.4,
 (1318, 1375, 2): 10.2,
 (1318, 1376, 2): 13.7,
 (1318, 1377, 2): 11.5,
 (1318, 1379, 2): 7.4,
 (1318, 1380, 2): 8.1,
 (1318, 1381, 2): 18.2,
 (1318, 1382, 2): 11.8,
 (1318, 1384, 2): 6.5,
 (1318, 1385, 2): 10.4,
 (1318, 1386, 2): 13.1,
 (1318, 1388, 2): 14.1,
 (1318, 1390, 2): 17.8,
 (1318, 1391, 2): 15.3,
 (1318, 1392, 2): 7.0,
 (1318, 1393, 2): 3.7,
 (1318, 1394, 2): 3.1,
 (1318, 1395, 2): 14.2,
 (1318, 1396, 2): 13.8,
 (1318, 4406, 2): 15.2,
 (1318, 4411, 2): 7.7,
 (1318, 4414, 2): 11.1,
 (1318, 4415, 2): 13.3,
 (1318, 4417, 2): 11.0,
 (1318, 4418, 2): 4.9,
 (1318, 4419, 2): 17.4,
 (1318, 4420, 2): 16.6,
 (1318, 4421, 2): 3.8,
 (1318, 4423, 2): 4.5,
 (1318, 4425, 2): 12.6,
 (1319, 0, 2): 22.5,
 (1319, 1302, 2): 17.1,
 (1319, 1303, 2): 16.0,
 (1319, 1304, 2): 15.8,
 (1319, 1305, 2): 17.8,
 (1319, 1306, 2): 22.2,
 (1319, 1308, 2): 19.0,
 (1319, 1314, 2): 11.4,
 (1319, 1315, 2): 9.5,
 (1319, 1316, 2): 7.4,
 (1319, 1318, 2): 15.8,
 (1319, 1321, 2): 17.2,
 (1319, 1322, 2): 15.2,
 (1319, 1325, 2): 20.5,
 (1319, 1326, 2): 18.6,
 (1319, 1327, 2): 18.6,
 (1319, 1328, 2): 16.8,
 (1319, 1332, 2): 25.4,
 (1319, 1333, 2): 22.6,
 (1319, 1334, 2): 23.0,
 (1319, 1336, 2): 19.0,
 (1319, 1337, 2): 16.9,
 (1319, 1338, 2): 19.4,
 (1319, 1339, 2): 19.2,
 (1319, 1340, 2): 26.9,
 (1319, 1342, 2): 17.0,
 (1319, 1343, 2): 16.8,
 (1319, 1344, 2): 16.4,
 (1319, 1346, 2): 18.1,
 (1319, 1347, 2): 17.3,
 (1319, 1348, 2): 15.9,
 (1319, 1349, 2): 27.4,
 (1319, 1351, 2): 22.0,
 (1319, 1352, 2): 26.3,
 (1319, 1353, 2): 13.1,
 (1319, 1354, 2): 16.1,
 (1319, 1357, 2): 12.0,
 (1319, 1358, 2): 9.6,
 (1319, 1360, 2): 11.9,
 (1319, 1361, 2): 19.3,
 (1319, 1363, 2): 20.7,
 (1319, 1364, 2): 18.8,
 (1319, 1365, 2): 16.1,
 (1319, 1366, 2): 21.4,
 (1319, 1367, 2): 16.1,
 (1319, 1368, 2): 16.4,
 (1319, 1370, 2): 16.7,
 (1319, 1372, 2): 17.3,
 (1319, 1373, 2): 17.5,
 (1319, 1375, 2): 19.6,
 (1319, 1376, 2): 21.6,
 (1319, 1377, 2): 19.4,
 (1319, 1379, 2): 16.7,
 (1319, 1380, 2): 16.3,
 (1319, 1381, 2): 26.0,
 (1319, 1382, 2): 21.2,
 (1319, 1384, 2): 14.6,
 (1319, 1385, 2): 20.0,
 (1319, 1386, 2): 20.8,
 (1319, 1388, 2): 23.0,
 (1319, 1390, 2): 25.8,
 (1319, 1391, 2): 24.9,
 (1319, 1392, 2): 15.7,
 (1319, 1393, 2): 13.2,
 (1319, 1394, 2): 13.4,
 (1319, 1395, 2): 25.8,
 (1319, 1396, 2): 26.2,
 (1319, 4406, 2): 25.5,
 (1319, 4411, 2): 20.0,
 (1319, 4414, 2): 23.0,
 (1319, 4415, 2): 21.4,
 (1319, 4417, 2): 13.3,
 (1319, 4418, 2): 13.0,
 (1319, 4419, 2): 25.4,
 (1319, 4420, 2): 24.6,
 (1319, 4421, 2): 15.4,
 (1319, 4423, 2): 17.2,
 (1319, 4425, 2): 26.6,
 (1321, 0, 2): 13.0,
 (1321, 1302, 2): 13.2,
 (1321, 1303, 2): 12.0,
 (1321, 1304, 2): 18.3,
 (1321, 1305, 2): 9.4,
 (1321, 1306, 2): 13.0,
 (1321, 1308, 2): 4.2,
 (1321, 1314, 2): 10.1,
 (1321, 1315, 2): 11.2,
 (1321, 1316, 2): 11.6,
 (1321, 1318, 2): 6.5,
 (1321, 1319, 2): 5.6,
 (1321, 1322, 2): 3.4,
 (1321, 1325, 2): 8.0,
 (1321, 1326, 2): 6.3,
 (1321, 1327, 2): 6.4,
 (1321, 1328, 2): 4.6,
 (1321, 1332, 2): 13.3,
 (1321, 1333, 2): 10.5,
 (1321, 1334, 2): 11.0,
 (1321, 1336, 2): 10.4,
 (1321, 1337, 2): 9.0,
 (1321, 1338, 2): 6.5,
 (1321, 1339, 2): 8.3,
 (1321, 1340, 2): 15.4,
 (1321, 1342, 2): 16.8,
 (1321, 1343, 2): 12.9,
 (1321, 1344, 2): 14.6,
 (1321, 1346, 2): 10.7,
 (1321, 1347, 2): 9.6,
 (1321, 1348, 2): 8.2,
 (1321, 1349, 2): 16.0,
 (1321, 1351, 2): 5.0,
 (1321, 1352, 2): 11.2,
 (1321, 1353, 2): 7.2,
 (1321, 1354, 2): 1.2,
 (1321, 1357, 2): 14.6,
 (1321, 1358, 2): 12.8,
 (1321, 1360, 2): 10.6,
 (1321, 1361, 2): 14.5,
 (1321, 1363, 2): 6.5,
 (1321, 1364, 2): 14.0,
 (1321, 1365, 2): 15.8,
 (1321, 1366, 2): 3.5,
 (1321, 1367, 2): 8.4,
 (1321, 1368, 2): 9.4,
 (1321, 1370, 2): 17.6,
 (1321, 1372, 2): 2.4,
 (1321, 1373, 2): 5.2,
 (1321, 1375, 2): 7.4,
 (1321, 1376, 2): 6.9,
 (1321, 1377, 2): 7.4,
 (1321, 1379, 2): 13.4,
 (1321, 1380, 2): 4.4,
 (1321, 1381, 2): 14.0,
 (1321, 1382, 2): 8.9,
 (1321, 1384, 2): 11.6,
 (1321, 1385, 2): 11.5,
 (1321, 1386, 2): 11.5,
 (1321, 1388, 2): 14.0,
 (1321, 1390, 2): 15.2,
 (1321, 1391, 2): 12.5,
 (1321, 1392, 2): 5.8,
 (1321, 1393, 2): 8.7,
 (1321, 1394, 2): 6.8,
 (1321, 1395, 2): 13.6,
 (1321, 1396, 2): 14.0,
 (1321, 4406, 2): 13.1,
 (1321, 4411, 2): 7.8,
 (1321, 4414, 2): 6.5,
 (1321, 4415, 2): 16.4,
 (1321, 4417, 2): 14.8,
 (1321, 4418, 2): 4.3,
 (1321, 4419, 2): 13.2,
 (1321, 4420, 2): 12.5,
 (1321, 4421, 2): 9.1,
 (1321, 4423, 2): 8.3,
 (1321, 4425, 2): 13.6,
 (1322, 0, 2): 15.0,
 (1322, 1302, 2): 15.3,
 (1322, 1303, 2): 14.0,
 (1322, 1304, 2): 20.3,
 (1322, 1305, 2): 11.5,
 (1322, 1306, 2): 15.1,
 (1322, 1308, 2): 6.2,
 (1322, 1314, 2): 12.2,
 (1322, 1315, 2): 13.3,
 (1322, 1316, 2): 13.6,
 (1322, 1318, 2): 8.5,
 (1322, 1319, 2): 5.8,
 (1322, 1321, 2): 2.0,
 (1322, 1325, 2): 7.1,
 (1322, 1326, 2): 5.4,
 (1322, 1327, 2): 5.5,
 (1322, 1328, 2): 3.7,
 (1322, 1332, 2): 12.4,
 (1322, 1333, 2): 9.6,
 (1322, 1334, 2): 10.1,
 (1322, 1336, 2): 12.5,
 (1322, 1337, 2): 11.1,
 (1322, 1338, 2): 8.6,
 (1322, 1339, 2): 10.5,
 (1322, 1340, 2): 14.5,
 (1322, 1342, 2): 18.8,
 (1322, 1343, 2): 15.0,
 (1322, 1344, 2): 16.8,
 (1322, 1346, 2): 12.8,
 (1322, 1347, 2): 11.7,
 (1322, 1348, 2): 10.3,
 (1322, 1349, 2): 15.2,
 (1322, 1351, 2): 7.1,
 (1322, 1352, 2): 13.3,
 (1322, 1353, 2): 9.3,
 (1322, 1354, 2): 3.2,
 (1322, 1357, 2): 16.8,
 (1322, 1358, 2): 14.9,
 (1322, 1360, 2): 12.8,
 (1322, 1361, 2): 16.6,
 (1322, 1363, 2): 8.6,
 (1322, 1364, 2): 16.1,
 (1322, 1365, 2): 17.8,
 (1322, 1366, 2): 2.6,
 (1322, 1367, 2): 10.5,
 (1322, 1368, 2): 11.6,
 (1322, 1370, 2): 19.6,
 (1322, 1372, 2): 4.5,
 (1322, 1373, 2): 4.4,
 (1322, 1375, 2): 6.5,
 (1322, 1376, 2): 8.9,
 (1322, 1377, 2): 6.5,
 (1322, 1379, 2): 14.4,
 (1322, 1380, 2): 3.5,
 (1322, 1381, 2): 13.2,
 (1322, 1382, 2): 8.0,
 (1322, 1384, 2): 13.6,
 (1322, 1385, 2): 13.5,
 (1322, 1386, 2): 13.6,
 (1322, 1388, 2): 16.0,
 (1322, 1390, 2): 17.3,
 (1322, 1391, 2): 11.6,
 (1322, 1392, 2): 8.1,
 (1322, 1393, 2): 11.3,
 (1322, 1394, 2): 9.0,
 (1322, 1395, 2): 13.0,
 (1322, 1396, 2): 13.3,
 (1322, 4406, 2): 11.8,
 (1322, 4411, 2): 7.3,
 (1322, 4414, 2): 8.6,
 (1322, 4415, 2): 18.2,
 (1322, 4417, 2): 17.1,
 (1322, 4418, 2): 6.6,
 (1322, 4419, 2): 12.2,
 (1322, 4420, 2): 11.4,
 (1322, 4421, 2): 11.6,
 (1322, 4423, 2): 10.9,
 (1322, 4425, 2): 12.9,
 (1325, 0, 2): 17.8,
 (1325, 1302, 2): 28.1,
 (1325, 1303, 2): 26.8,
 (1325, 1304, 2): 31.2,
 (1325, 1305, 2): 26.0,
 (1325, 1306, 2): 29.4,
 (1325, 1308, 2): 20.7,
 (1325, 1314, 2): 24.5,
 (1325, 1315, 2): 25.6,
 (1325, 1316, 2): 22.6,
 (1325, 1318, 2): 20.8,
 (1325, 1319, 2): 18.2,
 (1325, 1321, 2): 19.6,
 (1325, 1322, 2): 20.6,
 (1325, 1326, 2): 14.6,
 (1325, 1327, 2): 15.6,
 (1325, 1328, 2): 16.3,
 (1325, 1332, 2): 11.1,
 (1325, 1333, 2): 12.5,
 (1325, 1334, 2): 13.1,
 (1325, 1336, 2): 26.9,
 (1325, 1337, 2): 25.7,
 (1325, 1338, 2): 23.1,
 (1325, 1339, 2): 22.1,
 (1325, 1340, 2): 15.5,
 (1325, 1342, 2): 33.4,
 (1325, 1343, 2): 28.1,
 (1325, 1344, 2): 29.3,
 (1325, 1346, 2): 25.8,
 (1325, 1347, 2): 24.7,
 (1325, 1348, 2): 23.2,
 (1325, 1349, 2): 15.5,
 (1325, 1351, 2): 21.7,
 (1325, 1352, 2): 26.3,
 (1325, 1353, 2): 21.7,
 (1325, 1354, 2): 22.3,
 (1325, 1357, 2): 27.1,
 (1325, 1358, 2): 24.8,
 (1325, 1360, 2): 25.1,
 (1325, 1361, 2): 30.3,
 (1325, 1363, 2): 23.0,
 (1325, 1364, 2): 29.7,
 (1325, 1365, 2): 31.6,
 (1325, 1366, 2): 23.1,
 (1325, 1367, 2): 23.3,
 (1325, 1368, 2): 26.4,
 (1325, 1370, 2): 33.4,
 (1325, 1372, 2): 19.1,
 (1325, 1373, 2): 17.6,
 (1325, 1375, 2): 18.6,
 (1325, 1376, 2): 20.4,
 (1325, 1377, 2): 15.9,
 (1325, 1379, 2): 27.4,
 (1325, 1380, 2): 20.5,
 (1325, 1381, 2): 10.0,
 (1325, 1382, 2): 20.3,
 (1325, 1384, 2): 26.6,
 (1325, 1385, 2): 27.9,
 (1325, 1386, 2): 28.3,
 (1325, 1388, 2): 30.2,
 (1325, 1390, 2): 15.8,
 (1325, 1391, 2): 13.7,
 (1325, 1392, 2): 25.2,
 (1325, 1393, 2): 23.7,
 (1325, 1394, 2): 21.3,
 (1325, 1395, 2): 16.9,
 (1325, 1396, 2): 15.6,
 (1325, 4406, 2): 16.8,
 (1325, 4411, 2): 19.4,
 (1325, 4414, 2): 23.6,
 (1325, 4415, 2): 30.7,
 (1325, 4417, 2): 29.2,
 (1325, 4418, 2): 19.4,
 (1325, 4419, 2): 17.5,
 (1325, 4420, 2): 16.7,
 (1325, 4421, 2): 24.5,
 (1325, 4423, 2): 23.8,
 (1325, 4425, 2): 17.8,
 (1326, 0, 2): 16.4,
 (1326, 1302, 2): 13.9,
 (1326, 1303, 2): 12.4,
 (1326, 1304, 2): 19.8,
 (1326, 1305, 2): 15.6,
 (1326, 1306, 2): 18.0,
 (1326, 1308, 2): 10.8,
 (1326, 1314, 2): 10.2,
 (1326, 1315, 2): 11.3,
 (1326, 1316, 2): 11.4,
 (1326, 1318, 2): 6.4,
 (1326, 1319, 2): 3.6,
 (1326, 1321, 2): 8.0,
 (1326, 1322, 2): 6.0,
 (1326, 1325, 2): 3.6,
 (1326, 1327, 2): 3.1,
 (1326, 1328, 2): 1.9,
 (1326, 1332, 2): 9.9,
 (1326, 1333, 2): 7.1,
 (1326, 1334, 2): 7.6,
 (1326, 1336, 2): 16.8,
 (1326, 1337, 2): 13.9,
 (1326, 1338, 2): 13.0,
 (1326, 1339, 2): 7.6,
 (1326, 1340, 2): 12.2,
 (1326, 1342, 2): 18.3,
 (1326, 1343, 2): 13.6,
 (1326, 1344, 2): 14.9,
 (1326, 1346, 2): 11.2,
 (1326, 1347, 2): 10.1,
 (1326, 1348, 2): 8.6,
 (1326, 1349, 2): 12.6,
 (1326, 1351, 2): 11.7,
 (1326, 1352, 2): 18.1,
 (1326, 1353, 2): 7.3,
 (1326, 1354, 2): 7.9,
 (1326, 1357, 2): 14.9,
 (1326, 1358, 2): 13.0,
 (1326, 1360, 2): 10.8,
 (1326, 1361, 2): 16.1,
 (1326, 1363, 2): 13.0,
 (1326, 1364, 2): 15.6,
 (1326, 1365, 2): 17.4,
 (1326, 1366, 2): 8.9,
 (1326, 1367, 2): 8.8,
 (1326, 1368, 2): 12.5,
 (1326, 1370, 2): 19.1,
 (1326, 1372, 2): 9.1,
 (1326, 1373, 2): 3.1,
 (1326, 1375, 2): 4.1,
 (1326, 1376, 2): 10.1,
 (1326, 1377, 2): 0.8,
 (1326, 1379, 2): 13.6,
 (1326, 1380, 2): 3.4,
 (1326, 1381, 2): 10.6,
 (1326, 1382, 2): 5.7,
 (1326, 1384, 2): 11.9,
 (1326, 1385, 2): 15.5,
 (1326, 1386, 2): 16.2,
 (1326, 1388, 2): 18.8,
 (1326, 1390, 2): 10.2,
 (1326, 1391, 2): 9.0,
 (1326, 1392, 2): 10.8,
 (1326, 1393, 2): 9.2,
 (1326, 1394, 2): 6.9,
 (1326, 1395, 2): 10.5,
 (1326, 1396, 2): 10.8,
 (1326, 4406, 2): 9.3,
 (1326, 4411, 2): 4.9,
 (1326, 4414, 2): 13.2,
 (1326, 4415, 2): 16.3,
 (1326, 4417, 2): 15.0,
 (1326, 4418, 2): 4.8,
 (1326, 4419, 2): 9.7,
 (1326, 4420, 2): 8.9,
 (1326, 4421, 2): 8.0,
 (1326, 4423, 2): 9.0,
 (1326, 4425, 2): 10.4,
 (1327, 0, 2): 20.5,
 (1327, 1302, 2): 16.8,
 (1327, 1303, 2): 15.4,
 (1327, 1304, 2): 22.7,
 (1327, 1305, 2): 18.9,
 (1327, 1306, 2): 21.2,
 (1327, 1308, 2): 13.7,
 (1327, 1314, 2): 13.1,
 (1327, 1315, 2): 14.2,
 (1327, 1316, 2): 14.3,
 (1327, 1318, 2): 9.4,
 (1327, 1319, 2): 6.6,
 (1327, 1321, 2): 10.9,
 (1327, 1322, 2): 9.0,
 (1327, 1325, 2): 6.7,
 (1327, 1326, 2): 2.8,
 (1327, 1328, 2): 4.8,
 (1327, 1332, 2): 6.8,
 (1327, 1333, 2): 4.0,
 (1327, 1334, 2): 4.4,
 (1327, 1336, 2): 19.0,
 (1327, 1337, 2): 16.8,
 (1327, 1338, 2): 15.9,
 (1327, 1339, 2): 10.5,
 (1327, 1340, 2): 9.4,
 (1327, 1342, 2): 21.2,
 (1327, 1343, 2): 16.4,
 (1327, 1344, 2): 17.8,
 (1327, 1346, 2): 14.2,
 (1327, 1347, 2): 13.0,
 (1327, 1348, 2): 11.5,
 (1327, 1349, 2): 10.1,
 (1327, 1351, 2): 14.6,
 (1327, 1352, 2): 21.0,
 (1327, 1353, 2): 10.1,
 (1327, 1354, 2): 10.8,
 (1327, 1357, 2): 17.6,
 (1327, 1358, 2): 15.9,
 (1327, 1360, 2): 13.7,
 (1327, 1361, 2): 19.0,
 (1327, 1363, 2): 16.0,
 (1327, 1364, 2): 18.5,
 (1327, 1365, 2): 20.2,
 (1327, 1366, 2): 11.8,
 (1327, 1367, 2): 11.8,
 (1327, 1368, 2): 15.4,
 (1327, 1370, 2): 22.0,
 (1327, 1372, 2): 12.0,
 (1327, 1373, 2): 6.0,
 (1327, 1375, 2): 7.0,
 (1327, 1376, 2): 13.4,
 (1327, 1377, 2): 4.1,
 (1327, 1379, 2): 16.5,
 (1327, 1380, 2): 6.7,
 (1327, 1381, 2): 7.5,
 (1327, 1382, 2): 8.6,
 (1327, 1384, 2): 15.0,
 (1327, 1385, 2): 18.5,
 (1327, 1386, 2): 19.2,
 (1327, 1388, 2): 22.2,
 (1327, 1390, 2): 7.2,
 (1327, 1391, 2): 6.0,
 (1327, 1392, 2): 13.8,
 (1327, 1393, 2): 12.1,
 (1327, 1394, 2): 9.9,
 (1327, 1395, 2): 7.8,
 (1327, 1396, 2): 8.2,
 (1327, 4406, 2): 6.5,
 (1327, 4411, 2): 7.8,
 (1327, 4414, 2): 13.8,
 (1327, 4415, 2): 18.9,
 (1327, 4417, 2): 17.8,
 (1327, 4418, 2): 7.7,
 (1327, 4419, 2): 6.6,
 (1327, 4420, 2): 5.8,
 (1327, 4421, 2): 11.0,
 (1327, 4423, 2): 11.8,
 (1327, 4425, 2): 7.6,
 (1328, 0, 2): 16.7,
 (1328, 1302, 2): 14.2,
 (1328, 1303, 2): 12.7,
 (1328, 1304, 2): 20.0,
 (1328, 1305, 2): 16.2,
 (1328, 1306, 2): 18.5,
 (1328, 1308, 2): 11.1,
 (1328, 1314, 2): 10.4,
 (1328, 1315, 2): 11.5,
 (1328, 1316, 2): 11.7,
 (1328, 1318, 2): 6.7,
 (1328, 1319, 2): 3.9,
 (1328, 1321, 2): 8.3,
 (1328, 1322, 2): 6.4,
 (1328, 1325, 2): 5.9,
 (1328, 1326, 2): 4.5,
 (1328, 1327, 2): 5.2,
 (1328, 1332, 2): 11.9,
 (1328, 1333, 2): 9.1,
 (1328, 1334, 2): 9.6,
 (1328, 1336, 2): 16.4,
 (1328, 1337, 2): 14.2,
 (1328, 1338, 2): 13.2,
 (1328, 1339, 2): 7.9,
 (1328, 1340, 2): 12.0,
 (1328, 1342, 2): 18.6,
 (1328, 1343, 2): 13.8,
 (1328, 1344, 2): 15.2,
 (1328, 1346, 2): 11.6,
 (1328, 1347, 2): 10.4,
 (1328, 1348, 2): 8.9,
 (1328, 1349, 2): 12.6,
 (1328, 1351, 2): 12.0,
 (1328, 1352, 2): 18.5,
 (1328, 1353, 2): 7.5,
 (1328, 1354, 2): 8.2,
 (1328, 1357, 2): 15.0,
 (1328, 1358, 2): 13.2,
 (1328, 1360, 2): 11.0,
 (1328, 1361, 2): 16.3,
 (1328, 1363, 2): 13.2,
 (1328, 1364, 2): 15.8,
 (1328, 1365, 2): 17.6,
 (1328, 1366, 2): 9.2,
 (1328, 1367, 2): 9.2,
 (1328, 1368, 2): 12.8,
 (1328, 1370, 2): 19.3,
 (1328, 1372, 2): 9.4,
 (1328, 1373, 2): 3.4,
 (1328, 1375, 2): 3.6,
 (1328, 1376, 2): 13.4,
 (1328, 1377, 2): 5.9,
 (1328, 1379, 2): 13.9,
 (1328, 1380, 2): 6.3,
 (1328, 1381, 2): 12.6,
 (1328, 1382, 2): 5.2,
 (1328, 1384, 2): 12.3,
 (1328, 1385, 2): 15.8,
 (1328, 1386, 2): 16.5,
 (1328, 1388, 2): 19.5,
 (1328, 1390, 2): 12.3,
 (1328, 1391, 2): 11.5,
 (1328, 1392, 2): 11.0,
 (1328, 1393, 2): 9.5,
 (1328, 1394, 2): 7.1,
 (1328, 1395, 2): 10.4,
 (1328, 1396, 2): 10.8,
 (1328, 4406, 2): 9.3,
 (1328, 4411, 2): 5.2,
 (1328, 4414, 2): 13.7,
 (1328, 4415, 2): 16.5,
 (1328, 4417, 2): 15.1,
 (1328, 4418, 2): 5.1,
 (1328, 4419, 2): 11.8,
 (1328, 4420, 2): 11.0,
 (1328, 4421, 2): 10.2,
 (1328, 4423, 2): 9.4,
 (1328, 4425, 2): 10.4,
 (1332, 0, 2): 17.7,
 (1332, 1302, 2): 21.4,
 (1332, 1303, 2): 20.0,
 (1332, 1304, 2): 27.2,
 (1332, 1305, 2): 23.4,
 (1332, 1306, 2): 25.7,
 (1332, 1308, 2): 18.4,
 (1332, 1314, 2): 17.6,
 (1332, 1315, 2): 18.6,
 (1332, 1316, 2): 18.9,
 (1332, 1318, 2): 13.8,
 (1332, 1319, 2): 11.1,
 (1332, 1321, 2): 15.6,
 (1332, 1322, 2): 13.6,
 (1332, 1325, 2): 11.1,
 (1332, 1326, 2): 7.5,
 (1332, 1327, 2): 8.5,
 (1332, 1328, 2): 9.4,
 (1332, 1333, 2): 3.2,
 (1332, 1334, 2): 4.1,
 (1332, 1336, 2): 23.6,
 (1332, 1337, 2): 21.4,
 (1332, 1338, 2): 20.7,
 (1332, 1339, 2): 15.1,
 (1332, 1340, 2): 8.5,
 (1332, 1342, 2): 25.8,
 (1332, 1343, 2): 21.0,
 (1332, 1344, 2): 22.4,
 (1332, 1346, 2): 18.8,
 (1332, 1347, 2): 17.6,
 (1332, 1348, 2): 16.5,
 (1332, 1349, 2): 8.3,
 (1332, 1351, 2): 19.5,
 (1332, 1352, 2): 25.4,
 (1332, 1353, 2): 14.8,
 (1332, 1354, 2): 15.6,
 (1332, 1357, 2): 22.5,
 (1332, 1358, 2): 20.6,
 (1332, 1360, 2): 18.6,
 (1332, 1361, 2): 23.7,
 (1332, 1363, 2): 21.0,
 (1332, 1364, 2): 23.2,
 (1332, 1365, 2): 25.0,
 (1332, 1366, 2): 16.7,
 (1332, 1367, 2): 16.8,
 (1332, 1368, 2): 20.6,
 (1332, 1370, 2): 26.7,
 (1332, 1372, 2): 16.9,
 (1332, 1373, 2): 10.7,
 (1332, 1375, 2): 11.7,
 (1332, 1376, 2): 18.7,
 (1332, 1377, 2): 8.9,
 (1332, 1379, 2): 21.7,
 (1332, 1380, 2): 11.5,
 (1332, 1381, 2): 2.4,
 (1332, 1382, 2): 13.4,
 (1332, 1384, 2): 19.8,
 (1332, 1385, 2): 23.2,
 (1332, 1386, 2): 25.9,
 (1332, 1388, 2): 26.5,
 (1332, 1390, 2): 5.7,
 (1332, 1391, 2): 6.7,
 (1332, 1392, 2): 18.5,
 (1332, 1393, 2): 17.1,
 (1332, 1394, 2): 14.5,
 (1332, 1395, 2): 9.8,
 (1332, 1396, 2): 8.5,
 (1332, 4406, 2): 9.7,
 (1332, 4411, 2): 12.5,
 (1332, 4414, 2): 18.8,
 (1332, 4415, 2): 23.6,
 (1332, 4417, 2): 22.6,
 (1332, 4418, 2): 12.4,
 (1332, 4419, 2): 8.1,
 (1332, 4420, 2): 7.3,
 (1332, 4421, 2): 17.4,
 (1332, 4423, 2): 16.7,
 (1332, 4425, 2): 10.7,
 (1333, 0, 2): 21.6,
 (1333, 1302, 2): 18.1,
 (1333, 1303, 2): 16.9,
 (1333, 1304, 2): 24.0,
 (1333, 1305, 2): 20.3,
 (1333, 1306, 2): 22.5,
 (1333, 1308, 2): 15.2,
 (1333, 1314, 2): 14.6,
 (1333, 1315, 2): 15.5,
 (1333, 1316, 2): 15.5,
 (1333, 1318, 2): 10.6,
 (1333, 1319, 2): 7.9,
 (1333, 1321, 2): 12.4,
 (1333, 1322, 2): 10.4,
 (1333, 1325, 2): 7.4,
 (1333, 1326, 2): 4.3,
 (1333, 1327, 2): 5.3,
 (1333, 1328, 2): 6.2,
 (1333, 1332, 2): 6.7,
 (1333, 1334, 2): 4.3,
 (1333, 1336, 2): 21.5,
 (1333, 1337, 2): 18.3,
 (1333, 1338, 2): 17.7,
 (1333, 1339, 2): 11.8,
 (1333, 1340, 2): 9.5,
 (1333, 1342, 2): 22.5,
 (1333, 1343, 2): 17.8,
 (1333, 1344, 2): 19.7,
 (1333, 1346, 2): 15.7,
 (1333, 1347, 2): 14.6,
 (1333, 1348, 2): 13.2,
 (1333, 1349, 2): 9.0,
 (1333, 1351, 2): 16.1,
 (1333, 1352, 2): 22.6,
 (1333, 1353, 2): 11.4,
 (1333, 1354, 2): 12.2,
 (1333, 1357, 2): 19.1,
 (1333, 1358, 2): 17.3,
 (1333, 1360, 2): 15.2,
 (1333, 1361, 2): 20.3,
 (1333, 1363, 2): 17.6,
 (1333, 1364, 2): 19.8,
 (1333, 1365, 2): 21.6,
 (1333, 1366, 2): 13.4,
 (1333, 1367, 2): 13.4,
 (1333, 1368, 2): 17.2,
 (1333, 1370, 2): 23.3,
 (1333, 1372, 2): 13.6,
 (1333, 1373, 2): 7.4,
 (1333, 1375, 2): 8.4,
 (1333, 1376, 2): 15.4,
 (1333, 1377, 2): 5.6,
 (1333, 1379, 2): 17.3,
 (1333, 1380, 2): 8.2,
 (1333, 1381, 2): 7.4,
 (1333, 1382, 2): 10.1,
 (1333, 1384, 2): 16.4,
 (1333, 1385, 2): 19.8,
 (1333, 1386, 2): 22.5,
 (1333, 1388, 2): 23.2,
 (1333, 1390, 2): 9.3,
 (1333, 1391, 2): 5.6,
 (1333, 1392, 2): 15.2,
 (1333, 1393, 2): 13.7,
 (1333, 1394, 2): 11.1,
 (1333, 1395, 2): 7.5,
 (1333, 1396, 2): 8.0,
 (1333, 4406, 2): 6.1,
 (1333, 4411, 2): 9.2,
 (1333, 4414, 2): 15.4,
 (1333, 4415, 2): 20.2,
 (1333, 4417, 2): 22.6,
 (1333, 4418, 2): 9.0,
 (1333, 4419, 2): 5.2,
 (1333, 4420, 2): 4.4,
 (1333, 4421, 2): 14.0,
 (1333, 4423, 2): 13.3,
 (1333, 4425, 2): 7.2,
 (1334, 0, 2): 14.8,
 (1334, 1302, 2): 21.7,
 (1334, 1303, 2): 20.5,
 (1334, 1304, 2): 27.6,
 (1334, 1305, 2): 23.8,
 (1334, 1306, 2): 26.0,
 (1334, 1308, 2): 18.8,
 (1334, 1314, 2): 18.2,
 (1334, 1315, 2): 19.1,
 (1334, 1316, 2): 19.1,
 (1334, 1318, 2): 14.2,
 (1334, 1319, 2): 11.4,
 (1334, 1321, 2): 15.9,
 (1334, 1322, 2): 13.9,
 (1334, 1325, 2): 11.2,
 (1334, 1326, 2): 7.8,
 (1334, 1327, 2): 8.7,
 (1334, 1328, 2): 9.6,
 (1334, 1332, 2): 2.4,
 (1334, 1333, 2): 5.6,
 (1334, 1336, 2): 23.9,
 (1334, 1337, 2): 21.8,
 (1334, 1338, 2): 20.1,
 (1334, 1339, 2): 15.2,
 (1334, 1340, 2): 8.9,
 (1334, 1342, 2): 26.1,
 (1334, 1343, 2): 21.4,
 (1334, 1344, 2): 23.1,
 (1334, 1346, 2): 19.2,
 (1334, 1347, 2): 18.1,
 (1334, 1348, 2): 16.7,
 (1334, 1349, 2): 8.4,
 (1334, 1351, 2): 19.7,
 (1334, 1352, 2): 23.6,
 (1334, 1353, 2): 15.0,
 (1334, 1354, 2): 15.8,
 (1334, 1357, 2): 22.6,
 (1334, 1358, 2): 20.8,
 (1334, 1360, 2): 18.8,
 (1334, 1361, 2): 23.8,
 (1334, 1363, 2): 20.0,
 (1334, 1364, 2): 23.3,
 (1334, 1365, 2): 25.1,
 (1334, 1366, 2): 16.9,
 (1334, 1367, 2): 16.9,
 (1334, 1368, 2): 20.8,
 (1334, 1370, 2): 26.8,
 (1334, 1372, 2): 16.0,
 (1334, 1373, 2): 10.9,
 (1334, 1375, 2): 11.8,
 (1334, 1376, 2): 18.8,
 (1334, 1377, 2): 9.1,
 (1334, 1379, 2): 20.8,
 (1334, 1380, 2): 11.7,
 (1334, 1381, 2): 3.0,
 (1334, 1382, 2): 13.5,
 (1334, 1384, 2): 20.0,
 (1334, 1385, 2): 23.3,
 (1334, 1386, 2): 26.0,
 (1334, 1388, 2): 26.6,
 (1334, 1390, 2): 8.7,
 (1334, 1391, 2): 6.8,
 (1334, 1392, 2): 18.7,
 (1334, 1393, 2): 17.2,
 (1334, 1394, 2): 14.6,
 (1334, 1395, 2): 10.0,
 (1334, 1396, 2): 8.7,
 (1334, 4406, 2): 9.9,
 (1334, 4411, 2): 12.6,
 (1334, 4414, 2): 20.5,
 (1334, 4415, 2): 23.7,
 (1334, 4417, 2): 22.8,
 (1334, 4418, 2): 12.6,
 (1334, 4419, 2): 10.6,
 (1334, 4420, 2): 9.9,
 (1334, 4421, 2): 17.4,
 (1334, 4423, 2): 16.8,
 (1334, 4425, 2): 11.0,
 (1336, 0, 2): 20.6,
 (1336, 1302, 2): 3.0,
 (1336, 1303, 2): 1.5,
 (1336, 1304, 2): 8.8,
 (1336, 1305, 2): 3.8,
 (1336, 1306, 2): 7.6,
 (1336, 1308, 2): 12.1,
 (1336, 1314, 2): 7.4,
 (1336, 1315, 2): 10.6,
 (1336, 1316, 2): 14.0,
 (1336, 1318, 2): 6.4,
 (1336, 1319, 2): 8.4,
 (1336, 1321, 2): 11.2,
 (1336, 1322, 2): 9.2,
 (1336, 1325, 2): 13.8,
 (1336, 1326, 2): 12.4,
 (1336, 1327, 2): 12.3,
 (1336, 1328, 2): 10.8,
 (1336, 1332, 2): 19.0,
 (1336, 1333, 2): 16.1,
 (1336, 1334, 2): 16.6,
 (1336, 1337, 2): 2.9,
 (1336, 1338, 2): 5.4,
 (1336, 1339, 2): 9.6,
 (1336, 1340, 2): 17.1,
 (1336, 1342, 2): 7.2,
 (1336, 1343, 2): 2.7,
 (1336, 1344, 2): 6.0,
 (1336, 1346, 2): 9.4,
 (1336, 1347, 2): 8.4,
 (1336, 1348, 2): 6.9,
 (1336, 1349, 2): 18.4,
 (1336, 1351, 2): 7.8,
 (1336, 1352, 2): 11.3,
 (1336, 1353, 2): 4.1,
 (1336, 1354, 2): 10.2,
 (1336, 1357, 2): 14.1,
 (1336, 1358, 2): 12.3,
 (1336, 1360, 2): 7.8,
 (1336, 1361, 2): 5.0,
 (1336, 1363, 2): 6.5,
 (1336, 1364, 2): 4.6,
 (1336, 1365, 2): 6.2,
 (1336, 1366, 2): 12.2,
 (1336, 1367, 2): 7.2,
 (1336, 1368, 2): 5.2,
 (1336, 1370, 2): 8.0,
 (1336, 1372, 2): 11.5,
 (1336, 1373, 2): 11.5,
 (1336, 1375, 2): 13.3,
 (1336, 1376, 2): 15.4,
 (1336, 1377, 2): 13.3,
 (1336, 1379, 2): 11.1,
 (1336, 1380, 2): 10.3,
 (1336, 1381, 2): 19.7,
 (1336, 1382, 2): 15.0,
 (1336, 1384, 2): 3.5,
 (1336, 1385, 2): 1.1,
 (1336, 1386, 2): 1.8,
 (1336, 1388, 2): 8.3,
 (1336, 1390, 2): 19.4,
 (1336, 1391, 2): 18.3,
 (1336, 1392, 2): 3.4,
 (1336, 1393, 2): 7.4,
 (1336, 1394, 2): 6.8,
 (1336, 1395, 2): 17.4,
 (1336, 1396, 2): 17.1,
 (1336, 4406, 2): 18.1,
 (1336, 4411, 2): 11.2,
 (1336, 4414, 2): 9.6,
 (1336, 4415, 2): 16.1,
 (1336, 4417, 2): 14.4,
 (1336, 4418, 2): 7.5,
 (1336, 4419, 2): 19.4,
 (1336, 4420, 2): 18.7,
 (1336, 4421, 2): 7.6,
 (1336, 4423, 2): 8.2,
 (1336, 4425, 2): 15.7,
 (1337, 0, 2): 17.7,
 (1337, 1302, 2): 5.6,
 (1337, 1303, 2): 4.6,
 (1337, 1304, 2): 10.4,
 (1337, 1305, 2): 0.9,
 (1337, 1306, 2): 5.4,
 (1337, 1308, 2): 9.2,
 (1337, 1314, 2): 7.7,
 (1337, 1315, 2): 10.9,
 (1337, 1316, 2): 14.4,
 (1337, 1318, 2): 7.0,
 (1337, 1319, 2): 9.5,
 (1337, 1321, 2): 10.7,
 (1337, 1322, 2): 10.3,
 (1337, 1325, 2): 15.1,
 (1337, 1326, 2): 13.5,
 (1337, 1327, 2): 13.5,
 (1337, 1328, 2): 11.9,
 (1337, 1332, 2): 20.2,
 (1337, 1333, 2): 17.3,
 (1337, 1334, 2): 17.8,
 (1337, 1336, 2): 3.1,
 (1337, 1338, 2): 2.4,
 (1337, 1339, 2): 10.3,
 (1337, 1340, 2): 17.7,
 (1337, 1342, 2): 8.8,
 (1337, 1343, 2): 5.3,
 (1337, 1344, 2): 6.3,
 (1337, 1346, 2): 10.1,
 (1337, 1347, 2): 9.0,
 (1337, 1348, 2): 7.5,
 (1337, 1349, 2): 19.0,
 (1337, 1351, 2): 4.8,
 (1337, 1352, 2): 8.4,
 (1337, 1353, 2): 4.7,
 (1337, 1354, 2): 11.2,
 (1337, 1357, 2): 14.5,
 (1337, 1358, 2): 12.6,
 (1337, 1360, 2): 8.2,
 (1337, 1361, 2): 6.7,
 (1337, 1363, 2): 3.6,
 (1337, 1364, 2): 6.2,
 (1337, 1365, 2): 7.9,
 (1337, 1366, 2): 14.4,
 (1337, 1367, 2): 7.8,
 (1337, 1368, 2): 3.0,
 (1337, 1370, 2): 9.6,
 (1337, 1372, 2): 9.3,
 (1337, 1373, 2): 12.6,
 (1337, 1375, 2): 14.4,
 (1337, 1376, 2): 15.1,
 (1337, 1377, 2): 14.4,
 (1337, 1379, 2): 11.7,
 (1337, 1380, 2): 11.5,
 (1337, 1381, 2): 20.8,
 (1337, 1382, 2): 16.1,
 (1337, 1384, 2): 3.9,
 (1337, 1385, 2): 4.1,
 (1337, 1386, 2): 2.4,
 (1337, 1388, 2): 6.1,
 (1337, 1390, 2): 21.5,
 (1337, 1391, 2): 20.0,
 (1337, 1392, 2): 3.9,
 (1337, 1393, 2): 8.0,
 (1337, 1394, 2): 7.5,
 (1337, 1395, 2): 18.0,
 (1337, 1396, 2): 17.7,
 (1337, 4406, 2): 19.2,
 (1337, 4411, 2): 11.8,
 (1337, 4414, 2): 6.8,
 (1337, 4415, 2): 16.6,
 (1337, 4417, 2): 14.7,
 (1337, 4418, 2): 8.0,
 (1337, 4419, 2): 19.9,
 (1337, 4420, 2): 19.0,
 (1337, 4421, 2): 8.7,
 (1337, 4423, 2): 9.3,
 (1337, 4425, 2): 16.7,
 (1338, 0, 2): 15.1,
 (1338, 1302, 2): 8.5,
 (1338, 1303, 2): 7.4,
 (1338, 1304, 2): 12.4,
 (1338, 1305, 2): 3.0,
 (1338, 1306, 2): 7.3,
 (1338, 1308, 2): 6.7,
 (1338, 1314, 2): 10.4,
 (1338, 1315, 2): 12.7,
 (1338, 1316, 2): 15.9,
 (1338, 1318, 2): 9.4,
 (1338, 1319, 2): 11.1,
 (1338, 1321, 2): 8.0,
 (1338, 1322, 2): 12.0,
 (1338, 1325, 2): 15.7,
 (1338, 1326, 2): 15.2,
 (1338, 1327, 2): 15.2,
 (1338, 1328, 2): 13.6,
 (1338, 1332, 2): 21.8,
 (1338, 1333, 2): 19.0,
 (1338, 1334, 2): 19.6,
 (1338, 1336, 2): 5.6,
 (1338, 1337, 2): 2.6,
 (1338, 1339, 2): 12.5,
 (1338, 1340, 2): 19.9,
 (1338, 1342, 2): 10.8,
 (1338, 1343, 2): 8.1,
 (1338, 1344, 2): 8.2,
 (1338, 1346, 2): 12.2,
 (1338, 1347, 2): 11.2,
 (1338, 1348, 2): 9.8,
 (1338, 1349, 2): 21.2,
 (1338, 1351, 2): 2.4,
 (1338, 1352, 2): 9.5,
 (1338, 1353, 2): 6.4,
 (1338, 1354, 2): 10.3,
 (1338, 1357, 2): 15.9,
 (1338, 1358, 2): 14.2,
 (1338, 1360, 2): 10.0,
 (1338, 1361, 2): 8.7,
 (1338, 1363, 2): 1.2,
 (1338, 1364, 2): 8.2,
 (1338, 1365, 2): 9.9,
 (1338, 1366, 2): 11.6,
 (1338, 1367, 2): 9.9,
 (1338, 1368, 2): 3.1,
 (1338, 1370, 2): 11.6,
 (1338, 1372, 2): 6.6,
 (1338, 1373, 2): 14.3,
 (1338, 1375, 2): 16.1,
 (1338, 1376, 2): 12.3,
 (1338, 1377, 2): 16.0,
 (1338, 1379, 2): 13.8,
 (1338, 1380, 2): 13.1,
 (1338, 1381, 2): 22.5,
 (1338, 1382, 2): 17.8,
 (1338, 1384, 2): 6.4,
 (1338, 1385, 2): 6.8,
 (1338, 1386, 2): 4.9,
 (1338, 1388, 2): 7.8,
 (1338, 1390, 2): 18.6,
 (1338, 1391, 2): 22.3,
 (1338, 1392, 2): 5.7,
 (1338, 1393, 2): 10.2,
 (1338, 1394, 2): 9.9,
 (1338, 1395, 2): 20.2,
 (1338, 1396, 2): 19.9,
 (1338, 4406, 2): 20.6,
 (1338, 4411, 2): 14.0,
 (1338, 4414, 2): 4.1,
 (1338, 4415, 2): 19.0,
 (1338, 4417, 2): 16.4,
 (1338, 4418, 2): 9.6,
 (1338, 4419, 2): 21.0,
 (1338, 4420, 2): 20.3,
 (1338, 4421, 2): 10.6,
 (1338, 4423, 2): 11.2,
 (1338, 4425, 2): 18.6,
 (1339, 0, 2): 18.5,
 (1339, 1302, 2): 10.7,
 (1339, 1303, 2): 9.8,
 (1339, 1304, 2): 16.4,
 (1339, 1305, 2): 13.2,
 (1339, 1306, 2): 15.4,
 (1339, 1308, 2): 11.8,
 (1339, 1314, 2): 9.5,
 (1339, 1315, 2): 10.5,
 (1339, 1316, 2): 11.8,
 (1339, 1318, 2): 7.0,
 (1339, 1319, 2): 6.0,
 (1339, 1321, 2): 9.5,
 (1339, 1322, 2): 7.5,
 (1339, 1325, 2): 10.4,
 (1339, 1326, 2): 9.1,
 (1339, 1327, 2): 9.5,
 (1339, 1328, 2): 8.0,
 (1339, 1332, 2): 15.4,
 (1339, 1333, 2): 12.6,
 (1339, 1334, 2): 13.0,
 (1339, 1336, 2): 14.2,
 (1339, 1337, 2): 11.4,
 (1339, 1338, 2): 12.5,
 (1339, 1340, 2): 8.2,
 (1339, 1342, 2): 14.9,
 (1339, 1343, 2): 10.4,
 (1339, 1344, 2): 12.0,
 (1339, 1346, 2): 8.5,
 (1339, 1347, 2): 7.5,
 (1339, 1348, 2): 6.1,
 (1339, 1349, 2): 9.5,
 (1339, 1351, 2): 12.7,
 (1339, 1352, 2): 19.3,
 (1339, 1353, 2): 7.2,
 (1339, 1354, 2): 9.0,
 (1339, 1357, 2): 13.8,
 (1339, 1358, 2): 12.1,
 (1339, 1360, 2): 10.9,
 (1339, 1361, 2): 12.8,
 (1339, 1363, 2): 14.7,
 (1339, 1364, 2): 12.2,
 (1339, 1365, 2): 13.9,
 (1339, 1366, 2): 10.5,
 (1339, 1367, 2): 6.2,
 (1339, 1368, 2): 10.3,
 (1339, 1370, 2): 15.6,
 (1339, 1372, 2): 10.2,
 (1339, 1373, 2): 8.6,
 (1339, 1375, 2): 4.3,
 (1339, 1376, 2): 14.0,
 (1339, 1377, 2): 10.2,
 (1339, 1379, 2): 10.1,
 (1339, 1380, 2): 7.6,
 (1339, 1381, 2): 16.0,
 (1339, 1382, 2): 6.3,
 (1339, 1384, 2): 9.3,
 (1339, 1385, 2): 12.6,
 (1339, 1386, 2): 13.3,
 (1339, 1388, 2): 15.9,
 (1339, 1390, 2): 15.5,
 (1339, 1391, 2): 10.4,
 (1339, 1392, 2): 13.8,
 (1339, 1393, 2): 6.5,
 (1339, 1394, 2): 6.1,
 (1339, 1395, 2): 8.4,
 (1339, 1396, 2): 8.8,
 (1339, 4406, 2): 9.1,
 (1339, 4411, 2): 5.7,
 (1339, 4414, 2): 12.0,
 (1339, 4415, 2): 11.3,
 (1339, 4417, 2): 14.2,
 (1339, 4418, 2): 5.8,
 (1339, 4419, 2): 14.9,
 (1339, 4420, 2): 14.2,
 (1339, 4421, 2): 5.1,
 (1339, 4423, 2): 4.4,
 (1339, 4425, 2): 6.6,
 (1340, 0, 2): 20.6,
 (1340, 1302, 2): 18.6,
 (1340, 1303, 2): 17.7,
 (1340, 1304, 2): 24.4,
 (1340, 1305, 2): 21.0,
 (1340, 1306, 2): 23.2,
 (1340, 1308, 2): 18.2,
 (1340, 1314, 2): 17.5,
 (1340, 1315, 2): 18.4,
 (1340, 1316, 2): 18.1,
 (1340, 1318, 2): 13.7,
 (1340, 1319, 2): 10.8,
 (1340, 1321, 2): 15.3,
 (1340, 1322, 2): 13.4,
 (1340, 1325, 2): 10.0,
 (1340, 1326, 2): 7.2,
 (1340, 1327, 2): 8.2,
 (1340, 1328, 2): 9.1,
 (1340, 1332, 2): 5.8,
 (1340, 1333, 2): 5.0,
 (1340, 1334, 2): 3.5,
 (1340, 1336, 2): 21.8,
 (1340, 1337, 2): 19.1,
 (1340, 1338, 2): 20.9,
 (1340, 1339, 2): 14.4,
 (1340, 1342, 2): 23.0,
 (1340, 1343, 2): 18.4,
 (1340, 1344, 2): 19.9,
 (1340, 1346, 2): 16.4,
 (1340, 1347, 2): 15.4,
 (1340, 1348, 2): 13.9,
 (1340, 1349, 2): 4.0,
 (1340, 1351, 2): 19.1,
 (1340, 1352, 2): 25.7,
 (1340, 1353, 2): 14.5,
 (1340, 1354, 2): 15.3,
 (1340, 1357, 2): 21.8,
 (1340, 1358, 2): 20.1,
 (1340, 1360, 2): 19.2,
 (1340, 1361, 2): 20.7,
 (1340, 1363, 2): 20.8,
 (1340, 1364, 2): 20.2,
 (1340, 1365, 2): 22.0,
 (1340, 1366, 2): 16.4,
 (1340, 1367, 2): 14.1,
 (1340, 1368, 2): 18.2,
 (1340, 1370, 2): 23.7,
 (1340, 1372, 2): 16.6,
 (1340, 1373, 2): 10.3,
 (1340, 1375, 2): 11.2,
 (1340, 1376, 2): 17.8,
 (1340, 1377, 2): 8.5,
 (1340, 1379, 2): 17.7,
 (1340, 1380, 2): 11.1,
 (1340, 1381, 2): 6.4,
 (1340, 1382, 2): 13.0,
 (1340, 1384, 2): 16.8,
 (1340, 1385, 2): 20.0,
 (1340, 1386, 2): 20.8,
 (1340, 1388, 2): 23.4,
 (1340, 1390, 2): 8.3,
 (1340, 1391, 2): 1.0,
 (1340, 1392, 2): 18.2,
 (1340, 1393, 2): 14.0,
 (1340, 1394, 2): 14.1,
 (1340, 1395, 2): 4.9,
 (1340, 1396, 2): 3.6,
 (1340, 4406, 2): 4.8,
 (1340, 4411, 2): 11.9,
 (1340, 4414, 2): 18.4,
 (1340, 4415, 2): 18.9,
 (1340, 4417, 2): 21.4,
 (1340, 4418, 2): 12.1,
 (1340, 4419, 2): 10.0,
 (1340, 4420, 2): 9.2,
 (1340, 4421, 2): 12.6,
 (1340, 4423, 2): 11.9,
 (1340, 4425, 2): 5.8,
 (1342, 0, 2): 26.6,
 (1342, 1302, 2): 8.4,
 (1342, 1303, 2): 15.6,
 (1342, 1304, 2): 2.4,
 (1342, 1305, 2): 13.2,
 (1342, 1306, 2): 10.6,
 (1342, 1308, 2): 21.2,
 (1342, 1314, 2): 14.4,
 (1342, 1315, 2): 14.4,
 (1342, 1316, 2): 16.6,
 (1342, 1318, 2): 19.6,
 (1342, 1319, 2): 21.2,
 (1342, 1321, 2): 22.6,
 (1342, 1322, 2): 22.0,
 (1342, 1325, 2): 27.1,
 (1342, 1326, 2): 25.6,
 (1342, 1327, 2): 25.5,
 (1342, 1328, 2): 23.9,
 (1342, 1332, 2): 32.3,
 (1342, 1333, 2): 29.5,
 (1342, 1334, 2): 30.0,
 (1342, 1336, 2): 14.0,
 (1342, 1337, 2): 13.4,
 (1342, 1338, 2): 15.8,
 (1342, 1339, 2): 22.8,
 (1342, 1340, 2): 30.4,
 (1342, 1343, 2): 9.1,
 (1342, 1344, 2): 17.8,
 (1342, 1346, 2): 22.4,
 (1342, 1347, 2): 21.4,
 (1342, 1348, 2): 20.1,
 (1342, 1349, 2): 31.5,
 (1342, 1351, 2): 17.1,
 (1342, 1352, 2): 18.3,
 (1342, 1353, 2): 16.7,
 (1342, 1354, 2): 23.0,
 (1342, 1357, 2): 16.8,
 (1342, 1358, 2): 14.5,
 (1342, 1360, 2): 16.1,
 (1342, 1361, 2): 6.9,
 (1342, 1363, 2): 15.9,
 (1342, 1364, 2): 6.4,
 (1342, 1365, 2): 6.4,
 (1342, 1366, 2): 26.4,
 (1342, 1367, 2): 20.2,
 (1342, 1368, 2): 15.5,
 (1342, 1370, 2): 1.7,
 (1342, 1372, 2): 21.2,
 (1342, 1373, 2): 24.6,
 (1342, 1375, 2): 26.4,
 (1342, 1376, 2): 27.0,
 (1342, 1377, 2): 26.2,
 (1342, 1379, 2): 21.6,
 (1342, 1380, 2): 23.2,
 (1342, 1381, 2): 32.9,
 (1342, 1382, 2): 28.1,
 (1342, 1384, 2): 17.0,
 (1342, 1385, 2): 13.7,
 (1342, 1386, 2): 14.4,
 (1342, 1388, 2): 11.9,
 (1342, 1390, 2): 33.0,
 (1342, 1391, 2): 32.7,
 (1342, 1392, 2): 15.9,
 (1342, 1393, 2): 18.5,
 (1342, 1394, 2): 20.1,
 (1342, 1395, 2): 30.8,
 (1342, 1396, 2): 30.4,
 (1342, 4406, 2): 31.2,
 (1342, 4411, 2): 24.4,
 (1342, 4414, 2): 18.7,
 (1342, 4415, 2): 25.0,
 (1342, 4417, 2): 18.2,
 (1342, 4418, 2): 19.9,
 (1342, 4419, 2): 32.0,
 (1342, 4420, 2): 31.2,
 (1342, 4421, 2): 20.8,
 (1342, 4423, 2): 21.5,
 (1342, 4425, 2): 29.1,
 (1343, 0, 2): 24.4,
 (1343, 1302, 2): 0.3,
 (1343, 1303, 2): 2.6,
 (1343, 1304, 2): 6.0,
 (1343, 1305, 2): 9.1,
 (1343, 1306, 2): 7.0,
 (1343, 1308, 2): 17.2,
 (1343, 1314, 2): 9.1,
 (1343, 1315, 2): 13.1,
 (1343, 1316, 2): 16.3,
 (1343, 1318, 2): 16.4,
 (1343, 1319, 2): 16.5,
 (1343, 1321, 2): 18.6,
 (1343, 1322, 2): 17.3,
 (1343, 1325, 2): 22.4,
 (1343, 1326, 2): 20.8,
 (1343, 1327, 2): 20.8,
 (1343, 1328, 2): 19.2,
 (1343, 1332, 2): 27.5,
 (1343, 1333, 2): 24.7,
 (1343, 1334, 2): 25.1,
 (1343, 1336, 2): 10.1,
 (1343, 1337, 2): 9.4,
 (1343, 1338, 2): 11.8,
 (1343, 1339, 2): 19.6,
 (1343, 1340, 2): 27.4,
 (1343, 1342, 2): 4.5,
 (1343, 1344, 2): 14.4,
 (1343, 1346, 2): 18.3,
 (1343, 1347, 2): 17.2,
 (1343, 1348, 2): 15.8,
 (1343, 1349, 2): 27.3,
 (1343, 1351, 2): 13.2,
 (1343, 1352, 2): 16.2,
 (1343, 1353, 2): 13.5,
 (1343, 1354, 2): 18.3,
 (1343, 1357, 2): 16.3,
 (1343, 1358, 2): 14.0,
 (1343, 1360, 2): 10.3,
 (1343, 1361, 2): 2.2,
 (1343, 1363, 2): 11.2,
 (1343, 1364, 2): 1.7,
 (1343, 1365, 2): 3.3,
 (1343, 1366, 2): 22.4,
 (1343, 1367, 2): 15.8,
 (1343, 1368, 2): 10.8,
 (1343, 1370, 2): 5.0,
 (1343, 1372, 2): 16.0,
 (1343, 1373, 2): 19.6,
 (1343, 1375, 2): 21.4,
 (1343, 1376, 2): 21.9,
 (1343, 1377, 2): 21.2,
 (1343, 1379, 2): 17.7,
 (1343, 1380, 2): 18.6,
 (1343, 1381, 2): 27.8,
 (1343, 1382, 2): 23.2,
 (1343, 1384, 2): 12.6,
 (1343, 1385, 2): 9.8,
 (1343, 1386, 2): 10.8,
 (1343, 1388, 2): 8.0,
 (1343, 1390, 2): 28.0,
 (1343, 1391, 2): 26.7,
 (1343, 1392, 2): 13.1,
 (1343, 1393, 2): 12.8,
 (1343, 1394, 2): 13.0,
 (1343, 1395, 2): 27.3,
 (1343, 1396, 2): 27.7,
 (1343, 4406, 2): 26.2,
 (1343, 4411, 2): 20.0,
 (1343, 4414, 2): 13.7,
 (1343, 4415, 2): 23.5,
 (1343, 4417, 2): 16.6,
 (1343, 4418, 2): 15.1,
 (1343, 4419, 2): 26.9,
 (1343, 4420, 2): 26.1,
 (1343, 4421, 2): 15.4,
 (1343, 4423, 2): 17.1,
 (1343, 4425, 2): 27.2,
 (1344, 0, 2): 19.4,
 (1344, 1302, 2): 5.7,
 (1344, 1303, 2): 4.9,
 (1344, 1304, 2): 11.2,
 (1344, 1305, 2): 6.5,
 (1344, 1306, 2): 10.6,
 (1344, 1308, 2): 12.6,
 (1344, 1314, 2): 6.4,
 (1344, 1315, 2): 4.7,
 (1344, 1316, 2): 7.7,
 (1344, 1318, 2): 5.9,
 (1344, 1319, 2): 7.8,
 (1344, 1321, 2): 10.7,
 (1344, 1322, 2): 8.7,
 (1344, 1325, 2): 13.0,
 (1344, 1326, 2): 12.1,
 (1344, 1327, 2): 12.1,
 (1344, 1328, 2): 10.4,
 (1344, 1332, 2): 18.6,
 (1344, 1333, 2): 15.8,
 (1344, 1334, 2): 16.3,
 (1344, 1336, 2): 8.1,
 (1344, 1337, 2): 5.6,
 (1344, 1338, 2): 8.0,
 (1344, 1339, 2): 10.1,
 (1344, 1340, 2): 18.0,
 (1344, 1342, 2): 9.8,
 (1344, 1343, 2): 5.4,
 (1344, 1346, 2): 9.5,
 (1344, 1347, 2): 8.4,
 (1344, 1348, 2): 7.0,
 (1344, 1349, 2): 18.2,
 (1344, 1351, 2): 10.4,
 (1344, 1352, 2): 13.4,
 (1344, 1353, 2): 4.0,
 (1344, 1354, 2): 9.4,
 (1344, 1357, 2): 7.9,
 (1344, 1358, 2): 6.2,
 (1344, 1360, 2): 1.7,
 (1344, 1361, 2): 7.7,
 (1344, 1363, 2): 9.2,
 (1344, 1364, 2): 7.2,
 (1344, 1365, 2): 8.8,
 (1344, 1366, 2): 12.4,
 (1344, 1367, 2): 7.1,
 (1344, 1368, 2): 7.8,
 (1344, 1370, 2): 10.5,
 (1344, 1372, 2): 10.8,
 (1344, 1373, 2): 11.1,
 (1344, 1375, 2): 12.9,
 (1344, 1376, 2): 14.7,
 (1344, 1377, 2): 12.6,
 (1344, 1379, 2): 9.2,
 (1344, 1380, 2): 9.9,
 (1344, 1381, 2): 19.2,
 (1344, 1382, 2): 14.6,
 (1344, 1384, 2): 3.4,
 (1344, 1385, 2): 7.6,
 (1344, 1386, 2): 8.4,
 (1344, 1388, 2): 11.0,
 (1344, 1390, 2): 18.9,
 (1344, 1391, 2): 18.0,
 (1344, 1392, 2): 4.6,
 (1344, 1393, 2): 4.1,
 (1344, 1394, 2): 4.3,
 (1344, 1395, 2): 18.7,
 (1344, 1396, 2): 19.1,
 (1344, 4406, 2): 17.6,
 (1344, 4411, 2): 11.6,
 (1344, 4414, 2): 11.8,
 (1344, 4415, 2): 14.6,
 (1344, 4417, 2): 7.9,
 (1344, 4418, 2): 6.4,
 (1344, 4419, 2): 18.3,
 (1344, 4420, 2): 17.6,
 (1344, 4421, 2): 6.7,
 (1344, 4423, 2): 8.4,
 (1344, 4425, 2): 18.7,
 (1346, 0, 2): 20.8,
 (1346, 1302, 2): 9.2,
 (1346, 1303, 2): 8.3,
 (1346, 1304, 2): 14.8,
 (1346, 1305, 2): 12.0,
 (1346, 1306, 2): 14.0,
 (1346, 1308, 2): 14.0,
 (1346, 1314, 2): 7.5,
 (1346, 1315, 2): 8.7,
 (1346, 1316, 2): 11.7,
 (1346, 1318, 2): 7.8,
 (1346, 1319, 2): 9.2,
 (1346, 1321, 2): 12.1,
 (1346, 1322, 2): 10.1,
 (1346, 1325, 2): 14.3,
 (1346, 1326, 2): 13.2,
 (1346, 1327, 2): 13.2,
 (1346, 1328, 2): 11.6,
 (1346, 1332, 2): 19.8,
 (1346, 1333, 2): 17.0,
 (1346, 1334, 2): 17.5,
 (1346, 1336, 2): 12.3,
 (1346, 1337, 2): 9.9,
 (1346, 1338, 2): 10.6,
 (1346, 1339, 2): 11.1,
 (1346, 1340, 2): 19.0,
 (1346, 1342, 2): 13.4,
 (1346, 1343, 2): 8.9,
 (1346, 1344, 2): 10.6,
 (1346, 1347, 2): 1.0,
 (1346, 1348, 2): 2.8,
 (1346, 1349, 2): 19.2,
 (1346, 1351, 2): 15.4,
 (1346, 1352, 2): 20.9,
 (1346, 1353, 2): 5.4,
 (1346, 1354, 2): 10.8,
 (1346, 1357, 2): 10.1,
 (1346, 1358, 2): 10.2,
 (1346, 1360, 2): 9.2,
 (1346, 1361, 2): 11.2,
 (1346, 1363, 2): 14.2,
 (1346, 1364, 2): 10.7,
 (1346, 1365, 2): 12.4,
 (1346, 1366, 2): 13.7,
 (1346, 1367, 2): 2.8,
 (1346, 1368, 2): 8.4,
 (1346, 1370, 2): 14.1,
 (1346, 1372, 2): 12.1,
 (1346, 1373, 2): 12.3,
 (1346, 1375, 2): 14.2,
 (1346, 1376, 2): 16.1,
 (1346, 1377, 2): 13.9,
 (1346, 1379, 2): 1.7,
 (1346, 1380, 2): 11.2,
 (1346, 1381, 2): 20.5,
 (1346, 1382, 2): 15.9,
 (1346, 1384, 2): 7.8,
 (1346, 1385, 2): 11.0,
 (1346, 1386, 2): 11.8,
 (1346, 1388, 2): 14.4,
 (1346, 1390, 2): 20.2,
 (1346, 1391, 2): 20.6,
 (1346, 1392, 2): 12.2,
 (1346, 1393, 2): 4.7,
 (1346, 1394, 2): 4.3,
 (1346, 1395, 2): 18.8,
 (1346, 1396, 2): 18.4,
 (1346, 4406, 2): 18.8,
 (1346, 4411, 2): 12.6,
 (1346, 4414, 2): 16.7,
 (1346, 4415, 2): 10.1,
 (1346, 4417, 2): 8.4,
 (1346, 4418, 2): 7.7,
 (1346, 4419, 2): 19.6,
 (1346, 4420, 2): 18.8,
 (1346, 4421, 2): 7.6,
 (1346, 4423, 2): 6.6,
 (1346, 4425, 2): 17.1,
 (1347, 0, 2): 19.8,
 (1347, 1302, 2): 8.2,
 (1347, 1303, 2): 7.3,
 (1347, 1304, 2): 13.8,
 (1347, 1305, 2): 10.9,
 (1347, 1306, 2): 12.9,
 (1347, 1308, 2): 12.9,
 (1347, 1314, 2): 6.5,
 (1347, 1315, 2): 7.7,
 (1347, 1316, 2): 10.7,
 (1347, 1318, 2): 5.5,
 (1347, 1319, 2): 8.1,
 (1347, 1321, 2): 11.0,
 (1347, 1322, 2): 9.0,
 (1347, 1325, 2): 13.3,
 (1347, 1326, 2): 12.2,
 (1347, 1327, 2): 12.1,
 (1347, 1328, 2): 10.6,
 (1347, 1332, 2): 18.8,
 (1347, 1333, 2): 16.0,
 (1347, 1334, 2): 16.4,
 (1347, 1336, 2): 11.3,
 (1347, 1337, 2): 8.8,
 (1347, 1338, 2): 9.6,
 (1347, 1339, 2): 10.0,
 (1347, 1340, 2): 17.9,
 (1347, 1342, 2): 12.4,
 (1347, 1343, 2): 7.9,
 (1347, 1344, 2): 9.5,
 (1347, 1346, 2): 1.1,
 (1347, 1348, 2): 1.8,
 (1347, 1349, 2): 18.2,
 (1347, 1351, 2): 14.3,
 (1347, 1352, 2): 19.9,
 (1347, 1353, 2): 4.4,
 (1347, 1354, 2): 9.8,
 (1347, 1357, 2): 10.9,
 (1347, 1358, 2): 9.2,
 (1347, 1360, 2): 8.1,
 (1347, 1361, 2): 10.2,
 (1347, 1363, 2): 13.2,
 (1347, 1364, 2): 9.6,
 (1347, 1365, 2): 11.4,
 (1347, 1366, 2): 12.6,
 (1347, 1367, 2): 1.8,
 (1347, 1368, 2): 7.4,
 (1347, 1370, 2): 13.1,
 (1347, 1372, 2): 11.0,
 (1347, 1373, 2): 11.2,
 (1347, 1375, 2): 13.2,
 (1347, 1376, 2): 15.0,
 (1347, 1377, 2): 12.8,
 (1347, 1379, 2): 2.7,
 (1347, 1380, 2): 10.2,
 (1347, 1381, 2): 19.4,
 (1347, 1382, 2): 14.9,
 (1347, 1384, 2): 6.8,
 (1347, 1385, 2): 10.0,
 (1347, 1386, 2): 10.8,
 (1347, 1388, 2): 13.4,
 (1347, 1390, 2): 19.1,
 (1347, 1391, 2): 18.2,
 (1347, 1392, 2): 11.1,
 (1347, 1393, 2): 3.7,
 (1347, 1394, 2): 3.2,
 (1347, 1395, 2): 18.9,
 (1347, 1396, 2): 19.3,
 (1347, 4406, 2): 17.7,
 (1347, 4411, 2): 11.6,
 (1347, 4414, 2): 15.6,
 (1347, 4415, 2): 11.2,
 (1347, 4417, 2): 9.5,
 (1347, 4418, 2): 6.7,
 (1347, 4419, 2): 18.5,
 (1347, 4420, 2): 17.8,
 (1347, 4421, 2): 7.0,
 (1347, 4423, 2): 7.7,
 (1347, 4425, 2): 18.9,
 (1348, 0, 2): 18.2,
 (1348, 1302, 2): 6.6,
 (1348, 1303, 2): 5.7,
 (1348, 1304, 2): 12.2,
 (1348, 1305, 2): 9.4,
 (1348, 1306, 2): 11.4,
 (1348, 1308, 2): 11.3,
 (1348, 1314, 2): 4.9,
 (1348, 1315, 2): 6.1,
 (1348, 1316, 2): 9.1,
 (1348, 1318, 2): 4.7,
 (1348, 1319, 2): 6.5,
 (1348, 1321, 2): 9.4,
 (1348, 1322, 2): 7.4,
 (1348, 1325, 2): 11.7,
 (1348, 1326, 2): 10.7,
 (1348, 1327, 2): 10.6,
 (1348, 1328, 2): 9.1,
 (1348, 1332, 2): 17.3,
 (1348, 1333, 2): 14.5,
 (1348, 1334, 2): 14.9,
 (1348, 1336, 2): 9.7,
 (1348, 1337, 2): 7.3,
 (1348, 1338, 2): 8.0,
 (1348, 1339, 2): 8.4,
 (1348, 1340, 2): 16.4,
 (1348, 1342, 2): 10.8,
 (1348, 1343, 2): 6.3,
 (1348, 1344, 2): 7.9,
 (1348, 1346, 2): 2.5,
 (1348, 1347, 2): 1.4,
 (1348, 1349, 2): 16.6,
 (1348, 1351, 2): 12.8,
 (1348, 1352, 2): 18.3,
 (1348, 1353, 2): 2.8,
 (1348, 1354, 2): 8.2,
 (1348, 1357, 2): 9.3,
 (1348, 1358, 2): 7.6,
 (1348, 1360, 2): 6.5,
 (1348, 1361, 2): 8.6,
 (1348, 1363, 2): 11.6,
 (1348, 1364, 2): 8.1,
 (1348, 1365, 2): 9.8,
 (1348, 1366, 2): 11.1,
 (1348, 1367, 2): 0.2,
 (1348, 1368, 2): 5.8,
 (1348, 1370, 2): 11.5,
 (1348, 1372, 2): 9.5,
 (1348, 1373, 2): 9.6,
 (1348, 1375, 2): 11.6,
 (1348, 1376, 2): 13.5,
 (1348, 1377, 2): 11.2,
 (1348, 1379, 2): 4.1,
 (1348, 1380, 2): 8.6,
 (1348, 1381, 2): 17.8,
 (1348, 1382, 2): 13.3,
 (1348, 1384, 2): 5.2,
 (1348, 1385, 2): 8.4,
 (1348, 1386, 2): 9.2,
 (1348, 1388, 2): 11.8,
 (1348, 1390, 2): 17.6,
 (1348, 1391, 2): 16.7,
 (1348, 1392, 2): 5.0,
 (1348, 1393, 2): 2.1,
 (1348, 1394, 2): 1.7,
 (1348, 1395, 2): 16.2,
 (1348, 1396, 2): 15.8,
 (1348, 4406, 2): 16.2,
 (1348, 4411, 2): 10.0,
 (1348, 4414, 2): 14.1,
 (1348, 4415, 2): 12.7,
 (1348, 4417, 2): 9.3,
 (1348, 4418, 2): 5.1,
 (1348, 4419, 2): 17.0,
 (1348, 4420, 2): 16.2,
 (1348, 4421, 2): 5.4,
 (1348, 4423, 2): 9.1,
 (1348, 4425, 2): 14.5,
 (1349, 0, 2): 18.7,
 (1349, 1302, 2): 14.8,
 (1349, 1303, 2): 13.9,
 (1349, 1304, 2): 20.1,
 (1349, 1305, 2): 17.0,
 (1349, 1306, 2): 19.0,
 (1349, 1308, 2): 15.4,
 (1349, 1314, 2): 12.7,
 (1349, 1315, 2): 13.6,
 (1349, 1316, 2): 14.9,
 (1349, 1318, 2): 10.5,
 (1349, 1319, 2): 10.0,
 (1349, 1321, 2): 13.5,
 (1349, 1322, 2): 11.5,
 (1349, 1325, 2): 11.8,
 (1349, 1326, 2): 8.8,
 (1349, 1327, 2): 11.7,
 (1349, 1328, 2): 10.6,
 (1349, 1332, 2): 9.5,
 (1349, 1333, 2): 8.7,
 (1349, 1334, 2): 7.2,
 (1349, 1336, 2): 17.7,
 (1349, 1337, 2): 15.1,
 (1349, 1338, 2): 15.8,
 (1349, 1339, 2): 9.9,
 (1349, 1340, 2): 3.6,
 (1349, 1342, 2): 18.5,
 (1349, 1343, 2): 14.1,
 (1349, 1344, 2): 15.4,
 (1349, 1346, 2): 11.8,
 (1349, 1347, 2): 10.8,
 (1349, 1348, 2): 9.5,
 (1349, 1351, 2): 16.4,
 (1349, 1352, 2): 22.5,
 (1349, 1353, 2): 10.6,
 (1349, 1354, 2): 12.3,
 (1349, 1357, 2): 16.9,
 (1349, 1358, 2): 15.2,
 (1349, 1360, 2): 14.4,
 (1349, 1361, 2): 16.4,
 (1349, 1363, 2): 19.2,
 (1349, 1364, 2): 15.9,
 (1349, 1365, 2): 17.6,
 (1349, 1366, 2): 15.8,
 (1349, 1367, 2): 9.6,
 (1349, 1368, 2): 13.5,
 (1349, 1370, 2): 19.3,
 (1349, 1372, 2): 13.6,
 (1349, 1373, 2): 11.9,
 (1349, 1375, 2): 8.2,
 (1349, 1376, 2): 17.6,
 (1349, 1377, 2): 10.2,
 (1349, 1379, 2): 13.4,
 (1349, 1380, 2): 11.5,
 (1349, 1381, 2): 10.2,
 (1349, 1382, 2): 10.0,
 (1349, 1384, 2): 13.0,
 (1349, 1385, 2): 16.3,
 (1349, 1386, 2): 17.0,
 (1349, 1388, 2): 19.5,
 (1349, 1390, 2): 11.9,
 (1349, 1391, 2): 4.6,
 (1349, 1392, 2): 12.9,
 (1349, 1393, 2): 9.9,
 (1349, 1394, 2): 9.5,
 (1349, 1395, 2): 4.9,
 (1349, 1396, 2): 3.6,
 (1349, 4406, 2): 4.7,
 (1349, 4411, 2): 10.1,
 (1349, 4414, 2): 17.9,
 (1349, 4415, 2): 14.8,
 (1349, 4417, 2): 17.5,
 (1349, 4418, 2): 9.2,
 (1349, 4419, 2): 13.6,
 (1349, 4420, 2): 12.9,
 (1349, 4421, 2): 8.4,
 (1349, 4423, 2): 7.7,
 (1349, 4425, 2): 4.0,
 (1351, 0, 2): 12.6,
 (1351, 1302, 2): 10.8,
 (1351, 1303, 2): 9.7,
 (1351, 1304, 2): 14.5,
 (1351, 1305, 2): 6.4,
 (1351, 1306, 2): 9.7,
 (1351, 1308, 2): 5.8,
 (1351, 1314, 2): 14.4,
 (1351, 1315, 2): 16.5,
 (1351, 1316, 2): 19.8,
 (1351, 1318, 2): 12.8,
 (1351, 1319, 2): 14.1,
 (1351, 1321, 2): 9.3,
 (1351, 1322, 2): 15.0,
 (1351, 1325, 2): 16.4,
 (1351, 1326, 2): 15.5,
 (1351, 1327, 2): 15.4,
 (1351, 1328, 2): 16.8,
 (1351, 1332, 2): 22.0,
 (1351, 1333, 2): 19.3,
 (1351, 1334, 2): 19.7,
 (1351, 1336, 2): 8.0,
 (1351, 1337, 2): 6.0,
 (1351, 1338, 2): 3.4,
 (1351, 1339, 2): 15.7,
 (1351, 1340, 2): 23.1,
 (1351, 1342, 2): 13.0,
 (1351, 1343, 2): 10.6,
 (1351, 1344, 2): 11.9,
 (1351, 1346, 2): 15.7,
 (1351, 1347, 2): 14.7,
 (1351, 1348, 2): 13.3,
 (1351, 1349, 2): 24.0,
 (1351, 1352, 2): 6.6,
 (1351, 1353, 2): 9.7,
 (1351, 1354, 2): 11.4,
 (1351, 1357, 2): 19.7,
 (1351, 1358, 2): 18.1,
 (1351, 1360, 2): 13.7,
 (1351, 1361, 2): 11.0,
 (1351, 1363, 2): 1.2,
 (1351, 1364, 2): 10.5,
 (1351, 1365, 2): 12.2,
 (1351, 1366, 2): 16.2,
 (1351, 1367, 2): 13.4,
 (1351, 1368, 2): 6.4,
 (1351, 1370, 2): 13.8,
 (1351, 1372, 2): 7.9,
 (1351, 1373, 2): 17.5,
 (1351, 1375, 2): 19.3,
 (1351, 1376, 2): 14.0,
 (1351, 1377, 2): 16.0,
 (1351, 1379, 2): 17.3,
 (1351, 1380, 2): 13.5,
 (1351, 1381, 2): 22.7,
 (1351, 1382, 2): 21.0,
 (1351, 1384, 2): 10.5,
 (1351, 1385, 2): 9.2,
 (1351, 1386, 2): 8.8,
 (1351, 1388, 2): 10.3,
 (1351, 1390, 2): 18.8,
 (1351, 1391, 2): 21.6,
 (1351, 1392, 2): 9.2,
 (1351, 1393, 2): 13.8,
 (1351, 1394, 2): 13.2,
 (1351, 1395, 2): 23.4,
 (1351, 1396, 2): 23.2,
 (1351, 4406, 2): 21.0,
 (1351, 4411, 2): 17.5,
 (1351, 4414, 2): 5.6,
 (1351, 4415, 2): 21.7,
 (1351, 4417, 2): 20.0,
 (1351, 4418, 2): 12.8,
 (1351, 4419, 2): 21.2,
 (1351, 4420, 2): 20.5,
 (1351, 4421, 2): 12.6,
 (1351, 4423, 2): 14.7,
 (1351, 4425, 2): 21.8,
 (1352, 0, 2): 15.2,
 (1352, 1302, 2): 8.6,
 (1352, 1303, 2): 7.4,
 (1352, 1304, 2): 12.3,
 (1352, 1305, 2): 5.1,
 (1352, 1306, 2): 7.4,
 (1352, 1308, 2): 7.4,
 (1352, 1314, 2): 13.6,
 (1352, 1315, 2): 15.6,
 (1352, 1316, 2): 18.9,
 (1352, 1318, 2): 11.9,
 (1352, 1319, 2): 13.2,
 (1352, 1321, 2): 8.6,
 (1352, 1322, 2): 14.0,
 (1352, 1325, 2): 15.7,
 (1352, 1326, 2): 14.7,
 (1352, 1327, 2): 14.7,
 (1352, 1328, 2): 15.8,
 (1352, 1332, 2): 21.2,
 (1352, 1333, 2): 18.5,
 (1352, 1334, 2): 19.0,
 (1352, 1336, 2): 5.8,
 (1352, 1337, 2): 5.2,
 (1352, 1338, 2): 2.8,
 (1352, 1339, 2): 14.9,
 (1352, 1340, 2): 22.3,
 (1352, 1342, 2): 10.8,
 (1352, 1343, 2): 8.2,
 (1352, 1344, 2): 11.0,
 (1352, 1346, 2): 14.9,
 (1352, 1347, 2): 13.9,
 (1352, 1348, 2): 12.5,
 (1352, 1349, 2): 23.1,
 (1352, 1351, 2): 3.8,
 (1352, 1353, 2): 8.8,
 (1352, 1354, 2): 10.7,
 (1352, 1357, 2): 18.9,
 (1352, 1358, 2): 17.3,
 (1352, 1360, 2): 12.9,
 (1352, 1361, 2): 8.8,
 (1352, 1363, 2): 2.6,
 (1352, 1364, 2): 8.3,
 (1352, 1365, 2): 9.9,
 (1352, 1366, 2): 15.4,
 (1352, 1367, 2): 12.5,
 (1352, 1368, 2): 5.5,
 (1352, 1370, 2): 11.6,
 (1352, 1372, 2): 7.1,
 (1352, 1373, 2): 16.6,
 (1352, 1375, 2): 18.4,
 (1352, 1376, 2): 13.2,
 (1352, 1377, 2): 15.2,
 (1352, 1379, 2): 16.3,
 (1352, 1380, 2): 12.7,
 (1352, 1381, 2): 21.9,
 (1352, 1382, 2): 21.1,
 (1352, 1384, 2): 9.6,
 (1352, 1385, 2): 6.9,
 (1352, 1386, 2): 7.7,
 (1352, 1388, 2): 8.0,
 (1352, 1390, 2): 18.8,
 (1352, 1391, 2): 20.9,
 (1352, 1392, 2): 8.2,
 (1352, 1393, 2): 13.0,
 (1352, 1394, 2): 12.4,
 (1352, 1395, 2): 22.6,
 (1352, 1396, 2): 22.9,
 (1352, 4406, 2): 20.3,
 (1352, 4411, 2): 16.6,
 (1352, 4414, 2): 4.8,
 (1352, 4415, 2): 20.7,
 (1352, 4417, 2): 19.1,
 (1352, 4418, 2): 11.9,
 (1352, 4419, 2): 21.4,
 (1352, 4420, 2): 20.6,
 (1352, 4421, 2): 11.7,
 (1352, 4423, 2): 13.8,
 (1352, 4425, 2): 20.8,
 (1353, 0, 2): 15.9,
 (1353, 1302, 2): 8.7,
 (1353, 1303, 2): 7.7,
 (1353, 1304, 2): 14.2,
 (1353, 1305, 2): 11.3,
 (1353, 1306, 2): 13.2,
 (1353, 1308, 2): 9.9,
 (1353, 1314, 2): 6.9,
 (1353, 1315, 2): 7.9,
 (1353, 1316, 2): 9.8,
 (1353, 1318, 2): 3.0,
 (1353, 1319, 2): 4.9,
 (1353, 1321, 2): 7.6,
 (1353, 1322, 2): 5.7,
 (1353, 1325, 2): 9.8,
 (1353, 1326, 2): 9.1,
 (1353, 1327, 2): 9.0,
 (1353, 1328, 2): 7.5,
 (1353, 1332, 2): 15.6,
 (1353, 1333, 2): 12.8,
 (1353, 1334, 2): 13.2,
 (1353, 1336, 2): 12.2,
 (1353, 1337, 2): 9.4,
 (1353, 1338, 2): 12.0,
 (1353, 1339, 2): 6.2,
 (1353, 1340, 2): 13.5,
 (1353, 1342, 2): 12.8,
 (1353, 1343, 2): 8.4,
 (1353, 1344, 2): 9.7,
 (1353, 1346, 2): 6.2,
 (1353, 1347, 2): 5.1,
 (1353, 1348, 2): 3.8,
 (1353, 1349, 2): 14.4,
 (1353, 1351, 2): 10.8,
 (1353, 1352, 2): 16.9,
 (1353, 1354, 2): 6.7,
 (1353, 1357, 2): 11.1,
 (1353, 1358, 2): 9.4,
 (1353, 1360, 2): 8.8,
 (1353, 1361, 2): 10.6,
 (1353, 1363, 2): 12.1,
 (1353, 1364, 2): 10.2,
 (1353, 1365, 2): 11.8,
 (1353, 1366, 2): 10.0,
 (1353, 1367, 2): 3.8,
 (1353, 1368, 2): 7.8,
 (1353, 1370, 2): 13.5,
 (1353, 1372, 2): 8.0,
 (1353, 1373, 2): 8.2,
 (1353, 1375, 2): 10.0,
 (1353, 1376, 2): 12.1,
 (1353, 1377, 2): 9.7,
 (1353, 1379, 2): 7.8,
 (1353, 1380, 2): 6.7,
 (1353, 1381, 2): 16.2,
 (1353, 1382, 2): 11.6,
 (1353, 1384, 2): 7.2,
 (1353, 1385, 2): 10.5,
 (1353, 1386, 2): 11.3,
 (1353, 1388, 2): 13.7,
 (1353, 1390, 2): 16.0,
 (1353, 1391, 2): 15.1,
 (1353, 1392, 2): 7.1,
 (1353, 1393, 2): 4.2,
 (1353, 1394, 2): 3.7,
 (1353, 1395, 2): 13.7,
 (1353, 1396, 2): 13.5,
 (1353, 4406, 2): 14.6,
 (1353, 4411, 2): 7.9,
 (1353, 4414, 2): 9.8,
 (1353, 4415, 2): 12.9,
 (1353, 4417, 2): 11.4,
 (1353, 4418, 2): 3.6,
 (1353, 4419, 2): 15.4,
 (1353, 4420, 2): 14.6,
 (1353, 4421, 2): 3.5,
 (1353, 4423, 2): 5.1,
 (1353, 4425, 2): 12.0,
 (1354, 0, 2): 13.0,
 (1354, 1302, 2): 14.1,
 (1354, 1303, 2): 12.8,
 (1354, 1304, 2): 17.8,
 (1354, 1305, 2): 9.7,
 (1354, 1306, 2): 12.8,
 (1354, 1308, 2): 4.5,
 (1354, 1314, 2): 12.2,
 (1354, 1315, 2): 13.2,
 (1354, 1316, 2): 13.2,
 (1354, 1318, 2): 8.6,
 (1354, 1319, 2): 8.0,
 (1354, 1321, 2): 3.1,
 (1354, 1322, 2): 9.2,
 (1354, 1325, 2): 9.8,
 (1354, 1326, 2): 9.2,
 (1354, 1327, 2): 9.1,
 (1354, 1328, 2): 11.8,
 (1354, 1332, 2): 15.6,
 (1354, 1333, 2): 12.9,
 (1354, 1334, 2): 13.3,
 (1354, 1336, 2): 11.1,
 (1354, 1337, 2): 9.2,
 (1354, 1338, 2): 6.6,
 (1354, 1339, 2): 10.1,
 (1354, 1340, 2): 17.8,
 (1354, 1342, 2): 16.3,
 (1354, 1343, 2): 13.8,
 (1354, 1344, 2): 15.2,
 (1354, 1346, 2): 13.4,
 (1354, 1347, 2): 12.4,
 (1354, 1348, 2): 11.0,
 (1354, 1349, 2): 18.1,
 (1354, 1351, 2): 5.4,
 (1354, 1352, 2): 11.4,
 (1354, 1353, 2): 9.4,
 (1354, 1357, 2): 16.5,
 (1354, 1358, 2): 14.9,
 (1354, 1360, 2): 13.0,
 (1354, 1361, 2): 14.2,
 (1354, 1363, 2): 6.7,
 (1354, 1364, 2): 13.7,
 (1354, 1365, 2): 15.4,
 (1354, 1366, 2): 10.2,
 (1354, 1367, 2): 11.1,
 (1354, 1368, 2): 9.7,
 (1354, 1370, 2): 17.1,
 (1354, 1372, 2): 2.6,
 (1354, 1373, 2): 12.5,
 (1354, 1375, 2): 14.4,
 (1354, 1376, 2): 7.7,
 (1354, 1377, 2): 9.7,
 (1354, 1379, 2): 15.5,
 (1354, 1380, 2): 7.0,
 (1354, 1381, 2): 16.6,
 (1354, 1382, 2): 13.0,
 (1354, 1384, 2): 13.0,
 (1354, 1385, 2): 11.0,
 (1354, 1386, 2): 11.3,
 (1354, 1388, 2): 12.8,
 (1354, 1390, 2): 14.0,
 (1354, 1391, 2): 15.2,
 (1354, 1392, 2): 11.9,
 (1354, 1393, 2): 14.6,
 (1354, 1394, 2): 9.4,
 (1354, 1395, 2): 16.2,
 (1354, 1396, 2): 16.5,
 (1354, 4406, 2): 14.9,
 (1354, 4411, 2): 12.0,
 (1354, 4414, 2): 6.7,
 (1354, 4415, 2): 17.6,
 (1354, 4417, 2): 17.2,
 (1354, 4418, 2): 7.2,
 (1354, 4419, 2): 16.4,
 (1354, 4420, 2): 15.7,
 (1354, 4421, 2): 11.1,
 (1354, 4423, 2): 10.3,
 (1354, 4425, 2): 15.9,
 (1357, 0, 2): 21.0,
 (1357, 1302, 2): 10.9,
 (1357, 1303, 2): 9.5,
 (1357, 1304, 2): 14.2,
 (1357, 1305, 2): 10.8,
 (1357, 1306, 2): 14.6,
 (1357, 1308, 2): 14.8,
 (1357, 1314, 2): 4.6,
 (1357, 1315, 2): 4.5,
 (1357, 1316, 2): 8.2,
 (1357, 1318, 2): 8.1,
 (1357, 1319, 2): 9.8,
 (1357, 1321, 2): 12.7,
 (1357, 1322, 2): 10.8,
 (1357, 1325, 2): 14.8,
 (1357, 1326, 2): 14.0,
 (1357, 1327, 2): 14.0,
 (1357, 1328, 2): 12.3,
 (1357, 1332, 2): 20.9,
 (1357, 1333, 2): 18.1,
 (1357, 1334, 2): 18.6,
 (1357, 1336, 2): 12.2,
 (1357, 1337, 2): 10.0,
 (1357, 1338, 2): 12.5,
 (1357, 1339, 2): 11.9,
 (1357, 1340, 2): 19.1,
 (1357, 1342, 2): 15.0,
 (1357, 1343, 2): 10.6,
 (1357, 1344, 2): 9.9,
 (1357, 1346, 2): 11.4,
 (1357, 1347, 2): 10.5,
 (1357, 1348, 2): 9.1,
 (1357, 1349, 2): 19.9,
 (1357, 1351, 2): 14.9,
 (1357, 1352, 2): 21.6,
 (1357, 1353, 2): 6.4,
 (1357, 1354, 2): 11.8,
 (1357, 1358, 2): 6.0,
 (1357, 1360, 2): 5.2,
 (1357, 1361, 2): 13.0,
 (1357, 1363, 2): 13.7,
 (1357, 1364, 2): 12.4,
 (1357, 1365, 2): 14.1,
 (1357, 1366, 2): 14.1,
 (1357, 1367, 2): 9.2,
 (1357, 1368, 2): 12.7,
 (1357, 1370, 2): 15.8,
 (1357, 1372, 2): 13.0,
 (1357, 1373, 2): 13.1,
 (1357, 1375, 2): 14.9,
 (1357, 1376, 2): 17.0,
 (1357, 1377, 2): 14.6,
 (1357, 1379, 2): 10.0,
 (1357, 1380, 2): 12.0,
 (1357, 1381, 2): 21.6,
 (1357, 1382, 2): 16.6,
 (1357, 1384, 2): 8.1,
 (1357, 1385, 2): 13.2,
 (1357, 1386, 2): 14.0,
 (1357, 1388, 2): 15.0,
 (1357, 1390, 2): 21.3,
 (1357, 1391, 2): 20.4,
 (1357, 1392, 2): 9.4,
 (1357, 1393, 2): 6.5,
 (1357, 1394, 2): 6.6,
 (1357, 1395, 2): 19.4,
 (1357, 1396, 2): 19.8,
 (1357, 4406, 2): 19.9,
 (1357, 4411, 2): 13.6,
 (1357, 4414, 2): 16.3,
 (1357, 4415, 2): 12.4,
 (1357, 4417, 2): 5.6,
 (1357, 4418, 2): 8.7,
 (1357, 4419, 2): 20.8,
 (1357, 4420, 2): 20.0,
 (1357, 4421, 2): 8.7,
 (1357, 4423, 2): 10.4,
 (1357, 4425, 2): 17.7,
 (1358, 0, 2): 18.3,
 (1358, 1302, 2): 8.2,
 (1358, 1303, 2): 6.8,
 (1358, 1304, 2): 12.0,
 (1358, 1305, 2): 8.1,
 (1358, 1306, 2): 11.8,
 (1358, 1308, 2): 12.0,
 (1358, 1314, 2): 1.8,
 (1358, 1315, 2): 2.3,
 (1358, 1316, 2): 6.1,
 (1358, 1318, 2): 5.2,
 (1358, 1319, 2): 7.0,
 (1358, 1321, 2): 9.8,
 (1358, 1322, 2): 7.9,
 (1358, 1325, 2): 12.0,
 (1358, 1326, 2): 11.2,
 (1358, 1327, 2): 11.2,
 (1358, 1328, 2): 9.5,
 (1358, 1332, 2): 18.1,
 (1358, 1333, 2): 15.3,
 (1358, 1334, 2): 15.8,
 (1358, 1336, 2): 9.5,
 (1358, 1337, 2): 7.2,
 (1358, 1338, 2): 9.8,
 (1358, 1339, 2): 9.1,
 (1358, 1340, 2): 16.4,
 (1358, 1342, 2): 12.4,
 (1358, 1343, 2): 7.8,
 (1358, 1344, 2): 7.1,
 (1358, 1346, 2): 8.7,
 (1358, 1347, 2): 7.6,
 (1358, 1348, 2): 6.2,
 (1358, 1349, 2): 17.2,
 (1358, 1351, 2): 12.2,
 (1358, 1352, 2): 18.8,
 (1358, 1353, 2): 3.6,
 (1358, 1354, 2): 8.9,
 (1358, 1357, 2): 5.6,
 (1358, 1360, 2): 2.5,
 (1358, 1361, 2): 10.2,
 (1358, 1363, 2): 11.0,
 (1358, 1364, 2): 9.8,
 (1358, 1365, 2): 11.5,
 (1358, 1366, 2): 11.3,
 (1358, 1367, 2): 6.4,
 (1358, 1368, 2): 10.0,
 (1358, 1370, 2): 13.2,
 (1358, 1372, 2): 10.2,
 (1358, 1373, 2): 10.2,
 (1358, 1375, 2): 12.0,
 (1358, 1376, 2): 14.1,
 (1358, 1377, 2): 11.8,
 (1358, 1379, 2): 7.2,
 (1358, 1380, 2): 9.2,
 (1358, 1381, 2): 18.8,
 (1358, 1382, 2): 13.7,
 (1358, 1384, 2): 5.4,
 (1358, 1385, 2): 10.5,
 (1358, 1386, 2): 11.2,
 (1358, 1388, 2): 12.3,
 (1358, 1390, 2): 18.5,
 (1358, 1391, 2): 17.5,
 (1358, 1392, 2): 6.7,
 (1358, 1393, 2): 3.6,
 (1358, 1394, 2): 3.8,
 (1358, 1395, 2): 16.6,
 (1358, 1396, 2): 16.9,
 (1358, 4406, 2): 17.1,
 (1358, 4411, 2): 10.8,
 (1358, 4414, 2): 13.6,
 (1358, 4415, 2): 14.0,
 (1358, 4417, 2): 7.0,
 (1358, 4418, 2): 5.8,
 (1358, 4419, 2): 18.0,
 (1358, 4420, 2): 17.2,
 (1358, 4421, 2): 5.8,
 (1358, 4423, 2): 7.6,
 (1358, 4425, 2): 14.9,
 (1360, 0, 2): 17.2,
 (1360, 1302, 2): 7.6,
 (1360, 1303, 2): 6.1,
 (1360, 1304, 2): 13.2,
 (1360, 1305, 2): 10.5,
 (1360, 1306, 2): 12.7,
 (1360, 1308, 2): 10.8,
 (1360, 1314, 2): 1.7,
 (1360, 1315, 2): 4.8,
 (1360, 1316, 2): 8.5,
 (1360, 1318, 2): 5.1,
 (1360, 1319, 2): 5.9,
 (1360, 1321, 2): 8.7,
 (1360, 1322, 2): 6.8,
 (1360, 1325, 2): 10.8,
 (1360, 1326, 2): 10.1,
 (1360, 1327, 2): 10.1,
 (1360, 1328, 2): 8.4,
 (1360, 1332, 2): 17.0,
 (1360, 1333, 2): 14.2,
 (1360, 1334, 2): 14.7,
 (1360, 1336, 2): 9.1,
 (1360, 1337, 2): 6.9,
 (1360, 1338, 2): 9.4,
 (1360, 1339, 2): 8.0,
 (1360, 1340, 2): 15.2,
 (1360, 1342, 2): 12.8,
 (1360, 1343, 2): 7.2,
 (1360, 1344, 2): 8.1,
 (1360, 1346, 2): 7.6,
 (1360, 1347, 2): 6.5,
 (1360, 1348, 2): 5.1,
 (1360, 1349, 2): 16.0,
 (1360, 1351, 2): 13.8,
 (1360, 1352, 2): 15.2,
 (1360, 1353, 2): 2.4,
 (1360, 1354, 2): 7.8,
 (1360, 1357, 2): 8.1,
 (1360, 1358, 2): 6.4,
 (1360, 1361, 2): 9.6,
 (1360, 1363, 2): 12.6,
 (1360, 1364, 2): 9.1,
 (1360, 1365, 2): 10.8,
 (1360, 1366, 2): 10.2,
 (1360, 1367, 2): 5.2,
 (1360, 1368, 2): 6.5,
 (1360, 1370, 2): 12.5,
 (1360, 1372, 2): 9.1,
 (1360, 1373, 2): 9.1,
 (1360, 1375, 2): 10.9,
 (1360, 1376, 2): 13.0,
 (1360, 1377, 2): 10.6,
 (1360, 1379, 2): 7.5,
 (1360, 1380, 2): 8.0,
 (1360, 1381, 2): 17.7,
 (1360, 1382, 2): 12.6,
 (1360, 1384, 2): 5.0,
 (1360, 1385, 2): 9.8,
 (1360, 1386, 2): 10.6,
 (1360, 1388, 2): 13.1,
 (1360, 1390, 2): 17.4,
 (1360, 1391, 2): 16.4,
 (1360, 1392, 2): 4.6,
 (1360, 1393, 2): 2.5,
 (1360, 1394, 2): 2.7,
 (1360, 1395, 2): 15.6,
 (1360, 1396, 2): 15.8,
 (1360, 4406, 2): 16.2,
 (1360, 4411, 2): 9.7,
 (1360, 4414, 2): 15.2,
 (1360, 4415, 2): 14.5,
 (1360, 4417, 2): 8.4,
 (1360, 4418, 2): 4.7,
 (1360, 4419, 2): 16.8,
 (1360, 4420, 2): 16.1,
 (1360, 4421, 2): 4.7,
 (1360, 4423, 2): 6.5,
 (1360, 4425, 2): 13.8,
 (1361, 0, 2): 23.5,
 (1361, 1302, 2): 4.7,
 (1361, 1303, 2): 10.3,
 (1361, 1304, 2): 3.6,
 (1361, 1305, 2): 8.6,
 (1361, 1306, 2): 6.4,
 (1361, 1308, 2): 15.2,
 (1361, 1314, 2): 11.6,
 (1361, 1315, 2): 11.5,
 (1361, 1316, 2): 13.6,
 (1361, 1318, 2): 14.6,
 (1361, 1319, 2): 16.2,
 (1361, 1321, 2): 17.3,
 (1361, 1322, 2): 17.1,
 (1361, 1325, 2): 21.3,
 (1361, 1326, 2): 20.4,
 (1361, 1327, 2): 20.4,
 (1361, 1328, 2): 18.7,
 (1361, 1332, 2): 27.2,
 (1361, 1333, 2): 24.4,
 (1361, 1334, 2): 24.9,
 (1361, 1336, 2): 8.9,
 (1361, 1337, 2): 9.0,
 (1361, 1338, 2): 11.4,
 (1361, 1339, 2): 17.6,
 (1361, 1340, 2): 25.1,
 (1361, 1342, 2): 2.2,
 (1361, 1343, 2): 5.5,
 (1361, 1344, 2): 13.0,
 (1361, 1346, 2): 17.5,
 (1361, 1347, 2): 16.5,
 (1361, 1348, 2): 15.1,
 (1361, 1349, 2): 26.0,
 (1361, 1351, 2): 12.2,
 (1361, 1352, 2): 13.7,
 (1361, 1353, 2): 11.9,
 (1361, 1354, 2): 18.2,
 (1361, 1357, 2): 13.9,
 (1361, 1358, 2): 11.7,
 (1361, 1360, 2): 13.2,
 (1361, 1363, 2): 11.1,
 (1361, 1364, 2): 2.6,
 (1361, 1365, 2): 1.2,
 (1361, 1366, 2): 21.8,
 (1361, 1367, 2): 15.4,
 (1361, 1368, 2): 11.5,
 (1361, 1370, 2): 2.9,
 (1361, 1372, 2): 16.1,
 (1361, 1373, 2): 19.5,
 (1361, 1375, 2): 21.4,
 (1361, 1376, 2): 22.0,
 (1361, 1377, 2): 21.0,
 (1361, 1379, 2): 18.8,
 (1361, 1380, 2): 18.4,
 (1361, 1381, 2): 28.0,
 (1361, 1382, 2): 23.0,
 (1361, 1384, 2): 11.6,
 (1361, 1385, 2): 9.0,
 (1361, 1386, 2): 9.7,
 (1361, 1388, 2): 7.2,
 (1361, 1390, 2): 27.5,
 (1361, 1391, 2): 26.5,
 (1361, 1392, 2): 11.2,
 (1361, 1393, 2): 15.8,
 (1361, 1394, 2): 15.0,
 (1361, 1395, 2): 25.4,
 (1361, 1396, 2): 25.6,
 (1361, 4406, 2): 26.0,
 (1361, 4411, 2): 19.4,
 (1361, 4414, 2): 13.6,
 (1361, 4415, 2): 22.5,
 (1361, 4417, 2): 15.4,
 (1361, 4418, 2): 15.0,
 (1361, 4419, 2): 26.1,
 (1361, 4420, 2): 25.4,
 (1361, 4421, 2): 15.3,
 (1361, 4423, 2): 15.9,
 (1361, 4425, 2): 23.0,
 (1363, 0, 2): 13.6,
 (1363, 1302, 2): 8.5,
 (1363, 1303, 2): 7.2,
 (1363, 1304, 2): 14.0,
 (1363, 1305, 2): 4.9,
 (1363, 1306, 2): 8.6,
 (1363, 1308, 2): 7.3,
 (1363, 1314, 2): 12.6,
 (1363, 1315, 2): 15.2,
 (1363, 1316, 2): 18.8,
 (1363, 1318, 2): 10.9,
 (1363, 1319, 2): 12.3,
 (1363, 1321, 2): 7.9,
 (1363, 1322, 2): 13.2,
 (1363, 1325, 2): 15.6,
 (1363, 1326, 2): 14.5,
 (1363, 1327, 2): 14.5,
 (1363, 1328, 2): 12.8,
 (1363, 1332, 2): 21.5,
 (1363, 1333, 2): 18.6,
 (1363, 1334, 2): 19.1,
 (1363, 1336, 2): 5.8,
 (1363, 1337, 2): 4.4,
 (1363, 1338, 2): 2.1,
 (1363, 1339, 2): 14.0,
 (1363, 1340, 2): 21.6,
 (1363, 1342, 2): 12.7,
 (1363, 1343, 2): 8.2,
 (1363, 1344, 2): 10.3,
 (1363, 1346, 2): 13.9,
 (1363, 1347, 2): 12.8,
 (1363, 1348, 2): 11.4,
 (1363, 1349, 2): 22.5,
 (1363, 1351, 2): 1.2,
 (1363, 1352, 2): 7.9,
 (1363, 1353, 2): 8.6,
 (1363, 1354, 2): 10.3,
 (1363, 1357, 2): 18.6,
 (1363, 1358, 2): 16.9,
 (1363, 1360, 2): 12.2,
 (1363, 1361, 2): 10.4,
 (1363, 1364, 2): 9.9,
 (1363, 1365, 2): 11.8,
 (1363, 1366, 2): 11.6,
 (1363, 1367, 2): 11.6,
 (1363, 1368, 2): 4.9,
 (1363, 1370, 2): 13.4,
 (1363, 1372, 2): 6.6,
 (1363, 1373, 2): 13.5,
 (1363, 1375, 2): 15.3,
 (1363, 1376, 2): 12.3,
 (1363, 1377, 2): 14.9,
 (1363, 1379, 2): 15.4,
 (1363, 1380, 2): 12.7,
 (1363, 1381, 2): 22.2,
 (1363, 1382, 2): 17.0,
 (1363, 1384, 2): 8.8,
 (1363, 1385, 2): 6.9,
 (1363, 1386, 2): 7.1,
 (1363, 1388, 2): 8.9,
 (1363, 1390, 2): 18.4,
 (1363, 1391, 2): 20.8,
 (1363, 1392, 2): 7.5,
 (1363, 1393, 2): 11.9,
 (1363, 1394, 2): 11.4,
 (1363, 1395, 2): 21.7,
 (1363, 1396, 2): 22.0,
 (1363, 4406, 2): 20.8,
 (1363, 4411, 2): 16.0,
 (1363, 4414, 2): 4.3,
 (1363, 4415, 2): 20.3,
 (1363, 4417, 2): 18.6,
 (1363, 4418, 2): 11.0,
 (1363, 4419, 2): 20.9,
 (1363, 4420, 2): 20.2,
 (1363, 4421, 2): 12.1,
 (1363, 4423, 2): 12.8,
 (1363, 4425, 2): 19.7,
 (1364, 0, 2): 23.1,
 (1364, 1302, 2): 3.6,
 (1364, 1303, 2): 9.3,
 (1364, 1304, 2): 7.8,
 (1364, 1305, 2): 7.7,
 (1364, 1306, 2): 4.5,
 (1364, 1308, 2): 15.6,
 (1364, 1314, 2): 13.0,
 (1364, 1315, 2): 15.7,
 (1364, 1316, 2): 19.3,
 (1364, 1318, 2): 12.3,
 (1364, 1319, 2): 13.7,
 (1364, 1321, 2): 16.2,
 (1364, 1322, 2): 14.6,
 (1364, 1325, 2): 19.0,
 (1364, 1326, 2): 17.9,
 (1364, 1327, 2): 17.9,
 (1364, 1328, 2): 16.2,
 (1364, 1332, 2): 24.8,
 (1364, 1333, 2): 21.9,
 (1364, 1334, 2): 22.5,
 (1364, 1336, 2): 7.9,
 (1364, 1337, 2): 8.0,
 (1364, 1338, 2): 10.3,
 (1364, 1339, 2): 15.2,
 (1364, 1340, 2): 22.9,
 (1364, 1342, 2): 5.5,
 (1364, 1343, 2): 4.3,
 (1364, 1344, 2): 11.0,
 (1364, 1346, 2): 15.2,
 (1364, 1347, 2): 14.2,
 (1364, 1348, 2): 12.8,
 (1364, 1349, 2): 23.9,
 (1364, 1351, 2): 11.4,
 (1364, 1352, 2): 15.7,
 (1364, 1353, 2): 10.1,
 (1364, 1354, 2): 15.8,
 (1364, 1357, 2): 19.1,
 (1364, 1358, 2): 17.5,
 (1364, 1360, 2): 12.9,
 (1364, 1361, 2): 5.0,
 (1364, 1363, 2): 10.3,
 (1364, 1365, 2): 6.4,
 (1364, 1366, 2): 20.0,
 (1364, 1367, 2): 13.0,
 (1364, 1368, 2): 10.4,
 (1364, 1370, 2): 7.0,
 (1364, 1372, 2): 15.0,
 (1364, 1373, 2): 16.9,
 (1364, 1375, 2): 18.7,
 (1364, 1376, 2): 20.8,
 (1364, 1377, 2): 18.5,
 (1364, 1379, 2): 17.2,
 (1364, 1380, 2): 15.8,
 (1364, 1381, 2): 25.5,
 (1364, 1382, 2): 20.4,
 (1364, 1384, 2): 9.4,
 (1364, 1385, 2): 6.7,
 (1364, 1386, 2): 7.4,
 (1364, 1388, 2): 5.0,
 (1364, 1390, 2): 25.0,
 (1364, 1391, 2): 24.2,
 (1364, 1392, 2): 8.9,
 (1364, 1393, 2): 13.7,
 (1364, 1394, 2): 12.8,
 (1364, 1395, 2): 23.2,
 (1364, 1396, 2): 23.5,
 (1364, 4406, 2): 24.3,
 (1364, 4411, 2): 17.1,
 (1364, 4414, 2): 12.6,
 (1364, 4415, 2): 21.8,
 (1364, 4417, 2): 19.3,
 (1364, 4418, 2): 12.5,
 (1364, 4419, 2): 24.4,
 (1364, 4420, 2): 23.7,
 (1364, 4421, 2): 12.8,
 (1364, 4423, 2): 14.2,
 (1364, 4425, 2): 21.4,
 (1365, 0, 2): 24.7,
 (1365, 1302, 2): 5.2,
 (1365, 1303, 2): 10.8,
 (1365, 1304, 2): 2.3,
 (1365, 1305, 2): 9.2,
 (1365, 1306, 2): 6.2,
 (1365, 1308, 2): 17.0,
 (1365, 1314, 2): 11.0,
 (1365, 1315, 2): 10.9,
 (1365, 1316, 2): 13.1,
 (1365, 1318, 2): 14.2,
 (1365, 1319, 2): 15.6,
 (1365, 1321, 2): 17.7,
 (1365, 1322, 2): 16.4,
 (1365, 1325, 2): 20.7,
 (1365, 1326, 2): 19.7,
 (1365, 1327, 2): 19.7,
 (1365, 1328, 2): 18.0,
 (1365, 1332, 2): 26.6,
 (1365, 1333, 2): 23.7,
 (1365, 1334, 2): 24.3,
 (1365, 1336, 2): 9.4,
 (1365, 1337, 2): 9.4,
 (1365, 1338, 2): 11.8,
 (1365, 1339, 2): 17.0,
 (1365, 1340, 2): 24.6,
 (1365, 1342, 2): 0.9,
 (1365, 1343, 2): 5.8,
 (1365, 1344, 2): 12.7,
 (1365, 1346, 2): 17.4,
 (1365, 1347, 2): 16.3,
 (1365, 1348, 2): 15.0,
 (1365, 1349, 2): 25.5,
 (1365, 1351, 2): 12.8,
 (1365, 1352, 2): 19.2,
 (1365, 1353, 2): 11.9,
 (1365, 1354, 2): 17.5,
 (1365, 1357, 2): 13.5,
 (1365, 1358, 2): 11.4,
 (1365, 1360, 2): 12.6,
 (1365, 1361, 2): 3.3,
 (1365, 1363, 2): 11.7,
 (1365, 1364, 2): 2.4,
 (1365, 1366, 2): 21.5,
 (1365, 1367, 2): 14.9,
 (1365, 1368, 2): 12.0,
 (1365, 1370, 2): 1.6,
 (1365, 1372, 2): 16.5,
 (1365, 1373, 2): 18.8,
 (1365, 1375, 2): 20.5,
 (1365, 1376, 2): 22.1,
 (1365, 1377, 2): 20.3,
 (1365, 1379, 2): 18.7,
 (1365, 1380, 2): 17.8,
 (1365, 1381, 2): 27.4,
 (1365, 1382, 2): 22.2,
 (1365, 1384, 2): 11.1,
 (1365, 1385, 2): 8.5,
 (1365, 1386, 2): 9.2,
 (1365, 1388, 2): 6.7,
 (1365, 1390, 2): 26.8,
 (1365, 1391, 2): 26.1,
 (1365, 1392, 2): 10.7,
 (1365, 1393, 2): 15.2,
 (1365, 1394, 2): 14.6,
 (1365, 1395, 2): 24.8,
 (1365, 1396, 2): 25.0,
 (1365, 4406, 2): 26.0,
 (1365, 4411, 2): 18.9,
 (1365, 4414, 2): 14.1,
 (1365, 4415, 2): 22.0,
 (1365, 4417, 2): 15.0,
 (1365, 4418, 2): 14.3,
 (1365, 4419, 2): 26.3,
 (1365, 4420, 2): 25.6,
 (1365, 4421, 2): 15.4,
 (1365, 4423, 2): 16.0,
 (1365, 4425, 2): 23.0,
 (1366, 0, 2): 16.2,
 (1366, 1302, 2): 16.6,
 (1366, 1303, 2): 15.4,
 (1366, 1304, 2): 22.1,
 (1366, 1305, 2): 12.9,
 (1366, 1306, 2): 16.7,
 (1366, 1308, 2): 8.6,
 (1366, 1314, 2): 12.6,
 (1366, 1315, 2): 13.7,
 (1366, 1316, 2): 13.8,
 (1366, 1318, 2): 9.1,
 (1366, 1319, 2): 6.3,
 (1366, 1321, 2): 4.4,
 (1366, 1322, 2): 8.5,
 (1366, 1325, 2): 5.5,
 (1366, 1326, 2): 4.5,
 (1366, 1327, 2): 4.5,
 (1366, 1328, 2): 4.3,
 (1366, 1332, 2): 11.5,
 (1366, 1333, 2): 8.6,
 (1366, 1334, 2): 9.2,
 (1366, 1336, 2): 13.9,
 (1366, 1337, 2): 12.6,
 (1366, 1338, 2): 10.2,
 (1366, 1339, 2): 9.8,
 (1366, 1340, 2): 14.0,
 (1366, 1342, 2): 20.7,
 (1366, 1343, 2): 16.3,
 (1366, 1344, 2): 18.4,
 (1366, 1346, 2): 15.1,
 (1366, 1347, 2): 14.1,
 (1366, 1348, 2): 12.7,
 (1366, 1349, 2): 14.4,
 (1366, 1351, 2): 9.4,
 (1366, 1352, 2): 15.8,
 (1366, 1353, 2): 10.1,
 (1366, 1354, 2): 6.8,
 (1366, 1357, 2): 17.1,
 (1366, 1358, 2): 15.5,
 (1366, 1360, 2): 13.2,
 (1366, 1361, 2): 18.4,
 (1366, 1363, 2): 10.2,
 (1366, 1364, 2): 17.8,
 (1366, 1365, 2): 19.8,
 (1366, 1367, 2): 12.9,
 (1366, 1368, 2): 12.9,
 (1366, 1370, 2): 21.4,
 (1366, 1372, 2): 6.3,
 (1366, 1373, 2): 5.6,
 (1366, 1375, 2): 6.5,
 (1366, 1376, 2): 6.8,
 (1366, 1377, 2): 4.6,
 (1366, 1379, 2): 18.2,
 (1366, 1380, 2): 2.5,
 (1366, 1381, 2): 12.1,
 (1366, 1382, 2): 8.2,
 (1366, 1384, 2): 15.8,
 (1366, 1385, 2): 14.9,
 (1366, 1386, 2): 15.1,
 (1366, 1388, 2): 16.9,
 (1366, 1390, 2): 11.7,
 (1366, 1391, 2): 10.7,
 (1366, 1392, 2): 15.5,
 (1366, 1393, 2): 13.2,
 (1366, 1394, 2): 9.4,
 (1366, 1395, 2): 12.2,
 (1366, 1396, 2): 12.6,
 (1366, 4406, 2): 10.9,
 (1366, 4411, 2): 7.1,
 (1366, 4414, 2): 10.4,
 (1366, 4415, 2): 17.3,
 (1366, 4417, 2): 19.4,
 (1366, 4418, 2): 7.4,
 (1366, 4419, 2): 11.0,
 (1366, 4420, 2): 10.3,
 (1366, 4421, 2): 11.2,
 (1366, 4423, 2): 10.4,
 (1366, 4425, 2): 12.0,
 (1367, 0, 2): 17.8,
 (1367, 1302, 2): 6.4,
 (1367, 1303, 2): 5.4,
 (1367, 1304, 2): 12.6,
 (1367, 1305, 2): 8.8,
 (1367, 1306, 2): 11.4,
 (1367, 1308, 2): 11.7,
 (1367, 1314, 2): 4.5,
 (1367, 1315, 2): 5.6,
 (1367, 1316, 2): 9.3,
 (1367, 1318, 2): 4.5,
 (1367, 1319, 2): 6.0,
 (1367, 1321, 2): 8.7,
 (1367, 1322, 2): 6.8,
 (1367, 1325, 2): 11.3,
 (1367, 1326, 2): 10.2,
 (1367, 1327, 2): 10.2,
 (1367, 1328, 2): 8.5,
 (1367, 1332, 2): 17.2,
 (1367, 1333, 2): 14.3,
 (1367, 1334, 2): 14.8,
 (1367, 1336, 2): 9.3,
 (1367, 1337, 2): 7.2,
 (1367, 1338, 2): 9.6,
 (1367, 1339, 2): 7.8,
 (1367, 1340, 2): 15.2,
 (1367, 1342, 2): 11.2,
 (1367, 1343, 2): 6.1,
 (1367, 1344, 2): 7.7,
 (1367, 1346, 2): 2.9,
 (1367, 1347, 2): 1.8,
 (1367, 1348, 2): 0.4,
 (1367, 1349, 2): 16.2,
 (1367, 1351, 2): 10.6,
 (1367, 1352, 2): 13.9,
 (1367, 1353, 2): 2.7,
 (1367, 1354, 2): 8.1,
 (1367, 1357, 2): 9.2,
 (1367, 1358, 2): 7.5,
 (1367, 1360, 2): 5.2,
 (1367, 1361, 2): 8.9,
 (1367, 1363, 2): 9.8,
 (1367, 1364, 2): 8.8,
 (1367, 1365, 2): 10.6,
 (1367, 1366, 2): 9.9,
 (1367, 1368, 2): 6.4,
 (1367, 1370, 2): 12.3,
 (1367, 1372, 2): 9.3,
 (1367, 1373, 2): 8.7,
 (1367, 1375, 2): 10.5,
 (1367, 1376, 2): 13.1,
 (1367, 1377, 2): 10.3,
 (1367, 1379, 2): 4.6,
 (1367, 1380, 2): 8.2,
 (1367, 1381, 2): 17.2,
 (1367, 1382, 2): 12.1,
 (1367, 1384, 2): 5.6,
 (1367, 1385, 2): 8.5,
 (1367, 1386, 2): 9.3,
 (1367, 1388, 2): 8.6,
 (1367, 1390, 2): 16.6,
 (1367, 1391, 2): 15.8,
 (1367, 1392, 2): 8.9,
 (1367, 1393, 2): 2.1,
 (1367, 1394, 2): 1.5,
 (1367, 1395, 2): 16.6,
 (1367, 1396, 2): 16.9,
 (1367, 4406, 2): 15.4,
 (1367, 4411, 2): 8.6,
 (1367, 4414, 2): 14.4,
 (1367, 4415, 2): 13.7,
 (1367, 4417, 2): 9.2,
 (1367, 4418, 2): 4.8,
 (1367, 4419, 2): 16.0,
 (1367, 4420, 2): 15.3,
 (1367, 4421, 2): 4.7,
 (1367, 4423, 2): 6.4,
 (1367, 4425, 2): 16.4,
 (1368, 0, 2): 18.2,
 (1368, 1302, 2): 7.8,
 (1368, 1303, 2): 6.7,
 (1368, 1304, 2): 13.3,
 (1368, 1305, 2): 3.2,
 (1368, 1306, 2): 8.0,
 (1368, 1308, 2): 9.2,
 (1368, 1314, 2): 10.9,
 (1368, 1315, 2): 11.6,
 (1368, 1316, 2): 13.6,
 (1368, 1318, 2): 7.2,
 (1368, 1319, 2): 8.2,
 (1368, 1321, 2): 10.1,
 (1368, 1322, 2): 9.2,
 (1368, 1325, 2): 13.1,
 (1368, 1326, 2): 11.8,
 (1368, 1327, 2): 11.8,
 (1368, 1328, 2): 10.2,
 (1368, 1332, 2): 18.7,
 (1368, 1333, 2): 15.8,
 (1368, 1334, 2): 16.4,
 (1368, 1336, 2): 5.3,
 (1368, 1337, 2): 2.8,
 (1368, 1338, 2): 3.2,
 (1368, 1339, 2): 9.6,
 (1368, 1340, 2): 17.6,
 (1368, 1342, 2): 13.0,
 (1368, 1343, 2): 7.6,
 (1368, 1344, 2): 8.8,
 (1368, 1346, 2): 10.2,
 (1368, 1347, 2): 9.1,
 (1368, 1348, 2): 7.7,
 (1368, 1349, 2): 18.2,
 (1368, 1351, 2): 5.6,
 (1368, 1352, 2): 8.3,
 (1368, 1353, 2): 4.4,
 (1368, 1354, 2): 10.4,
 (1368, 1357, 2): 15.2,
 (1368, 1358, 2): 13.2,
 (1368, 1360, 2): 10.5,
 (1368, 1361, 2): 9.7,
 (1368, 1363, 2): 4.4,
 (1368, 1364, 2): 9.2,
 (1368, 1365, 2): 11.0,
 (1368, 1366, 2): 13.8,
 (1368, 1367, 2): 7.8,
 (1368, 1370, 2): 12.7,
 (1368, 1372, 2): 8.9,
 (1368, 1373, 2): 10.9,
 (1368, 1375, 2): 12.7,
 (1368, 1376, 2): 14.5,
 (1368, 1377, 2): 12.5,
 (1368, 1379, 2): 13.3,
 (1368, 1380, 2): 10.4,
 (1368, 1381, 2): 19.4,
 (1368, 1382, 2): 14.4,
 (1368, 1384, 2): 4.6,
 (1368, 1385, 2): 6.3,
 (1368, 1386, 2): 5.6,
 (1368, 1388, 2): 8.2,
 (1368, 1390, 2): 19.0,
 (1368, 1391, 2): 18.0,
 (1368, 1392, 2): 3.4,
 (1368, 1393, 2): 8.3,
 (1368, 1394, 2): 7.5,
 (1368, 1395, 2): 17.4,
 (1368, 1396, 2): 17.6,
 (1368, 4406, 2): 17.9,
 (1368, 4411, 2): 11.3,
 (1368, 4414, 2): 6.7,
 (1368, 4415, 2): 17.2,
 (1368, 4417, 2): 15.3,
 (1368, 4418, 2): 7.0,
 (1368, 4419, 2): 18.4,
 (1368, 4420, 2): 17.6,
 (1368, 4421, 2): 8.4,
 (1368, 4423, 2): 9.0,
 (1368, 4425, 2): 15.7,
 (1370, 0, 2): 27.5,
 (1370, 1302, 2): 7.9,
 (1370, 1303, 2): 13.6,
 (1370, 1304, 2): 0.7,
 (1370, 1305, 2): 12.0,
 (1370, 1306, 2): 9.0,
 (1370, 1308, 2): 20.3,
 (1370, 1314, 2): 12.4,
 (1370, 1315, 2): 12.4,
 (1370, 1316, 2): 14.4,
 (1370, 1318, 2): 16.9,
 (1370, 1319, 2): 18.3,
 (1370, 1321, 2): 21.3,
 (1370, 1322, 2): 19.2,
 (1370, 1325, 2): 23.3,
 (1370, 1326, 2): 22.1,
 (1370, 1327, 2): 22.1,
 (1370, 1328, 2): 20.5,
 (1370, 1332, 2): 29.0,
 (1370, 1333, 2): 26.1,
 (1370, 1334, 2): 26.6,
 (1370, 1336, 2): 12.2,
 (1370, 1337, 2): 12.3,
 (1370, 1338, 2): 14.7,
 (1370, 1339, 2): 19.4,
 (1370, 1340, 2): 27.7,
 (1370, 1342, 2): 1.6,
 (1370, 1343, 2): 8.7,
 (1370, 1344, 2): 15.3,
 (1370, 1346, 2): 19.9,
 (1370, 1347, 2): 18.8,
 (1370, 1348, 2): 17.4,
 (1370, 1349, 2): 28.3,
 (1370, 1351, 2): 16.0,
 (1370, 1352, 2): 19.8,
 (1370, 1353, 2): 14.2,
 (1370, 1354, 2): 20.1,
 (1370, 1357, 2): 15.1,
 (1370, 1358, 2): 12.5,
 (1370, 1360, 2): 13.6,
 (1370, 1361, 2): 6.0,
 (1370, 1363, 2): 14.8,
 (1370, 1364, 2): 5.5,
 (1370, 1365, 2): 5.3,
 (1370, 1366, 2): 25.2,
 (1370, 1367, 2): 17.6,
 (1370, 1368, 2): 14.5,
 (1370, 1372, 2): 20.0,
 (1370, 1373, 2): 21.3,
 (1370, 1375, 2): 23.1,
 (1370, 1376, 2): 25.7,
 (1370, 1377, 2): 22.8,
 (1370, 1379, 2): 19.9,
 (1370, 1380, 2): 20.2,
 (1370, 1381, 2): 29.8,
 (1370, 1382, 2): 24.8,
 (1370, 1384, 2): 13.7,
 (1370, 1385, 2): 11.0,
 (1370, 1386, 2): 11.8,
 (1370, 1388, 2): 9.4,
 (1370, 1390, 2): 29.2,
 (1370, 1391, 2): 28.2,
 (1370, 1392, 2): 13.3,
 (1370, 1393, 2): 18.0,
 (1370, 1394, 2): 17.3,
 (1370, 1395, 2): 27.2,
 (1370, 1396, 2): 27.1,
 (1370, 4406, 2): 27.9,
 (1370, 4411, 2): 21.1,
 (1370, 4414, 2): 17.6,
 (1370, 4415, 2): 23.7,
 (1370, 4417, 2): 16.6,
 (1370, 4418, 2): 16.9,
 (1370, 4419, 2): 28.7,
 (1370, 4420, 2): 28.0,
 (1370, 4421, 2): 18.1,
 (1370, 4423, 2): 18.8,
 (1370, 4425, 2): 25.7,
 (1372, 0, 2): 13.1,
 (1372, 1302, 2): 10.4,
 (1372, 1303, 2): 9.2,
 (1372, 1304, 2): 15.8,
 (1372, 1305, 2): 6.4,
 (1372, 1306, 2): 10.6,
 (1372, 1308, 2): 2.5,
 (1372, 1314, 2): 14.1,
 (1372, 1315, 2): 13.6,
 (1372, 1316, 2): 13.9,
 (1372, 1318, 2): 9.2,
 (1372, 1319, 2): 8.3,
 (1372, 1321, 2): 3.3,
 (1372, 1322, 2): 6.8,
 (1372, 1325, 2): 10.9,
 (1372, 1326, 2): 9.7,
 (1372, 1327, 2): 9.7,
 (1372, 1328, 2): 8.1,
 (1372, 1332, 2): 16.6,
 (1372, 1333, 2): 13.7,
 (1372, 1334, 2): 14.2,
 (1372, 1336, 2): 7.8,
 (1372, 1337, 2): 6.0,
 (1372, 1338, 2): 3.7,
 (1372, 1339, 2): 9.7,
 (1372, 1340, 2): 18.4,
 (1372, 1342, 2): 14.5,
 (1372, 1343, 2): 10.1,
 (1372, 1344, 2): 12.0,
 (1372, 1346, 2): 15.4,
 (1372, 1347, 2): 14.4,
 (1372, 1348, 2): 13.0,
 (1372, 1349, 2): 19.3,
 (1372, 1351, 2): 3.4,
 (1372, 1352, 2): 9.9,
 (1372, 1353, 2): 10.1,
 (1372, 1354, 2): 5.6,
 (1372, 1357, 2): 17.3,
 (1372, 1358, 2): 15.3,
 (1372, 1360, 2): 13.6,
 (1372, 1361, 2): 12.3,
 (1372, 1363, 2): 3.8,
 (1372, 1364, 2): 11.7,
 (1372, 1365, 2): 13.5,
 (1372, 1366, 2): 6.9,
 (1372, 1367, 2): 13.1,
 (1372, 1368, 2): 6.5,
 (1372, 1370, 2): 15.2,
 (1372, 1373, 2): 8.8,
 (1372, 1375, 2): 10.6,
 (1372, 1376, 2): 7.7,
 (1372, 1377, 2): 10.4,
 (1372, 1379, 2): 16.4,
 (1372, 1380, 2): 7.9,
 (1372, 1381, 2): 17.3,
 (1372, 1382, 2): 12.2,
 (1372, 1384, 2): 10.4,
 (1372, 1385, 2): 8.8,
 (1372, 1386, 2): 8.8,
 (1372, 1388, 2): 10.8,
 (1372, 1390, 2): 13.6,
 (1372, 1391, 2): 15.9,
 (1372, 1392, 2): 9.2,
 (1372, 1393, 2): 13.6,
 (1372, 1394, 2): 9.5,
 (1372, 1395, 2): 16.6,
 (1372, 1396, 2): 16.9,
 (1372, 4406, 2): 15.4,
 (1372, 4411, 2): 11.3,
 (1372, 4414, 2): 3.9,
 (1372, 4415, 2): 17.6,
 (1372, 4417, 2): 17.5,
 (1372, 4418, 2): 7.2,
 (1372, 4419, 2): 16.2,
 (1372, 4420, 2): 15.5,
 (1372, 4421, 2): 10.5,
 (1372, 4423, 2): 9.7,
 (1372, 4425, 2): 16.5,
 (1373, 0, 2): 13.1,
 (1373, 1302, 2): 13.8,
 (1373, 1303, 2): 12.8,
 (1373, 1304, 2): 19.7,
 (1373, 1305, 2): 16.2,
 (1373, 1306, 2): 18.9,
 (1373, 1308, 2): 8.8,
 (1373, 1314, 2): 8.0,
 (1373, 1315, 2): 9.0,
 (1373, 1316, 2): 9.0,
 (1373, 1318, 2): 4.6,
 (1373, 1319, 2): 1.6,
 (1373, 1321, 2): 4.5,
 (1373, 1322, 2): 2.6,
 (1373, 1325, 2): 5.4,
 (1373, 1326, 2): 4.1,
 (1373, 1327, 2): 4.0,
 (1373, 1328, 2): 2.5,
 (1373, 1332, 2): 10.9,
 (1373, 1333, 2): 8.0,
 (1373, 1334, 2): 8.6,
 (1373, 1336, 2): 14.9,
 (1373, 1337, 2): 14.2,
 (1373, 1338, 2): 10.8,
 (1373, 1339, 2): 5.3,
 (1373, 1340, 2): 12.8,
 (1373, 1342, 2): 18.3,
 (1373, 1343, 2): 13.5,
 (1373, 1344, 2): 15.0,
 (1373, 1346, 2): 10.8,
 (1373, 1347, 2): 9.7,
 (1373, 1348, 2): 8.3,
 (1373, 1349, 2): 13.1,
 (1373, 1351, 2): 9.8,
 (1373, 1352, 2): 17.0,
 (1373, 1353, 2): 5.3,
 (1373, 1354, 2): 5.7,
 (1373, 1357, 2): 12.5,
 (1373, 1358, 2): 10.5,
 (1373, 1360, 2): 8.4,
 (1373, 1361, 2): 16.1,
 (1373, 1363, 2): 10.9,
 (1373, 1364, 2): 15.6,
 (1373, 1365, 2): 17.4,
 (1373, 1366, 2): 5.6,
 (1373, 1367, 2): 8.4,
 (1373, 1368, 2): 13.0,
 (1373, 1370, 2): 19.0,
 (1373, 1372, 2): 6.8,
 (1373, 1375, 2): 5.0,
 (1373, 1376, 2): 10.7,
 (1373, 1377, 2): 4.8,
 (1373, 1379, 2): 13.4,
 (1373, 1380, 2): 2.3,
 (1373, 1381, 2): 11.6,
 (1373, 1382, 2): 6.6,
 (1373, 1384, 2): 12.4,
 (1373, 1385, 2): 15.6,
 (1373, 1386, 2): 16.3,
 (1373, 1388, 2): 19.1,
 (1373, 1390, 2): 11.1,
 (1373, 1391, 2): 10.2,
 (1373, 1392, 2): 8.8,
 (1373, 1393, 2): 8.8,
 (1373, 1394, 2): 4.9,
 (1373, 1395, 2): 11.0,
 (1373, 1396, 2): 11.3,
 (1373, 4406, 2): 9.8,
 (1373, 4411, 2): 5.6,
 (1373, 4414, 2): 11.4,
 (1373, 4415, 2): 13.1,
 (1373, 4417, 2): 12.7,
 (1373, 4418, 2): 2.9,
 (1373, 4419, 2): 10.5,
 (1373, 4420, 2): 9.8,
 (1373, 4421, 2): 6.0,
 (1373, 4423, 2): 5.3,
 (1373, 4425, 2): 10.8,
 (1375, 0, 2): 14.5,
 (1375, 1302, 2): 14.6,
 (1375, 1303, 2): 14.2,
 (1375, 1304, 2): 20.5,
 (1375, 1305, 2): 17.1,
 (1375, 1306, 2): 19.7,
 (1375, 1308, 2): 10.2,
 (1375, 1314, 2): 8.7,
 (1375, 1315, 2): 9.7,
 (1375, 1316, 2): 9.6,
 (1375, 1318, 2): 5.4,
 (1375, 1319, 2): 2.3,
 (1375, 1321, 2): 5.9,
 (1375, 1322, 2): 4.0,
 (1375, 1325, 2): 7.1,
 (1375, 1326, 2): 5.4,
 (1375, 1327, 2): 5.4,
 (1375, 1328, 2): 3.8,
 (1375, 1332, 2): 12.0,
 (1375, 1333, 2): 9.2,
 (1375, 1334, 2): 9.7,
 (1375, 1336, 2): 16.4,
 (1375, 1337, 2): 13.5,
 (1375, 1338, 2): 12.6,
 (1375, 1339, 2): 4.5,
 (1375, 1340, 2): 10.6,
 (1375, 1342, 2): 17.7,
 (1375, 1343, 2): 13.2,
 (1375, 1344, 2): 13.8,
 (1375, 1346, 2): 10.5,
 (1375, 1347, 2): 9.3,
 (1375, 1348, 2): 7.8,
 (1375, 1349, 2): 11.5,
 (1375, 1351, 2): 11.1,
 (1375, 1352, 2): 17.8,
 (1375, 1353, 2): 6.1,
 (1375, 1354, 2): 7.1,
 (1375, 1357, 2): 13.0,
 (1375, 1358, 2): 11.3,
 (1375, 1360, 2): 9.5,
 (1375, 1361, 2): 15.6,
 (1375, 1363, 2): 12.6,
 (1375, 1364, 2): 15.0,
 (1375, 1365, 2): 16.8,
 (1375, 1366, 2): 6.9,
 (1375, 1367, 2): 8.0,
 (1375, 1368, 2): 12.4,
 (1375, 1370, 2): 18.4,
 (1375, 1372, 2): 8.4,
 (1375, 1373, 2): 4.4,
 (1375, 1376, 2): 12.1,
 (1375, 1377, 2): 6.0,
 (1375, 1379, 2): 12.2,
 (1375, 1380, 2): 4.0,
 (1375, 1381, 2): 12.6,
 (1375, 1382, 2): 1.8,
 (1375, 1384, 2): 11.6,
 (1375, 1385, 2): 14.8,
 (1375, 1386, 2): 15.5,
 (1375, 1388, 2): 18.3,
 (1375, 1390, 2): 12.5,
 (1375, 1391, 2): 11.6,
 (1375, 1392, 2): 9.6,
 (1375, 1393, 2): 8.5,
 (1375, 1394, 2): 5.7,
 (1375, 1395, 2): 11.1,
 (1375, 1396, 2): 10.7,
 (1375, 4406, 2): 11.2,
 (1375, 4411, 2): 1.9,
 (1375, 4414, 2): 13.0,
 (1375, 4415, 2): 13.3,
 (1375, 4417, 2): 13.4,
 (1375, 4418, 2): 3.5,
 (1375, 4419, 2): 11.9,
 (1375, 4420, 2): 11.1,
 (1375, 4421, 2): 7.0,
 (1375, 4423, 2): 6.4,
 (1375, 4425, 2): 9.4,
 (1376, 0, 2): 17.6,
 (1376, 1302, 2): 17.4,
 (1376, 1303, 2): 16.0,
 (1376, 1304, 2): 22.5,
 (1376, 1305, 2): 13.6,
 (1376, 1306, 2): 17.4,
 (1376, 1308, 2): 8.4,
 (1376, 1314, 2): 14.6,
 (1376, 1315, 2): 15.4,
 (1376, 1316, 2): 15.7,
 (1376, 1318, 2): 11.1,
 (1376, 1319, 2): 10.4,
 (1376, 1321, 2): 4.8,
 (1376, 1322, 2): 8.3,
 (1376, 1325, 2): 10.2,
 (1376, 1326, 2): 8.9,
 (1376, 1327, 2): 8.9,
 (1376, 1328, 2): 9.6,
 (1376, 1332, 2): 15.5,
 (1376, 1333, 2): 12.7,
 (1376, 1334, 2): 13.2,
 (1376, 1336, 2): 14.6,
 (1376, 1337, 2): 13.4,
 (1376, 1338, 2): 10.8,
 (1376, 1339, 2): 12.0,
 (1376, 1340, 2): 17.4,
 (1376, 1342, 2): 21.2,
 (1376, 1343, 2): 17.2,
 (1376, 1344, 2): 19.1,
 (1376, 1346, 2): 16.3,
 (1376, 1347, 2): 15.2,
 (1376, 1348, 2): 13.7,
 (1376, 1349, 2): 17.8,
 (1376, 1351, 2): 9.4,
 (1376, 1352, 2): 15.9,
 (1376, 1353, 2): 11.9,
 (1376, 1354, 2): 7.1,
 (1376, 1357, 2): 18.8,
 (1376, 1358, 2): 17.0,
 (1376, 1360, 2): 15.3,
 (1376, 1361, 2): 19.1,
 (1376, 1363, 2): 10.9,
 (1376, 1364, 2): 18.6,
 (1376, 1365, 2): 20.3,
 (1376, 1366, 2): 8.4,
 (1376, 1367, 2): 13.9,
 (1376, 1368, 2): 13.9,
 (1376, 1370, 2): 22.0,
 (1376, 1372, 2): 6.6,
 (1376, 1373, 2): 10.3,
 (1376, 1375, 2): 10.9,
 (1376, 1377, 2): 9.5,
 (1376, 1379, 2): 18.8,
 (1376, 1380, 2): 7.0,
 (1376, 1381, 2): 16.2,
 (1376, 1382, 2): 12.4,
 (1376, 1384, 2): 17.5,
 (1376, 1385, 2): 15.6,
 (1376, 1386, 2): 16.1,
 (1376, 1388, 2): 17.7,
 (1376, 1390, 2): 13.7,
 (1376, 1391, 2): 15.0,
 (1376, 1392, 2): 16.5,
 (1376, 1393, 2): 14.4,
 (1376, 1394, 2): 11.5,
 (1376, 1395, 2): 15.8,
 (1376, 1396, 2): 16.0,
 (1376, 4406, 2): 14.6,
 (1376, 4411, 2): 11.5,
 (1376, 4414, 2): 11.3,
 (1376, 4415, 2): 19.0,
 (1376, 4417, 2): 19.1,
 (1376, 4418, 2): 9.2,
 (1376, 4419, 2): 16.2,
 (1376, 4420, 2): 15.4,
 (1376, 4421, 2): 12.7,
 (1376, 4423, 2): 12.1,
 (1376, 4425, 2): 15.6,
 (1377, 0, 2): 18.4,
 (1377, 1302, 2): 16.0,
 (1377, 1303, 2): 14.9,
 (1377, 1304, 2): 21.7,
 (1377, 1305, 2): 15.4,
 (1377, 1306, 2): 19.3,
 (1377, 1308, 2): 10.3,
 (1377, 1314, 2): 11.3,
 (1377, 1315, 2): 12.1,
 (1377, 1316, 2): 12.1,
 (1377, 1318, 2): 7.9,
 (1377, 1319, 2): 4.8,
 (1377, 1321, 2): 6.6,
 (1377, 1322, 2): 7.6,
 (1377, 1325, 2): 2.0,
 (1377, 1326, 2): 1.4,
 (1377, 1327, 2): 1.4,
 (1377, 1328, 2): 3.2,
 (1377, 1332, 2): 8.0,
 (1377, 1333, 2): 5.3,
 (1377, 1334, 2): 5.7,
 (1377, 1336, 2): 16.4,
 (1377, 1337, 2): 15.0,
 (1377, 1338, 2): 12.4,
 (1377, 1339, 2): 8.7,
 (1377, 1340, 2): 10.2,
 (1377, 1342, 2): 20.3,
 (1377, 1343, 2): 15.7,
 (1377, 1344, 2): 16.5,
 (1377, 1346, 2): 13.2,
 (1377, 1347, 2): 12.0,
 (1377, 1348, 2): 10.4,
 (1377, 1349, 2): 10.4,
 (1377, 1351, 2): 11.0,
 (1377, 1352, 2): 17.6,
 (1377, 1353, 2): 8.6,
 (1377, 1354, 2): 8.9,
 (1377, 1357, 2): 15.4,
 (1377, 1358, 2): 13.6,
 (1377, 1360, 2): 12.0,
 (1377, 1361, 2): 18.0,
 (1377, 1363, 2): 12.5,
 (1377, 1364, 2): 17.6,
 (1377, 1365, 2): 19.4,
 (1377, 1366, 2): 10.2,
 (1377, 1367, 2): 10.6,
 (1377, 1368, 2): 15.5,
 (1377, 1370, 2): 21.0,
 (1377, 1372, 2): 8.5,
 (1377, 1373, 2): 4.4,
 (1377, 1375, 2): 5.3,
 (1377, 1376, 2): 9.1,
 (1377, 1379, 2): 15.6,
 (1377, 1380, 2): 2.6,
 (1377, 1381, 2): 8.8,
 (1377, 1382, 2): 7.0,
 (1377, 1384, 2): 14.2,
 (1377, 1385, 2): 17.4,
 (1377, 1386, 2): 18.1,
 (1377, 1388, 2): 19.4,
 (1377, 1390, 2): 8.6,
 (1377, 1391, 2): 7.5,
 (1377, 1392, 2): 12.2,
 (1377, 1393, 2): 11.1,
 (1377, 1394, 2): 8.2,
 (1377, 1395, 2): 8.4,
 (1377, 1396, 2): 8.7,
 (1377, 4406, 2): 7.2,
 (1377, 4411, 2): 6.0,
 (1377, 4414, 2): 10.2,
 (1377, 4415, 2): 15.7,
 (1377, 4417, 2): 15.8,
 (1377, 4418, 2): 6.1,
 (1377, 4419, 2): 7.9,
 (1377, 4420, 2): 7.2,
 (1377, 4421, 2): 9.4,
 (1377, 4423, 2): 8.8,
 (1377, 4425, 2): 8.2,
 (1379, 0, 2): 23.3,
 (1379, 1302, 2): 11.1,
 (1379, 1303, 2): 9.9,
 (1379, 1304, 2): 16.7,
 (1379, 1305, 2): 13.2,
 (1379, 1306, 2): 15.8,
 (1379, 1308, 2): 17.3,
 (1379, 1314, 2): 7.6,
 (1379, 1315, 2): 8.6,
 (1379, 1316, 2): 12.3,
 (1379, 1318, 2): 8.9,
 (1379, 1319, 2): 12.0,
 (1379, 1321, 2): 14.8,
 (1379, 1322, 2): 12.9,
 (1379, 1325, 2): 17.0,
 (1379, 1326, 2): 15.6,
 (1379, 1327, 2): 15.6,
 (1379, 1328, 2): 14.0,
 (1379, 1332, 2): 22.3,
 (1379, 1333, 2): 19.5,
 (1379, 1334, 2): 20.0,
 (1379, 1336, 2): 15.3,
 (1379, 1337, 2): 13.4,
 (1379, 1338, 2): 15.9,
 (1379, 1339, 2): 11.3,
 (1379, 1340, 2): 18.8,
 (1379, 1342, 2): 15.4,
 (1379, 1343, 2): 10.8,
 (1379, 1344, 2): 11.4,
 (1379, 1346, 2): 1.6,
 (1379, 1347, 2): 2.2,
 (1379, 1348, 2): 4.1,
 (1379, 1349, 2): 19.6,
 (1379, 1351, 2): 17.0,
 (1379, 1352, 2): 23.3,
 (1379, 1353, 2): 6.6,
 (1379, 1354, 2): 14.2,
 (1379, 1357, 2): 9.6,
 (1379, 1358, 2): 10.1,
 (1379, 1360, 2): 8.3,
 (1379, 1361, 2): 13.2,
 (1379, 1363, 2): 15.9,
 (1379, 1364, 2): 12.7,
 (1379, 1365, 2): 14.4,
 (1379, 1366, 2): 15.9,
 (1379, 1367, 2): 4.0,
 (1379, 1368, 2): 10.0,
 (1379, 1370, 2): 16.1,
 (1379, 1372, 2): 15.5,
 (1379, 1373, 2): 14.7,
 (1379, 1375, 2): 15.1,
 (1379, 1376, 2): 19.2,
 (1379, 1377, 2): 16.2,
 (1379, 1380, 2): 13.9,
 (1379, 1381, 2): 23.0,
 (1379, 1382, 2): 17.0,
 (1379, 1384, 2): 11.4,
 (1379, 1385, 2): 12.4,
 (1379, 1386, 2): 13.1,
 (1379, 1388, 2): 16.0,
 (1379, 1390, 2): 22.8,
 (1379, 1391, 2): 19.8,
 (1379, 1392, 2): 12.2,
 (1379, 1393, 2): 6.2,
 (1379, 1394, 2): 5.5,
 (1379, 1395, 2): 18.9,
 (1379, 1396, 2): 18.7,
 (1379, 4406, 2): 19.6,
 (1379, 4411, 2): 13.0,
 (1379, 4414, 2): 18.8,
 (1379, 4415, 2): 11.4,
 (1379, 4417, 2): 7.4,
 (1379, 4418, 2): 11.1,
 (1379, 4419, 2): 22.2,
 (1379, 4420, 2): 21.4,
 (1379, 4421, 2): 8.6,
 (1379, 4423, 2): 7.7,
 (1379, 4425, 2): 17.3,
 (1380, 0, 2): 17.4,
 (1380, 1302, 2): 15.0,
 (1380, 1303, 2): 13.9,
 (1380, 1304, 2): 20.7,
 (1380, 1305, 2): 17.2,
 (1380, 1306, 2): 19.9,
 (1380, 1308, 2): 10.4,
 (1380, 1314, 2): 10.4,
 (1380, 1315, 2): 11.2,
 (1380, 1316, 2): 11.2,
 (1380, 1318, 2): 7.0,
 (1380, 1319, 2): 3.9,
 (1380, 1321, 2): 6.7,
 (1380, 1322, 2): 6.8,
 (1380, 1325, 2): 4.0,
 (1380, 1326, 2): 2.5,
 (1380, 1327, 2): 2.5,
 (1380, 1328, 2): 2.3,
 (1380, 1332, 2): 9.2,
 (1380, 1333, 2): 6.4,
 (1380, 1334, 2): 6.8,
 (1380, 1336, 2): 16.6,
 (1380, 1337, 2): 17.5,
 (1380, 1338, 2): 12.6,
 (1380, 1339, 2): 7.9,
 (1380, 1340, 2): 11.3,
 (1380, 1342, 2): 19.4,
 (1380, 1343, 2): 14.7,
 (1380, 1344, 2): 15.6,
 (1380, 1346, 2): 12.2,
 (1380, 1347, 2): 11.1,
 (1380, 1348, 2): 9.5,
 (1380, 1349, 2): 11.6,
 (1380, 1351, 2): 11.3,
 (1380, 1352, 2): 17.8,
 (1380, 1353, 2): 7.7,
 (1380, 1354, 2): 8.3,
 (1380, 1357, 2): 14.6,
 (1380, 1358, 2): 12.8,
 (1380, 1360, 2): 11.1,
 (1380, 1361, 2): 17.2,
 (1380, 1363, 2): 12.6,
 (1380, 1364, 2): 16.7,
 (1380, 1365, 2): 18.4,
 (1380, 1366, 2): 10.3,
 (1380, 1367, 2): 9.7,
 (1380, 1368, 2): 13.9,
 (1380, 1370, 2): 20.0,
 (1380, 1372, 2): 8.6,
 (1380, 1373, 2): 3.5,
 (1380, 1375, 2): 4.4,
 (1380, 1376, 2): 9.2,
 (1380, 1377, 2): 3.1,
 (1380, 1379, 2): 14.6,
 (1380, 1381, 2): 9.8,
 (1380, 1382, 2): 6.1,
 (1380, 1384, 2): 13.3,
 (1380, 1385, 2): 16.5,
 (1380, 1386, 2): 17.3,
 (1380, 1388, 2): 20.2,
 (1380, 1390, 2): 10.1,
 (1380, 1391, 2): 8.7,
 (1380, 1392, 2): 11.1,
 (1380, 1393, 2): 9.9,
 (1380, 1394, 2): 7.2,
 (1380, 1395, 2): 9.5,
 (1380, 1396, 2): 10.0,
 (1380, 4406, 2): 8.3,
 (1380, 4411, 2): 5.2,
 (1380, 4414, 2): 11.0,
 (1380, 4415, 2): 14.9,
 (1380, 4417, 2): 15.0,
 (1380, 4418, 2): 5.2,
 (1380, 4419, 2): 9.4,
 (1380, 4420, 2): 8.7,
 (1380, 4421, 2): 8.7,
 (1380, 4423, 2): 8.1,
 (1380, 4425, 2): 9.3,
 (1381, 0, 2): 12.8,
 (1381, 1302, 2): 24.9,
 (1381, 1303, 2): 24.0,
 (1381, 1304, 2): 30.6,
 (1381, 1305, 2): 21.6,
 (1381, 1306, 2): 25.5,
 (1381, 1308, 2): 15.8,
 (1381, 1314, 2): 20.4,
 (1381, 1315, 2): 21.2,
 (1381, 1316, 2): 18.7,
 (1381, 1318, 2): 16.7,
 (1381, 1319, 2): 14.1,
 (1381, 1321, 2): 14.4,
 (1381, 1322, 2): 16.7,
 (1381, 1325, 2): 14.6,
 (1381, 1326, 2): 10.6,
 (1381, 1327, 2): 11.7,
 (1381, 1328, 2): 12.3,
 (1381, 1332, 2): 6.8,
 (1381, 1333, 2): 8.2,
 (1381, 1334, 2): 9.5,
 (1381, 1336, 2): 22.7,
 (1381, 1337, 2): 21.2,
 (1381, 1338, 2): 18.6,
 (1381, 1339, 2): 17.8,
 (1381, 1340, 2): 11.2,
 (1381, 1342, 2): 29.2,
 (1381, 1343, 2): 24.6,
 (1381, 1344, 2): 25.5,
 (1381, 1346, 2): 22.2,
 (1381, 1347, 2): 20.9,
 (1381, 1348, 2): 19.3,
 (1381, 1349, 2): 10.7,
 (1381, 1351, 2): 16.8,
 (1381, 1352, 2): 21.8,
 (1381, 1353, 2): 17.5,
 (1381, 1354, 2): 16.8,
 (1381, 1357, 2): 24.5,
 (1381, 1358, 2): 21.1,
 (1381, 1360, 2): 25.2,
 (1381, 1361, 2): 27.0,
 (1381, 1363, 2): 18.7,
 (1381, 1364, 2): 26.5,
 (1381, 1365, 2): 28.2,
 (1381, 1366, 2): 18.3,
 (1381, 1367, 2): 19.4,
 (1381, 1368, 2): 21.4,
 (1381, 1370, 2): 29.9,
 (1381, 1372, 2): 14.0,
 (1381, 1373, 2): 13.6,
 (1381, 1375, 2): 14.5,
 (1381, 1376, 2): 15.9,
 (1381, 1377, 2): 11.7,
 (1381, 1379, 2): 24.5,
 (1381, 1380, 2): 16.3,
 (1381, 1382, 2): 16.2,
 (1381, 1384, 2): 23.3,
 (1381, 1385, 2): 23.7,
 (1381, 1386, 2): 23.8,
 (1381, 1388, 2): 25.7,
 (1381, 1390, 2): 11.6,
 (1381, 1391, 2): 9.0,
 (1381, 1392, 2): 21.0,
 (1381, 1393, 2): 19.9,
 (1381, 1394, 2): 17.1,
 (1381, 1395, 2): 12.4,
 (1381, 1396, 2): 11.1,
 (1381, 4406, 2): 12.9,
 (1381, 4411, 2): 15.3,
 (1381, 4414, 2): 19.5,
 (1381, 4415, 2): 25.1,
 (1381, 4417, 2): 24.9,
 (1381, 4418, 2): 15.4,
 (1381, 4419, 2): 13.9,
 (1381, 4420, 2): 13.1,
 (1381, 4421, 2): 18.8,
 (1381, 4423, 2): 18.2,
 (1381, 4425, 2): 14.0,
 (1382, 0, 2): 15.4,
 (1382, 1302, 2): 12.0,
 (1382, 1303, 2): 10.9,
 (1382, 1304, 2): 17.8,
 (1382, 1305, 2): 14.4,
 (1382, 1306, 2): 17.1,
 (1382, 1308, 2): 11.0,
 (1382, 1314, 2): 10.0,
 (1382, 1315, 2): 10.7,
 (1382, 1316, 2): 10.4,
 (1382, 1318, 2): 6.2,
 (1382, 1319, 2): 3.4,
 (1382, 1321, 2): 7.4,
 (1382, 1322, 2): 5.2,
 (1382, 1325, 2): 8.8,
 (1382, 1326, 2): 6.8,
 (1382, 1327, 2): 6.8,
 (1382, 1328, 2): 5.0,
 (1382, 1332, 2): 13.7,
 (1382, 1333, 2): 10.9,
 (1382, 1334, 2): 11.4,
 (1382, 1336, 2): 14.3,
 (1382, 1337, 2): 12.4,
 (1382, 1338, 2): 13.7,
 (1382, 1339, 2): 5.8,
 (1382, 1340, 2): 8.4,
 (1382, 1342, 2): 16.4,
 (1382, 1343, 2): 11.7,
 (1382, 1344, 2): 12.4,
 (1382, 1346, 2): 9.4,
 (1382, 1347, 2): 8.1,
 (1382, 1348, 2): 6.4,
 (1382, 1349, 2): 9.5,
 (1382, 1351, 2): 12.0,
 (1382, 1352, 2): 18.2,
 (1382, 1353, 2): 7.6,
 (1382, 1354, 2): 7.8,
 (1382, 1357, 2): 13.8,
 (1382, 1358, 2): 12.2,
 (1382, 1360, 2): 12.6,
 (1382, 1361, 2): 14.2,
 (1382, 1363, 2): 13.8,
 (1382, 1364, 2): 13.7,
 (1382, 1365, 2): 15.5,
 (1382, 1366, 2): 8.1,
 (1382, 1367, 2): 6.6,
 (1382, 1368, 2): 11.1,
 (1382, 1370, 2): 17.1,
 (1382, 1372, 2): 9.3,
 (1382, 1373, 2): 5.6,
 (1382, 1375, 2): 1.7,
 (1382, 1376, 2): 13.0,
 (1382, 1377, 2): 7.3,
 (1382, 1379, 2): 11.1,
 (1382, 1380, 2): 5.1,
 (1382, 1381, 2): 14.5,
 (1382, 1384, 2): 10.3,
 (1382, 1385, 2): 13.8,
 (1382, 1386, 2): 14.6,
 (1382, 1388, 2): 17.4,
 (1382, 1390, 2): 14.2,
 (1382, 1391, 2): 9.5,
 (1382, 1392, 2): 10.0,
 (1382, 1393, 2): 7.0,
 (1382, 1394, 2): 6.5,
 (1382, 1395, 2): 9.1,
 (1382, 1396, 2): 8.6,
 (1382, 4406, 2): 9.0,
 (1382, 4411, 2): 3.2,
 (1382, 4414, 2): 14.6,
 (1382, 4415, 2): 11.6,
 (1382, 4417, 2): 13.7,
 (1382, 4418, 2): 4.6,
 (1382, 4419, 2): 13.6,
 (1382, 4420, 2): 12.9,
 (1382, 4421, 2): 5.4,
 (1382, 4423, 2): 4.8,
 (1382, 4425, 2): 7.3,
 (1384, 0, 2): 20.6,
 (1384, 1302, 2): 2.8,
 (1384, 1303, 2): 1.6,
 (1384, 1304, 2): 8.5,
 (1384, 1305, 2): 5.0,
 (1384, 1306, 2): 7.7,
 (1384, 1308, 2): 12.9,
 (1384, 1314, 2): 6.8,
 (1384, 1315, 2): 8.6,
 (1384, 1316, 2): 12.1,
 (1384, 1318, 2): 9.9,
 (1384, 1319, 2): 11.5,
 (1384, 1321, 2): 14.3,
 (1384, 1322, 2): 12.5,
 (1384, 1325, 2): 17.5,
 (1384, 1326, 2): 15.4,
 (1384, 1327, 2): 15.5,
 (1384, 1328, 2): 13.7,
 (1384, 1332, 2): 22.4,
 (1384, 1333, 2): 19.5,
 (1384, 1334, 2): 20.0,
 (1384, 1336, 2): 4.9,
 (1384, 1337, 2): 3.1,
 (1384, 1338, 2): 5.6,
 (1384, 1339, 2): 12.2,
 (1384, 1340, 2): 19.8,
 (1384, 1342, 2): 7.2,
 (1384, 1343, 2): 2.4,
 (1384, 1344, 2): 4.3,
 (1384, 1346, 2): 13.2,
 (1384, 1347, 2): 11.9,
 (1384, 1348, 2): 10.2,
 (1384, 1349, 2): 20.4,
 (1384, 1351, 2): 8.8,
 (1384, 1352, 2): 14.5,
 (1384, 1353, 2): 7.4,
 (1384, 1354, 2): 16.6,
 (1384, 1357, 2): 11.8,
 (1384, 1358, 2): 10.2,
 (1384, 1360, 2): 6.1,
 (1384, 1361, 2): 5.0,
 (1384, 1363, 2): 7.6,
 (1384, 1364, 2): 4.4,
 (1384, 1365, 2): 6.2,
 (1384, 1366, 2): 18.2,
 (1384, 1367, 2): 10.4,
 (1384, 1368, 2): 7.2,
 (1384, 1370, 2): 7.8,
 (1384, 1372, 2): 13.0,
 (1384, 1373, 2): 14.4,
 (1384, 1375, 2): 16.3,
 (1384, 1376, 2): 18.7,
 (1384, 1377, 2): 16.0,
 (1384, 1379, 2): 13.8,
 (1384, 1380, 2): 13.4,
 (1384, 1381, 2): 23.2,
 (1384, 1382, 2): 18.0,
 (1384, 1385, 2): 4.4,
 (1384, 1386, 2): 5.2,
 (1384, 1388, 2): 7.9,
 (1384, 1390, 2): 24.8,
 (1384, 1391, 2): 20.9,
 (1384, 1392, 2): 6.7,
 (1384, 1393, 2): 10.8,
 (1384, 1394, 2): 10.3,
 (1384, 1395, 2): 19.8,
 (1384, 1396, 2): 19.5,
 (1384, 4406, 2): 20.6,
 (1384, 4411, 2): 13.9,
 (1384, 4414, 2): 10.4,
 (1384, 4415, 2): 19.1,
 (1384, 4417, 2): 12.4,
 (1384, 4418, 2): 10.4,
 (1384, 4419, 2): 22.3,
 (1384, 4420, 2): 21.6,
 (1384, 4421, 2): 11.0,
 (1384, 4423, 2): 11.6,
 (1384, 4425, 2): 18.1,
 (1385, 0, 2): 20.1,
 (1385, 1302, 2): 4.4,
 (1385, 1303, 2): 5.4,
 (1385, 1304, 2): 11.9,
 (1385, 1305, 2): 3.2,
 (1385, 1306, 2): 6.7,
 (1385, 1308, 2): 11.4,
 (1385, 1314, 2): 6.2,
 (1385, 1315, 2): 8.0,
 (1385, 1316, 2): 11.5,
 (1385, 1318, 2): 5.4,
 (1385, 1319, 2): 7.1,
 (1385, 1321, 2): 10.2,
 (1385, 1322, 2): 8.0,
 (1385, 1325, 2): 12.8,
 (1385, 1326, 2): 10.8,
 (1385, 1327, 2): 10.8,
 (1385, 1328, 2): 9.0,
 (1385, 1332, 2): 17.8,
 (1385, 1333, 2): 14.9,
 (1385, 1334, 2): 15.4,
 (1385, 1336, 2): 4.0,
 (1385, 1337, 2): 2.2,
 (1385, 1338, 2): 4.7,
 (1385, 1339, 2): 7.7,
 (1385, 1340, 2): 15.3,
 (1385, 1342, 2): 10.6,
 (1385, 1343, 2): 4.1,
 (1385, 1344, 2): 3.7,
 (1385, 1346, 2): 8.7,
 (1385, 1347, 2): 7.4,
 (1385, 1348, 2): 5.7,
 (1385, 1349, 2): 16.0,
 (1385, 1351, 2): 7.1,
 (1385, 1352, 2): 9.4,
 (1385, 1353, 2): 3.0,
 (1385, 1354, 2): 9.1,
 (1385, 1357, 2): 11.2,
 (1385, 1358, 2): 9.6,
 (1385, 1360, 2): 5.5,
 (1385, 1361, 2): 8.4,
 (1385, 1363, 2): 6.0,
 (1385, 1364, 2): 7.8,
 (1385, 1365, 2): 9.6,
 (1385, 1366, 2): 11.1,
 (1385, 1367, 2): 5.9,
 (1385, 1368, 2): 4.4,
 (1385, 1370, 2): 11.2,
 (1385, 1372, 2): 10.6,
 (1385, 1373, 2): 9.8,
 (1385, 1375, 2): 11.7,
 (1385, 1376, 2): 14.4,
 (1385, 1377, 2): 11.4,
 (1385, 1379, 2): 10.4,
 (1385, 1380, 2): 9.0,
 (1385, 1381, 2): 18.6,
 (1385, 1382, 2): 13.4,
 (1385, 1384, 2): 2.6,
 (1385, 1386, 2): 0.8,
 (1385, 1388, 2): 6.9,
 (1385, 1390, 2): 18.3,
 (1385, 1391, 2): 16.3,
 (1385, 1392, 2): 2.2,
 (1385, 1393, 2): 6.4,
 (1385, 1394, 2): 5.8,
 (1385, 1395, 2): 15.3,
 (1385, 1396, 2): 15.0,
 (1385, 4406, 2): 16.0,
 (1385, 4411, 2): 9.4,
 (1385, 4414, 2): 8.8,
 (1385, 4415, 2): 14.6,
 (1385, 4417, 2): 11.7,
 (1385, 4418, 2): 6.0,
 (1385, 4419, 2): 17.7,
 (1385, 4420, 2): 16.9,
 (1385, 4421, 2): 6.5,
 (1385, 4423, 2): 7.1,
 (1385, 4425, 2): 13.6,
 (1386, 0, 2): 17.4,
 (1386, 1302, 2): 10.6,
 (1386, 1303, 2): 8.2,
 (1386, 1304, 2): 14.8,
 (1386, 1305, 2): 5.2,
 (1386, 1306, 2): 9.6,
 (1386, 1308, 2): 11.6,
 (1386, 1314, 2): 8.4,
 (1386, 1315, 2): 9.2,
 (1386, 1316, 2): 12.6,
 (1386, 1318, 2): 4.6,
 (1386, 1319, 2): 6.3,
 (1386, 1321, 2): 9.4,
 (1386, 1322, 2): 7.2,
 (1386, 1325, 2): 12.1,
 (1386, 1326, 2): 10.0,
 (1386, 1327, 2): 10.1,
 (1386, 1328, 2): 8.3,
 (1386, 1332, 2): 17.0,
 (1386, 1333, 2): 14.1,
 (1386, 1334, 2): 14.6,
 (1386, 1336, 2): 6.8,
 (1386, 1337, 2): 4.7,
 (1386, 1338, 2): 4.8,
 (1386, 1339, 2): 6.6,
 (1386, 1340, 2): 13.6,
 (1386, 1342, 2): 13.1,
 (1386, 1343, 2): 9.7,
 (1386, 1344, 2): 10.8,
 (1386, 1346, 2): 7.3,
 (1386, 1347, 2): 6.1,
 (1386, 1348, 2): 4.6,
 (1386, 1349, 2): 14.7,
 (1386, 1351, 2): 7.2,
 (1386, 1352, 2): 9.7,
 (1386, 1353, 2): 2.2,
 (1386, 1354, 2): 7.9,
 (1386, 1357, 2): 11.8,
 (1386, 1358, 2): 10.1,
 (1386, 1360, 2): 6.3,
 (1386, 1361, 2): 11.0,
 (1386, 1363, 2): 6.1,
 (1386, 1364, 2): 10.4,
 (1386, 1365, 2): 12.2,
 (1386, 1366, 2): 10.1,
 (1386, 1367, 2): 4.8,
 (1386, 1368, 2): 2.4,
 (1386, 1370, 2): 13.8,
 (1386, 1372, 2): 9.4,
 (1386, 1373, 2): 8.9,
 (1386, 1375, 2): 10.7,
 (1386, 1376, 2): 13.0,
 (1386, 1377, 2): 10.5,
 (1386, 1379, 2): 9.0,
 (1386, 1380, 2): 8.0,
 (1386, 1381, 2): 17.9,
 (1386, 1382, 2): 12.4,
 (1386, 1384, 2): 8.1,
 (1386, 1385, 2): 7.5,
 (1386, 1388, 2): 10.7,
 (1386, 1390, 2): 17.5,
 (1386, 1391, 2): 14.6,
 (1386, 1392, 2): 1.4,
 (1386, 1393, 2): 5.2,
 (1386, 1394, 2): 4.5,
 (1386, 1395, 2): 14.2,
 (1386, 1396, 2): 13.7,
 (1386, 4406, 2): 15.4,
 (1386, 4411, 2): 8.3,
 (1386, 4414, 2): 8.9,
 (1386, 4415, 2): 13.9,
 (1386, 4417, 2): 12.0,
 (1386, 4418, 2): 4.9,
 (1386, 4419, 2): 16.9,
 (1386, 4420, 2): 16.2,
 (1386, 4421, 2): 5.3,
 (1386, 4423, 2): 5.9,
 (1386, 4425, 2): 12.4,
 (1388, 0, 2): 18.9,
 (1388, 1302, 2): 5.2,
 (1388, 1303, 2): 3.8,
 (1388, 1304, 2): 11.2,
 (1388, 1305, 2): 3.4,
 (1388, 1306, 2): 6.1,
 (1388, 1308, 2): 10.8,
 (1388, 1314, 2): 7.9,
 (1388, 1315, 2): 10.0,
 (1388, 1316, 2): 13.2,
 (1388, 1318, 2): 6.5,
 (1388, 1319, 2): 8.4,
 (1388, 1321, 2): 12.5,
 (1388, 1322, 2): 9.4,
 (1388, 1325, 2): 14.2,
 (1388, 1326, 2): 12.4,
 (1388, 1327, 2): 12.5,
 (1388, 1328, 2): 10.6,
 (1388, 1332, 2): 19.7,
 (1388, 1333, 2): 16.7,
 (1388, 1334, 2): 17.2,
 (1388, 1336, 2): 5.1,
 (1388, 1337, 2): 3.4,
 (1388, 1338, 2): 5.8,
 (1388, 1339, 2): 9.0,
 (1388, 1340, 2): 16.1,
 (1388, 1342, 2): 9.9,
 (1388, 1343, 2): 4.9,
 (1388, 1344, 2): 5.6,
 (1388, 1346, 2): 9.6,
 (1388, 1347, 2): 8.4,
 (1388, 1348, 2): 6.9,
 (1388, 1349, 2): 17.0,
 (1388, 1351, 2): 7.2,
 (1388, 1352, 2): 13.0,
 (1388, 1353, 2): 4.5,
 (1388, 1354, 2): 10.3,
 (1388, 1357, 2): 13.3,
 (1388, 1358, 2): 11.6,
 (1388, 1360, 2): 7.4,
 (1388, 1361, 2): 7.7,
 (1388, 1363, 2): 6.1,
 (1388, 1364, 2): 7.2,
 (1388, 1365, 2): 9.0,
 (1388, 1366, 2): 16.4,
 (1388, 1367, 2): 7.1,
 (1388, 1368, 2): 5.8,
 (1388, 1370, 2): 10.6,
 (1388, 1372, 2): 11.3,
 (1388, 1373, 2): 11.2,
 (1388, 1375, 2): 12.6,
 (1388, 1376, 2): 15.3,
 (1388, 1377, 2): 12.9,
 (1388, 1379, 2): 11.3,
 (1388, 1380, 2): 10.4,
 (1388, 1381, 2): 20.4,
 (1388, 1382, 2): 14.7,
 (1388, 1384, 2): 4.1,
 (1388, 1385, 2): 1.5,
 (1388, 1386, 2): 2.2,
 (1388, 1390, 2): 20.1,
 (1388, 1391, 2): 17.0,
 (1388, 1392, 2): 3.7,
 (1388, 1393, 2): 7.5,
 (1388, 1394, 2): 6.8,
 (1388, 1395, 2): 16.5,
 (1388, 1396, 2): 16.0,
 (1388, 4406, 2): 17.3,
 (1388, 4411, 2): 10.7,
 (1388, 4414, 2): 8.9,
 (1388, 4415, 2): 16.1,
 (1388, 4417, 2): 13.6,
 (1388, 4418, 2): 7.2,
 (1388, 4419, 2): 19.5,
 (1388, 4420, 2): 18.7,
 (1388, 4421, 2): 7.6,
 (1388, 4423, 2): 8.2,
 (1388, 4425, 2): 14.7,
 (1390, 0, 2): 17.6,
 (1390, 1302, 2): 19.0,
 (1390, 1303, 2): 17.8,
 (1390, 1304, 2): 24.7,
 (1390, 1305, 2): 21.5,
 (1390, 1306, 2): 24.2,
 (1390, 1308, 2): 16.9,
 (1390, 1314, 2): 14.6,
 (1390, 1315, 2): 15.4,
 (1390, 1316, 2): 15.2,
 (1390, 1318, 2): 11.0,
 (1390, 1319, 2): 8.1,
 (1390, 1321, 2): 12.9,
 (1390, 1322, 2): 10.7,
 (1390, 1325, 2): 8.2,
 (1390, 1326, 2): 4.6,
 (1390, 1327, 2): 5.6,
 (1390, 1328, 2): 6.3,
 (1390, 1332, 2): 4.8,
 (1390, 1333, 2): 4.3,
 (1390, 1334, 2): 4.8,
 (1390, 1336, 2): 23.6,
 (1390, 1337, 2): 20.1,
 (1390, 1338, 2): 19.5,
 (1390, 1339, 2): 11.9,
 (1390, 1340, 2): 9.5,
 (1390, 1342, 2): 23.4,
 (1390, 1343, 2): 18.7,
 (1390, 1344, 2): 19.8,
 (1390, 1346, 2): 16.2,
 (1390, 1347, 2): 15.1,
 (1390, 1348, 2): 13.6,
 (1390, 1349, 2): 10.2,
 (1390, 1351, 2): 17.8,
 (1390, 1352, 2): 22.5,
 (1390, 1353, 2): 11.8,
 (1390, 1354, 2): 12.4,
 (1390, 1357, 2): 18.8,
 (1390, 1358, 2): 17.0,
 (1390, 1360, 2): 15.3,
 (1390, 1361, 2): 21.3,
 (1390, 1363, 2): 19.7,
 (1390, 1364, 2): 20.8,
 (1390, 1365, 2): 22.6,
 (1390, 1366, 2): 13.8,
 (1390, 1367, 2): 13.8,
 (1390, 1368, 2): 17.8,
 (1390, 1370, 2): 24.0,
 (1390, 1372, 2): 15.0,
 (1390, 1373, 2): 7.5,
 (1390, 1375, 2): 8.4,
 (1390, 1376, 2): 14.8,
 (1390, 1377, 2): 5.7,
 (1390, 1379, 2): 18.2,
 (1390, 1380, 2): 8.3,
 (1390, 1381, 2): 7.9,
 (1390, 1382, 2): 10.1,
 (1390, 1384, 2): 18.2,
 (1390, 1385, 2): 20.8,
 (1390, 1386, 2): 21.6,
 (1390, 1388, 2): 24.4,
 (1390, 1391, 2): 6.4,
 (1390, 1392, 2): 15.3,
 (1390, 1393, 2): 14.2,
 (1390, 1394, 2): 11.4,
 (1390, 1395, 2): 7.9,
 (1390, 1396, 2): 8.2,
 (1390, 4406, 2): 6.6,
 (1390, 4411, 2): 9.2,
 (1390, 4414, 2): 15.6,
 (1390, 4415, 2): 19.3,
 (1390, 4417, 2): 19.0,
 (1390, 4418, 2): 9.3,
 (1390, 4419, 2): 3.1,
 (1390, 4420, 2): 2.4,
 (1390, 4421, 2): 12.8,
 (1390, 4423, 2): 12.0,
 (1390, 4425, 2): 7.7,
 (1391, 0, 2): 18.6,
 (1391, 1302, 2): 18.8,
 (1391, 1303, 2): 17.6,
 (1391, 1304, 2): 24.4,
 (1391, 1305, 2): 21.3,
 (1391, 1306, 2): 23.9,
 (1391, 1308, 2): 17.6,
 (1391, 1314, 2): 16.6,
 (1391, 1315, 2): 17.4,
 (1391, 1316, 2): 17.0,
 (1391, 1318, 2): 12.9,
 (1391, 1319, 2): 10.0,
 (1391, 1321, 2): 14.8,
 (1391, 1322, 2): 12.8,
 (1391, 1325, 2): 10.0,
 (1391, 1326, 2): 6.4,
 (1391, 1327, 2): 7.5,
 (1391, 1328, 2): 8.2,
 (1391, 1332, 2): 6.3,
 (1391, 1333, 2): 5.5,
 (1391, 1334, 2): 3.9,
 (1391, 1336, 2): 21.0,
 (1391, 1337, 2): 19.1,
 (1391, 1338, 2): 20.3,
 (1391, 1339, 2): 13.8,
 (1391, 1340, 2): 4.8,
 (1391, 1342, 2): 23.2,
 (1391, 1343, 2): 18.4,
 (1391, 1344, 2): 19.5,
 (1391, 1346, 2): 15.9,
 (1391, 1347, 2): 14.8,
 (1391, 1348, 2): 13.3,
 (1391, 1349, 2): 4.7,
 (1391, 1351, 2): 18.6,
 (1391, 1352, 2): 26.0,
 (1391, 1353, 2): 13.6,
 (1391, 1354, 2): 14.2,
 (1391, 1357, 2): 20.6,
 (1391, 1358, 2): 19.0,
 (1391, 1360, 2): 18.6,
 (1391, 1361, 2): 21.0,
 (1391, 1363, 2): 19.8,
 (1391, 1364, 2): 20.4,
 (1391, 1365, 2): 22.2,
 (1391, 1366, 2): 15.9,
 (1391, 1367, 2): 13.5,
 (1391, 1368, 2): 17.6,
 (1391, 1370, 2): 23.8,
 (1391, 1372, 2): 15.7,
 (1391, 1373, 2): 9.4,
 (1391, 1375, 2): 10.3,
 (1391, 1376, 2): 16.7,
 (1391, 1377, 2): 7.6,
 (1391, 1379, 2): 17.6,
 (1391, 1380, 2): 10.2,
 (1391, 1381, 2): 7.0,
 (1391, 1382, 2): 12.0,
 (1391, 1384, 2): 17.2,
 (1391, 1385, 2): 20.6,
 (1391, 1386, 2): 21.4,
 (1391, 1388, 2): 24.2,
 (1391, 1390, 2): 9.0,
 (1391, 1392, 2): 17.2,
 (1391, 1393, 2): 13.9,
 (1391, 1394, 2): 13.4,
 (1391, 1395, 2): 2.2,
 (1391, 1396, 2): 4.8,
 (1391, 4406, 2): 4.8,
 (1391, 4411, 2): 11.1,
 (1391, 4414, 2): 17.4,
 (1391, 4415, 2): 19.0,
 (1391, 4417, 2): 21.0,
 (1391, 4418, 2): 11.1,
 (1391, 4419, 2): 9.5,
 (1391, 4420, 2): 8.7,
 (1391, 4421, 2): 12.4,
 (1391, 4423, 2): 11.7,
 (1391, 4425, 2): 5.8,
 (1392, 0, 2): 17.5,
 (1392, 1302, 2): 9.5,
 (1392, 1303, 2): 8.3,
 (1392, 1304, 2): 15.3,
 (1392, 1305, 2): 9.8,
 (1392, 1306, 2): 13.2,
 (1392, 1308, 2): 9.3,
 (1392, 1314, 2): 9.7,
 (1392, 1315, 2): 7.8,
 (1392, 1316, 2): 11.0,
 (1392, 1318, 2): 4.8,
 (1392, 1319, 2): 6.4,
 (1392, 1321, 2): 9.6,
 (1392, 1322, 2): 7.4,
 (1392, 1325, 2): 12.1,
 (1392, 1326, 2): 10.3,
 (1392, 1327, 2): 10.4,
 (1392, 1328, 2): 8.5,
 (1392, 1332, 2): 17.5,
 (1392, 1333, 2): 14.5,
 (1392, 1334, 2): 15.0,
 (1392, 1336, 2): 10.6,
 (1392, 1337, 2): 8.8,
 (1392, 1338, 2): 11.2,
 (1392, 1339, 2): 7.4,
 (1392, 1340, 2): 14.7,
 (1392, 1342, 2): 13.9,
 (1392, 1343, 2): 9.2,
 (1392, 1344, 2): 8.6,
 (1392, 1346, 2): 8.4,
 (1392, 1347, 2): 7.2,
 (1392, 1348, 2): 5.7,
 (1392, 1349, 2): 15.5,
 (1392, 1351, 2): 7.9,
 (1392, 1352, 2): 11.4,
 (1392, 1353, 2): 7.5,
 (1392, 1354, 2): 8.4,
 (1392, 1357, 2): 11.0,
 (1392, 1358, 2): 9.3,
 (1392, 1360, 2): 5.1,
 (1392, 1361, 2): 11.6,
 (1392, 1363, 2): 6.7,
 (1392, 1364, 2): 11.2,
 (1392, 1365, 2): 12.9,
 (1392, 1366, 2): 10.4,
 (1392, 1367, 2): 6.0,
 (1392, 1368, 2): 11.1,
 (1392, 1370, 2): 14.5,
 (1392, 1372, 2): 9.8,
 (1392, 1373, 2): 9.2,
 (1392, 1375, 2): 10.9,
 (1392, 1376, 2): 13.3,
 (1392, 1377, 2): 10.8,
 (1392, 1379, 2): 12.4,
 (1392, 1380, 2): 8.4,
 (1392, 1381, 2): 18.2,
 (1392, 1382, 2): 12.6,
 (1392, 1384, 2): 6.9,
 (1392, 1385, 2): 11.6,
 (1392, 1386, 2): 12.3,
 (1392, 1388, 2): 13.5,
 (1392, 1390, 2): 17.9,
 (1392, 1391, 2): 16.5,
 (1392, 1393, 2): 4.3,
 (1392, 1394, 2): 7.8,
 (1392, 1395, 2): 14.9,
 (1392, 1396, 2): 14.5,
 (1392, 4406, 2): 15.7,
 (1392, 4411, 2): 8.9,
 (1392, 4414, 2): 6.8,
 (1392, 4415, 2): 18.4,
 (1392, 4417, 2): 11.4,
 (1392, 4418, 2): 5.2,
 (1392, 4419, 2): 17.1,
 (1392, 4420, 2): 16.4,
 (1392, 4421, 2): 4.8,
 (1392, 4423, 2): 6.5,
 (1392, 4425, 2): 12.8,
 (1393, 0, 2): 21.6,
 (1393, 1302, 2): 9.3,
 (1393, 1303, 2): 8.2,
 (1393, 1304, 2): 14.7,
 (1393, 1305, 2): 9.2,
 (1393, 1306, 2): 12.8,
 (1393, 1308, 2): 15.6,
 (1393, 1314, 2): 9.2,
 (1393, 1315, 2): 7.3,
 (1393, 1316, 2): 10.7,
 (1393, 1318, 2): 9.0,
 (1393, 1319, 2): 10.4,
 (1393, 1321, 2): 13.2,
 (1393, 1322, 2): 11.3,
 (1393, 1325, 2): 15.2,
 (1393, 1326, 2): 14.3,
 (1393, 1327, 2): 14.4,
 (1393, 1328, 2): 12.7,
 (1393, 1332, 2): 21.5,
 (1393, 1333, 2): 18.5,
 (1393, 1334, 2): 19.0,
 (1393, 1336, 2): 10.2,
 (1393, 1337, 2): 8.3,
 (1393, 1338, 2): 10.6,
 (1393, 1339, 2): 11.2,
 (1393, 1340, 2): 18.8,
 (1393, 1342, 2): 13.4,
 (1393, 1343, 2): 9.0,
 (1393, 1344, 2): 8.0,
 (1393, 1346, 2): 12.0,
 (1393, 1347, 2): 10.9,
 (1393, 1348, 2): 9.4,
 (1393, 1349, 2): 19.6,
 (1393, 1351, 2): 13.0,
 (1393, 1352, 2): 20.1,
 (1393, 1353, 2): 6.9,
 (1393, 1354, 2): 12.2,
 (1393, 1357, 2): 10.6,
 (1393, 1358, 2): 8.8,
 (1393, 1360, 2): 4.5,
 (1393, 1361, 2): 11.3,
 (1393, 1363, 2): 11.8,
 (1393, 1364, 2): 10.8,
 (1393, 1365, 2): 12.5,
 (1393, 1366, 2): 14.3,
 (1393, 1367, 2): 9.6,
 (1393, 1368, 2): 10.8,
 (1393, 1370, 2): 14.1,
 (1393, 1372, 2): 13.5,
 (1393, 1373, 2): 13.3,
 (1393, 1375, 2): 15.1,
 (1393, 1376, 2): 17.3,
 (1393, 1377, 2): 15.0,
 (1393, 1379, 2): 11.9,
 (1393, 1380, 2): 12.4,
 (1393, 1381, 2): 22.2,
 (1393, 1382, 2): 16.7,
 (1393, 1384, 2): 6.3,
 (1393, 1385, 2): 11.2,
 (1393, 1386, 2): 11.9,
 (1393, 1388, 2): 13.0,
 (1393, 1390, 2): 21.7,
 (1393, 1391, 2): 20.0,
 (1393, 1392, 2): 4.9,
 (1393, 1394, 2): 7.2,
 (1393, 1395, 2): 19.6,
 (1393, 1396, 2): 19.7,
 (1393, 4406, 2): 19.8,
 (1393, 4411, 2): 12.8,
 (1393, 4414, 2): 14.6,
 (1393, 4415, 2): 17.8,
 (1393, 4417, 2): 10.8,
 (1393, 4418, 2): 9.2,
 (1393, 4419, 2): 21.1,
 (1393, 4420, 2): 20.3,
 (1393, 4421, 2): 8.9,
 (1393, 4423, 2): 10.7,
 (1393, 4425, 2): 17.2,
 (1394, 0, 2): 19.6,
 (1394, 1302, 2): 9.6,
 (1394, 1303, 2): 8.4,
 (1394, 1304, 2): 13.6,
 (1394, 1305, 2): 9.4,
 (1394, 1306, 2): 13.1,
 (1394, 1308, 2): 13.6,
 (1394, 1314, 2): 3.2,
 (1394, 1315, 2): 4.2,
 (1394, 1316, 2): 7.6,
 (1394, 1318, 2): 6.9,
 (1394, 1319, 2): 8.4,
 (1394, 1321, 2): 11.1,
 (1394, 1322, 2): 9.2,
 (1394, 1325, 2): 13.2,
 (1394, 1326, 2): 12.3,
 (1394, 1327, 2): 12.4,
 (1394, 1328, 2): 10.6,
 (1394, 1332, 2): 19.4,
 (1394, 1333, 2): 16.5,
 (1394, 1334, 2): 17.0,
 (1394, 1336, 2): 10.4,
 (1394, 1337, 2): 8.5,
 (1394, 1338, 2): 10.8,
 (1394, 1339, 2): 9.1,
 (1394, 1340, 2): 16.8,
 (1394, 1342, 2): 13.6,
 (1394, 1343, 2): 9.2,
 (1394, 1344, 2): 8.2,
 (1394, 1346, 2): 8.0,
 (1394, 1347, 2): 4.4,
 (1394, 1348, 2): 4.4,
 (1394, 1349, 2): 17.5,
 (1394, 1351, 2): 13.2,
 (1394, 1352, 2): 15.8,
 (1394, 1353, 2): 4.9,
 (1394, 1354, 2): 10.2,
 (1394, 1357, 2): 7.4,
 (1394, 1358, 2): 5.7,
 (1394, 1360, 2): 3.8,
 (1394, 1361, 2): 11.6,
 (1394, 1363, 2): 12.0,
 (1394, 1364, 2): 11.1,
 (1394, 1365, 2): 12.7,
 (1394, 1366, 2): 12.3,
 (1394, 1367, 2): 4.4,
 (1394, 1368, 2): 11.1,
 (1394, 1370, 2): 14.4,
 (1394, 1372, 2): 11.5,
 (1394, 1373, 2): 11.2,
 (1394, 1375, 2): 13.0,
 (1394, 1376, 2): 15.2,
 (1394, 1377, 2): 12.9,
 (1394, 1379, 2): 6.7,
 (1394, 1380, 2): 10.4,
 (1394, 1381, 2): 20.2,
 (1394, 1382, 2): 14.7,
 (1394, 1384, 2): 6.5,
 (1394, 1385, 2): 11.4,
 (1394, 1386, 2): 12.2,
 (1394, 1388, 2): 13.2,
 (1394, 1390, 2): 19.7,
 (1394, 1391, 2): 18.9,
 (1394, 1392, 2): 7.5,
 (1394, 1393, 2): 2.7,
 (1394, 1395, 2): 17.5,
 (1394, 1396, 2): 16.5,
 (1394, 4406, 2): 17.7,
 (1394, 4411, 2): 10.8,
 (1394, 4414, 2): 14.9,
 (1394, 4415, 2): 14.7,
 (1394, 4417, 2): 7.7,
 (1394, 4418, 2): 7.2,
 (1394, 4419, 2): 19.1,
 (1394, 4420, 2): 18.3,
 (1394, 4421, 2): 6.8,
 (1394, 4423, 2): 8.6,
 (1394, 4425, 2): 15.2,
 (1395, 0, 2): 22.5,
 (1395, 1302, 2): 16.8,
 (1395, 1303, 2): 16.4,
 (1395, 1304, 2): 22.3,
 (1395, 1305, 2): 19.4,
 (1395, 1306, 2): 22.2,
 (1395, 1308, 2): 17.3,
 (1395, 1314, 2): 14.2,
 (1395, 1315, 2): 15.2,
 (1395, 1316, 2): 16.7,
 (1395, 1318, 2): 12.5,
 (1395, 1319, 2): 9.6,
 (1395, 1321, 2): 14.0,
 (1395, 1322, 2): 12.1,
 (1395, 1325, 2): 9.4,
 (1395, 1326, 2): 6.1,
 (1395, 1327, 2): 9.3,
 (1395, 1328, 2): 7.9,
 (1395, 1332, 2): 8.7,
 (1395, 1333, 2): 8.0,
 (1395, 1334, 2): 6.3,
 (1395, 1336, 2): 18.8,
 (1395, 1337, 2): 16.9,
 (1395, 1338, 2): 17.8,
 (1395, 1339, 2): 13.4,
 (1395, 1340, 2): 2.4,
 (1395, 1342, 2): 21.0,
 (1395, 1343, 2): 16.4,
 (1395, 1344, 2): 17.2,
 (1395, 1346, 2): 13.5,
 (1395, 1347, 2): 12.4,
 (1395, 1348, 2): 11.0,
 (1395, 1349, 2): 3.2,
 (1395, 1351, 2): 18.2,
 (1395, 1352, 2): 25.4,
 (1395, 1353, 2): 13.3,
 (1395, 1354, 2): 13.9,
 (1395, 1357, 2): 18.5,
 (1395, 1358, 2): 16.6,
 (1395, 1360, 2): 16.2,
 (1395, 1361, 2): 18.8,
 (1395, 1363, 2): 19.4,
 (1395, 1364, 2): 18.4,
 (1395, 1365, 2): 20.1,
 (1395, 1366, 2): 15.2,
 (1395, 1367, 2): 11.2,
 (1395, 1368, 2): 15.3,
 (1395, 1370, 2): 21.7,
 (1395, 1372, 2): 15.1,
 (1395, 1373, 2): 9.1,
 (1395, 1375, 2): 9.9,
 (1395, 1376, 2): 16.5,
 (1395, 1377, 2): 7.2,
 (1395, 1379, 2): 15.2,
 (1395, 1380, 2): 9.8,
 (1395, 1381, 2): 9.4,
 (1395, 1382, 2): 11.6,
 (1395, 1384, 2): 14.9,
 (1395, 1385, 2): 18.9,
 (1395, 1386, 2): 19.7,
 (1395, 1388, 2): 22.5,
 (1395, 1390, 2): 11.3,
 (1395, 1391, 2): 3.4,
 (1395, 1392, 2): 18.6,
 (1395, 1393, 2): 11.6,
 (1395, 1394, 2): 11.0,
 (1395, 1396, 2): 0.4,
 (1395, 4406, 2): 2.4,
 (1395, 4411, 2): 10.9,
 (1395, 4414, 2): 17.0,
 (1395, 4415, 2): 17.1,
 (1395, 4417, 2): 19.1,
 (1395, 4418, 2): 10.8,
 (1395, 4419, 2): 11.3,
 (1395, 4420, 2): 10.5,
 (1395, 4421, 2): 10.1,
 (1395, 4423, 2): 9.4,
 (1395, 4425, 2): 3.6,
 (1396, 0, 2): 22.4,
 (1396, 1302, 2): 16.7,
 (1396, 1303, 2): 15.6,
 (1396, 1304, 2): 22.2,
 (1396, 1305, 2): 19.4,
 (1396, 1306, 2): 22.1,
 (1396, 1308, 2): 17.2,
 (1396, 1314, 2): 14.1,
 (1396, 1315, 2): 15.1,
 (1396, 1316, 2): 16.6,
 (1396, 1318, 2): 12.5,
 (1396, 1319, 2): 9.6,
 (1396, 1321, 2): 13.9,
 (1396, 1322, 2): 12.0,
 (1396, 1325, 2): 9.4,
 (1396, 1326, 2): 6.0,
 (1396, 1327, 2): 9.4,
 (1396, 1328, 2): 7.8,
 (1396, 1332, 2): 9.1,
 (1396, 1333, 2): 8.4,
 (1396, 1334, 2): 6.7,
 (1396, 1336, 2): 18.8,
 (1396, 1337, 2): 16.8,
 (1396, 1338, 2): 17.7,
 (1396, 1339, 2): 13.4,
 (1396, 1340, 2): 2.9,
 (1396, 1342, 2): 21.0,
 (1396, 1343, 2): 16.4,
 (1396, 1344, 2): 17.2,
 (1396, 1346, 2): 13.4,
 (1396, 1347, 2): 12.4,
 (1396, 1348, 2): 11.0,
 (1396, 1349, 2): 3.6,
 (1396, 1351, 2): 18.1,
 (1396, 1352, 2): 25.4,
 (1396, 1353, 2): 13.2,
 (1396, 1354, 2): 13.8,
 (1396, 1357, 2): 18.4,
 (1396, 1358, 2): 16.6,
 (1396, 1360, 2): 16.1,
 (1396, 1361, 2): 18.8,
 (1396, 1363, 2): 19.3,
 (1396, 1364, 2): 18.3,
 (1396, 1365, 2): 20.0,
 (1396, 1366, 2): 15.0,
 (1396, 1367, 2): 11.1,
 (1396, 1368, 2): 15.3,
 (1396, 1370, 2): 21.6,
 (1396, 1372, 2): 15.0,
 (1396, 1373, 2): 9.0,
 (1396, 1375, 2): 9.8,
 (1396, 1376, 2): 16.4,
 (1396, 1377, 2): 7.1,
 (1396, 1379, 2): 15.1,
 (1396, 1380, 2): 9.7,
 (1396, 1381, 2): 9.9,
 (1396, 1382, 2): 11.5,
 (1396, 1384, 2): 14.8,
 (1396, 1385, 2): 18.8,
 (1396, 1386, 2): 19.6,
 (1396, 1388, 2): 22.5,
 (1396, 1390, 2): 11.8,
 (1396, 1391, 2): 3.8,
 (1396, 1392, 2): 18.6,
 (1396, 1393, 2): 11.6,
 (1396, 1394, 2): 11.0,
 (1396, 1395, 2): 1.8,
 (1396, 4406, 2): 2.3,
 (1396, 4411, 2): 10.8,
 (1396, 4414, 2): 16.9,
 (1396, 4415, 2): 16.8,
 (1396, 4417, 2): 18.8,
 (1396, 4418, 2): 10.7,
 (1396, 4419, 2): 11.2,
 (1396, 4420, 2): 10.4,
 (1396, 4421, 2): 10.0,
 (1396, 4423, 2): 9.3,
 (1396, 4425, 2): 3.5,
 (4406, 0, 2): 21.9,
 (4406, 1302, 2): 14.4,
 (4406, 1303, 2): 13.2,
 (4406, 1304, 2): 19.8,
 (4406, 1305, 2): 16.9,
 (4406, 1306, 2): 19.6,
 (4406, 1308, 2): 16.6,
 (4406, 1314, 2): 11.8,
 (4406, 1315, 2): 12.8,
 (4406, 1316, 2): 16.1,
 (4406, 1318, 2): 12.0,
 (4406, 1319, 2): 9.0,
 (4406, 1321, 2): 13.4,
 (4406, 1322, 2): 11.5,
 (4406, 1325, 2): 8.8,
 (4406, 1326, 2): 5.5,
 (4406, 1327, 2): 8.7,
 (4406, 1328, 2): 7.3,
 (4406, 1332, 2): 11.2,
 (4406, 1333, 2): 8.2,
 (4406, 1334, 2): 8.8,
 (4406, 1336, 2): 16.4,
 (4406, 1337, 2): 14.5,
 (4406, 1338, 2): 15.4,
 (4406, 1339, 2): 8.9,
 (4406, 1340, 2): 4.5,
 (4406, 1342, 2): 18.6,
 (4406, 1343, 2): 14.1,
 (4406, 1344, 2): 14.8,
 (4406, 1346, 2): 11.6,
 (4406, 1347, 2): 10.6,
 (4406, 1348, 2): 9.2,
 (4406, 1349, 2): 4.9,
 (4406, 1351, 2): 17.9,
 (4406, 1352, 2): 24.8,
 (4406, 1353, 2): 10.4,
 (4406, 1354, 2): 13.4,
 (4406, 1357, 2): 16.8,
 (4406, 1358, 2): 14.8,
 (4406, 1360, 2): 14.4,
 (4406, 1361, 2): 16.8,
 (4406, 1363, 2): 19.6,
 (4406, 1364, 2): 16.3,
 (4406, 1365, 2): 18.1,
 (4406, 1366, 2): 14.5,
 (4406, 1367, 2): 9.3,
 (4406, 1368, 2): 13.3,
 (4406, 1370, 2): 19.7,
 (4406, 1372, 2): 14.7,
 (4406, 1373, 2): 8.4,
 (4406, 1375, 2): 7.6,
 (4406, 1376, 2): 15.8,
 (4406, 1377, 2): 6.5,
 (4406, 1379, 2): 13.3,
 (4406, 1380, 2): 9.1,
 (4406, 1381, 2): 10.7,
 (4406, 1382, 2): 9.5,
 (4406, 1384, 2): 12.9,
 (4406, 1385, 2): 16.9,
 (4406, 1386, 2): 17.7,
 (4406, 1388, 2): 20.8,
 (4406, 1390, 2): 10.4,
 (4406, 1391, 2): 5.9,
 (4406, 1392, 2): 16.2,
 (4406, 1393, 2): 9.8,
 (4406, 1394, 2): 9.2,
 (4406, 1395, 2): 2.8,
 (4406, 1396, 2): 3.0,
 (4406, 4411, 2): 9.2,
 (4406, 4414, 2): 16.6,
 (4406, 4415, 2): 15.4,
 (4406, 4417, 2): 17.5,
 (4406, 4418, 2): 10.1,
 (4406, 4419, 2): 10.1,
 (4406, 4420, 2): 9.3,
 (4406, 4421, 2): 8.3,
 (4406, 4423, 2): 7.6,
 (4406, 4425, 2): 1.0,
 (4411, 0, 2): 15.1,
 (4411, 1302, 2): 13.0,
 (4411, 1303, 2): 12.4,
 (4411, 1304, 2): 18.6,
 (4411, 1305, 2): 16.0,
 (4411, 1306, 2): 18.8,
 (4411, 1308, 2): 11.2,
 (4411, 1314, 2): 9.2,
 (4411, 1315, 2): 10.1,
 (4411, 1316, 2): 9.9,
 (4411, 1318, 2): 5.7,
 (4411, 1319, 2): 2.8,
 (4411, 1321, 2): 6.2,
 (4411, 1322, 2): 4.4,
 (4411, 1325, 2): 6.8,
 (4411, 1326, 2): 6.6,
 (4411, 1327, 2): 6.7,
 (4411, 1328, 2): 4.9,
 (4411, 1332, 2): 13.2,
 (4411, 1333, 2): 10.5,
 (4411, 1334, 2): 10.8,
 (4411, 1336, 2): 16.6,
 (4411, 1337, 2): 14.7,
 (4411, 1338, 2): 13.8,
 (4411, 1339, 2): 2.7,
 (4411, 1340, 2): 10.0,
 (4411, 1342, 2): 18.2,
 (4411, 1343, 2): 12.7,
 (4411, 1344, 2): 13.6,
 (4411, 1346, 2): 10.0,
 (4411, 1347, 2): 8.9,
 (4411, 1348, 2): 7.5,
 (4411, 1349, 2): 10.9,
 (4411, 1351, 2): 12.1,
 (4411, 1352, 2): 18.0,
 (4411, 1353, 2): 6.5,
 (4411, 1354, 2): 7.4,
 (4411, 1357, 2): 13.6,
 (4411, 1358, 2): 11.7,
 (4411, 1360, 2): 9.8,
 (4411, 1361, 2): 15.2,
 (4411, 1363, 2): 13.7,
 (4411, 1364, 2): 14.8,
 (4411, 1365, 2): 16.5,
 (4411, 1366, 2): 7.4,
 (4411, 1367, 2): 7.7,
 (4411, 1368, 2): 11.7,
 (4411, 1370, 2): 18.1,
 (4411, 1372, 2): 8.8,
 (4411, 1373, 2): 5.5,
 (4411, 1375, 2): 2.4,
 (4411, 1376, 2): 12.5,
 (4411, 1377, 2): 7.8,
 (4411, 1379, 2): 11.6,
 (4411, 1380, 2): 4.9,
 (4411, 1381, 2): 13.7,
 (4411, 1382, 2): 4.0,
 (4411, 1384, 2): 12.5,
 (4411, 1385, 2): 15.2,
 (4411, 1386, 2): 15.9,
 (4411, 1388, 2): 19.1,
 (4411, 1390, 2): 13.8,
 (4411, 1391, 2): 11.0,
 (4411, 1392, 2): 9.9,
 (4411, 1393, 2): 8.0,
 (4411, 1394, 2): 6.1,
 (4411, 1395, 2): 10.6,
 (4411, 1396, 2): 10.8,
 (4411, 4406, 2): 11.4,
 (4411, 4414, 2): 10.7,
 (4411, 4415, 2): 13.7,
 (4411, 4417, 2): 13.5,
 (4411, 4418, 2): 4.0,
 (4411, 4419, 2): 13.4,
 (4411, 4420, 2): 12.7,
 (4411, 4421, 2): 6.8,
 (4411, 4423, 2): 6.1,
 (4411, 4425, 2): 8.4,
 (4414, 0, 2): 13.2,
 (4414, 1302, 2): 11.8,
 (4414, 1303, 2): 10.5,
 (4414, 1304, 2): 17.9,
 (4414, 1305, 2): 7.8,
 (4414, 1306, 2): 12.8,
 (4414, 1308, 2): 3.2,
 (4414, 1314, 2): 15.1,
 (4414, 1315, 2): 14.8,
 (4414, 1316, 2): 14.8,
 (4414, 1318, 2): 10.4,
 (4414, 1319, 2): 9.4,
 (4414, 1321, 2): 4.2,
 (4414, 1322, 2): 7.9,
 (4414, 1325, 2): 11.0,
 (4414, 1326, 2): 11.0,
 (4414, 1327, 2): 11.0,
 (4414, 1328, 2): 9.2,
 (4414, 1332, 2): 17.5,
 (4414, 1333, 2): 14.7,
 (4414, 1334, 2): 15.2,
 (4414, 1336, 2): 9.2,
 (4414, 1337, 2): 7.4,
 (4414, 1338, 2): 4.7,
 (4414, 1339, 2): 10.7,
 (4414, 1340, 2): 19.1,
 (4414, 1342, 2): 17.0,
 (4414, 1343, 2): 11.5,
 (4414, 1344, 2): 12.8,
 (4414, 1346, 2): 14.7,
 (4414, 1347, 2): 13.7,
 (4414, 1348, 2): 12.2,
 (4414, 1349, 2): 19.6,
 (4414, 1351, 2): 4.1,
 (4414, 1352, 2): 10.8,
 (4414, 1353, 2): 10.9,
 (4414, 1354, 2): 6.5,
 (4414, 1357, 2): 18.2,
 (4414, 1358, 2): 16.2,
 (4414, 1360, 2): 14.6,
 (4414, 1361, 2): 13.9,
 (4414, 1363, 2): 4.7,
 (4414, 1364, 2): 13.4,
 (4414, 1365, 2): 15.1,
 (4414, 1366, 2): 8.1,
 (4414, 1367, 2): 12.2,
 (4414, 1368, 2): 7.6,
 (4414, 1370, 2): 16.7,
 (4414, 1372, 2): 2.6,
 (4414, 1373, 2): 9.9,
 (4414, 1375, 2): 11.6,
 (4414, 1376, 2): 8.5,
 (4414, 1377, 2): 12.1,
 (4414, 1379, 2): 17.2,
 (4414, 1380, 2): 9.2,
 (4414, 1381, 2): 18.2,
 (4414, 1382, 2): 13.2,
 (4414, 1384, 2): 11.4,
 (4414, 1385, 2): 10.2,
 (4414, 1386, 2): 9.8,
 (4414, 1388, 2): 12.9,
 (4414, 1390, 2): 14.3,
 (4414, 1391, 2): 17.0,
 (4414, 1392, 2): 10.2,
 (4414, 1393, 2): 12.8,
 (4414, 1394, 2): 10.8,
 (4414, 1395, 2): 17.6,
 (4414, 1396, 2): 17.8,
 (4414, 4406, 2): 16.4,
 (4414, 4411, 2): 12.4,
 (4414, 4415, 2): 18.0,
 (4414, 4417, 2): 18.1,
 (4414, 4418, 2): 8.3,
 (4414, 4419, 2): 16.9,
 (4414, 4420, 2): 16.2,
 (4414, 4421, 2): 11.5,
 (4414, 4423, 2): 10.9,
 (4414, 4425, 2): 17.5,
 (4415, 0, 2): 19.7,
 (4415, 1302, 2): 10.8,
 (4415, 1303, 2): 9.7,
 (4415, 1304, 2): 16.4,
 (4415, 1305, 2): 13.7,
 (4415, 1306, 2): 16.6,
 (4415, 1308, 2): 14.0,
 (4415, 1314, 2): 8.3,
 (4415, 1315, 2): 9.3,
 (4415, 1316, 2): 12.5,
 (4415, 1318, 2): 5.2,
 (4415, 1319, 2): 8.4,
 (4415, 1321, 2): 11.0,
 (4415, 1322, 2): 9.2,
 (4415, 1325, 2): 12.4,
 (4415, 1326, 2): 12.2,
 (4415, 1327, 2): 12.3,
 (4415, 1328, 2): 10.5,
 (4415, 1332, 2): 19.2,
 (4415, 1333, 2): 16.3,
 (4415, 1334, 2): 16.8,
 (4415, 1336, 2): 13.0,
 (4415, 1337, 2): 14.0,
 (4415, 1338, 2): 11.8,
 (4415, 1339, 2): 7.6,
 (4415, 1340, 2): 15.0,
 (4415, 1342, 2): 15.9,
 (4415, 1343, 2): 10.5,
 (4415, 1344, 2): 11.3,
 (4415, 1346, 2): 3.5,
 (4415, 1347, 2): 4.6,
 (4415, 1348, 2): 5.2,
 (4415, 1349, 2): 15.7,
 (4415, 1351, 2): 14.8,
 (4415, 1352, 2): 21.5,
 (4415, 1353, 2): 6.0,
 (4415, 1354, 2): 10.3,
 (4415, 1357, 2): 9.1,
 (4415, 1358, 2): 10.8,
 (4415, 1360, 2): 10.4,
 (4415, 1361, 2): 12.9,
 (4415, 1363, 2): 13.1,
 (4415, 1364, 2): 12.4,
 (4415, 1365, 2): 14.1,
 (4415, 1366, 2): 12.2,
 (4415, 1367, 2): 5.3,
 (4415, 1368, 2): 9.4,
 (4415, 1370, 2): 15.7,
 (4415, 1372, 2): 11.6,
 (4415, 1373, 2): 11.1,
 (4415, 1375, 2): 11.2,
 (4415, 1376, 2): 15.3,
 (4415, 1377, 2): 13.3,
 (4415, 1379, 2): 3.9,
 (4415, 1380, 2): 10.3,
 (4415, 1381, 2): 19.9,
 (4415, 1382, 2): 13.1,
 (4415, 1384, 2): 8.9,
 (4415, 1385, 2): 12.9,
 (4415, 1386, 2): 13.7,
 (4415, 1388, 2): 16.8,
 (4415, 1390, 2): 19.4,
 (4415, 1391, 2): 16.0,
 (4415, 1392, 2): 9.4,
 (4415, 1393, 2): 5.8,
 (4415, 1394, 2): 5.2,
 (4415, 1395, 2): 15.4,
 (4415, 1396, 2): 15.6,
 (4415, 4406, 2): 16.2,
 (4415, 4411, 2): 9.3,
 (4415, 4414, 2): 13.6,
 (4415, 4417, 2): 7.4,
 (4415, 4418, 2): 7.0,
 (4415, 4419, 2): 18.9,
 (4415, 4420, 2): 18.2,
 (4415, 4421, 2): 6.6,
 (4415, 4423, 2): 4.3,
 (4415, 4425, 2): 13.3,
 (4417, 0, 2): 22.2,
 (4417, 1302, 2): 11.8,
 (4417, 1303, 2): 10.7,
 (4417, 1304, 2): 13.1,
 (4417, 1305, 2): 12.2,
 (4417, 1306, 2): 17.7,
 (4417, 1308, 2): 16.6,
 (4417, 1314, 2): 5.6,
 (4417, 1315, 2): 3.5,
 (4417, 1316, 2): 7.5,
 (4417, 1318, 2): 9.9,
 (4417, 1319, 2): 11.0,
 (4417, 1321, 2): 13.7,
 (4417, 1322, 2): 11.8,
 (4417, 1325, 2): 15.7,
 (4417, 1326, 2): 14.8,
 (4417, 1327, 2): 14.9,
 (4417, 1328, 2): 13.2,
 (4417, 1332, 2): 21.7,
 (4417, 1333, 2): 18.8,
 (4417, 1334, 2): 19.3,
 (4417, 1336, 2): 13.1,
 (4417, 1337, 2): 11.3,
 (4417, 1338, 2): 13.7,
 (4417, 1339, 2): 11.6,
 (4417, 1340, 2): 19.0,
 (4417, 1342, 2): 14.2,
 (4417, 1343, 2): 11.6,
 (4417, 1344, 2): 10.9,
 (4417, 1346, 2): 8.2,
 (4417, 1347, 2): 9.3,
 (4417, 1348, 2): 11.1,
 (4417, 1349, 2): 19.8,
 (4417, 1351, 2): 16.0,
 (4417, 1352, 2): 20.1,
 (4417, 1353, 2): 7.4,
 (4417, 1354, 2): 12.9,
 (4417, 1357, 2): 1.7,
 (4417, 1358, 2): 4.8,
 (4417, 1360, 2): 6.3,
 (4417, 1361, 2): 14.0,
 (4417, 1363, 2): 14.9,
 (4417, 1364, 2): 13.5,
 (4417, 1365, 2): 13.3,
 (4417, 1366, 2): 14.7,
 (4417, 1367, 2): 11.1,
 (4417, 1368, 2): 11.1,
 (4417, 1370, 2): 13.9,
 (4417, 1372, 2): 14.2,
 (4417, 1373, 2): 13.9,
 (4417, 1375, 2): 15.7,
 (4417, 1376, 2): 17.8,
 (4417, 1377, 2): 15.7,
 (4417, 1379, 2): 6.9,
 (4417, 1380, 2): 12.9,
 (4417, 1381, 2): 22.4,
 (4417, 1382, 2): 17.4,
 (4417, 1384, 2): 9.1,
 (4417, 1385, 2): 14.1,
 (4417, 1386, 2): 14.8,
 (4417, 1388, 2): 17.1,
 (4417, 1390, 2): 22.4,
 (4417, 1391, 2): 21.6,
 (4417, 1392, 2): 9.7,
 (4417, 1393, 2): 7.8,
 (4417, 1394, 2): 7.8,
 (4417, 1395, 2): 19.4,
 (4417, 1396, 2): 19.5,
 (4417, 4406, 2): 20.4,
 (4417, 4411, 2): 13.7,
 (4417, 4414, 2): 18.4,
 (4417, 4415, 2): 8.2,
 (4417, 4418, 2): 10.0,
 (4417, 4419, 2): 22.0,
 (4417, 4420, 2): 21.2,
 (4417, 4421, 2): 11.5,
 (4417, 4423, 2): 10.8,
 (4417, 4425, 2): 17.7,
 (4418, 0, 2): 15.0,
 (4418, 1302, 2): 14.1,
 (4418, 1303, 2): 12.5,
 (4418, 1304, 2): 20.4,
 (4418, 1305, 2): 12.4,
 (4418, 1306, 2): 17.2,
 (4418, 1308, 2): 7.0,
 (4418, 1314, 2): 8.5,
 (4418, 1315, 2): 9.6,
 (4418, 1316, 2): 10.0,
 (4418, 1318, 2): 5.2,
 (4418, 1319, 2): 4.7,
 (4418, 1321, 2): 5.4,
 (4418, 1322, 2): 5.4,
 (4418, 1325, 2): 9.0,
 (4418, 1326, 2): 8.7,
 (4418, 1327, 2): 8.7,
 (4418, 1328, 2): 7.0,
 (4418, 1332, 2): 15.5,
 (4418, 1333, 2): 12.6,
 (4418, 1334, 2): 13.2,
 (4418, 1336, 2): 13.8,
 (4418, 1337, 2): 11.9,
 (4418, 1338, 2): 9.4,
 (4418, 1339, 2): 6.1,
 (4418, 1340, 2): 13.6,
 (4418, 1342, 2): 19.1,
 (4418, 1343, 2): 13.7,
 (4418, 1344, 2): 14.6,
 (4418, 1346, 2): 9.9,
 (4418, 1347, 2): 8.8,
 (4418, 1348, 2): 7.4,
 (4418, 1349, 2): 14.2,
 (4418, 1351, 2): 8.0,
 (4418, 1352, 2): 14.3,
 (4418, 1353, 2): 5.9,
 (4418, 1354, 2): 3.2,
 (4418, 1357, 2): 13.2,
 (4418, 1358, 2): 11.0,
 (4418, 1360, 2): 9.2,
 (4418, 1361, 2): 16.7,
 (4418, 1363, 2): 9.4,
 (4418, 1364, 2): 16.0,
 (4418, 1365, 2): 18.2,
 (4418, 1366, 2): 8.9,
 (4418, 1367, 2): 7.5,
 (4418, 1368, 2): 12.4,
 (4418, 1370, 2): 19.8,
 (4418, 1372, 2): 4.5,
 (4418, 1373, 2): 7.7,
 (4418, 1375, 2): 9.5,
 (4418, 1376, 2): 8.2,
 (4418, 1377, 2): 10.2,
 (4418, 1379, 2): 11.9,
 (4418, 1380, 2): 6.5,
 (4418, 1381, 2): 16.2,
 (4418, 1382, 2): 11.2,
 (4418, 1384, 2): 12.0,
 (4418, 1385, 2): 14.8,
 (4418, 1386, 2): 14.4,
 (4418, 1388, 2): 17.2,
 (4418, 1390, 2): 16.6,
 (4418, 1391, 2): 14.8,
 (4418, 1392, 2): 7.7,
 (4418, 1393, 2): 7.9,
 (4418, 1394, 2): 5.5,
 (4418, 1395, 2): 13.4,
 (4418, 1396, 2): 13.6,
 (4418, 4406, 2): 14.3,
 (4418, 4411, 2): 7.8,
 (4418, 4414, 2): 6.5,
 (4418, 4415, 2): 13.5,
 (4418, 4417, 2): 12.8,
 (4418, 4419, 2): 15.2,
 (4418, 4420, 2): 14.5,
 (4418, 4421, 2): 7.0,
 (4418, 4423, 2): 6.3,
 (4418, 4425, 2): 11.8,
 (4419, 0, 2): 22.6,
 (4419, 1302, 2): 24.5,
 (4419, 1303, 2): 23.6,
 (4419, 1304, 2): 29.9,
 (4419, 1305, 2): 24.4,
 (4419, 1306, 2): 29.8,
 (4419, 1308, 2): 19.1,
 (4419, 1314, 2): 19.4,
 (4419, 1315, 2): 20.4,
 (4419, 1316, 2): 19.8,
 (4419, 1318, 2): 16.1,
 (4419, 1319, 2): 12.6,
 (4419, 1321, 2): 14.9,
 (4419, 1322, 2): 15.0,
 (4419, 1325, 2): 12.1,
 (4419, 1326, 2): 9.1,
 (4419, 1327, 2): 10.1,
 (4419, 1328, 2): 10.8,
 (4419, 1332, 2): 8.8,
 (4419, 1333, 2): 8.7,
 (4419, 1334, 2): 9.2,
 (4419, 1336, 2): 25.7,
 (4419, 1337, 2): 24.1,
 (4419, 1338, 2): 21.5,
 (4419, 1339, 2): 16.2,
 (4419, 1340, 2): 14.2,
 (4419, 1342, 2): 29.9,
 (4419, 1343, 2): 24.2,
 (4419, 1344, 2): 24.9,
 (4419, 1346, 2): 20.4,
 (4419, 1347, 2): 19.4,
 (4419, 1348, 2): 18.0,
 (4419, 1349, 2): 14.4,
 (4419, 1351, 2): 20.0,
 (4419, 1352, 2): 24.1,
 (4419, 1353, 2): 16.8,
 (4419, 1354, 2): 16.9,
 (4419, 1357, 2): 24.0,
 (4419, 1358, 2): 21.9,
 (4419, 1360, 2): 20.0,
 (4419, 1361, 2): 27.0,
 (4419, 1363, 2): 21.5,
 (4419, 1364, 2): 26.4,
 (4419, 1365, 2): 28.4,
 (4419, 1366, 2): 18.3,
 (4419, 1367, 2): 18.0,
 (4419, 1368, 2): 24.4,
 (4419, 1370, 2): 30.0,
 (4419, 1372, 2): 16.6,
 (4419, 1373, 2): 12.1,
 (4419, 1375, 2): 13.0,
 (4419, 1376, 2): 16.4,
 (4419, 1377, 2): 10.1,
 (4419, 1379, 2): 23.2,
 (4419, 1380, 2): 12.7,
 (4419, 1381, 2): 11.2,
 (4419, 1382, 2): 14.7,
 (4419, 1384, 2): 23.2,
 (4419, 1385, 2): 26.2,
 (4419, 1386, 2): 26.6,
 (4419, 1388, 2): 29.9,
 (4419, 1390, 2): 4.5,
 (4419, 1391, 2): 10.9,
 (4419, 1392, 2): 20.3,
 (4419, 1393, 2): 18.5,
 (4419, 1394, 2): 16.4,
 (4419, 1395, 2): 12.7,
 (4419, 1396, 2): 12.8,
 (4419, 4406, 2): 11.4,
 (4419, 4411, 2): 13.8,
 (4419, 4414, 2): 22.2,
 (4419, 4415, 2): 24.0,
 (4419, 4417, 2): 23.8,
 (4419, 4418, 2): 13.8,
 (4419, 4420, 2): 2.4,
 (4419, 4421, 2): 17.3,
 (4419, 4423, 2): 16.6,
 (4419, 4425, 2): 12.5,
 (4420, 0, 2): 21.7,
 (4420, 1302, 2): 23.6,
 (4420, 1303, 2): 22.2,
 (4420, 1304, 2): 29.8,
 (4420, 1305, 2): 23.5,
 (4420, 1306, 2): 28.9,
 (4420, 1308, 2): 18.2,
 (4420, 1314, 2): 18.4,
 (4420, 1315, 2): 19.5,
 (4420, 1316, 2): 18.8,
 (4420, 1318, 2): 15.1,
 (4420, 1319, 2): 11.7,
 (4420, 1321, 2): 14.0,
 (4420, 1322, 2): 14.1,
 (4420, 1325, 2): 11.2,
 (4420, 1326, 2): 8.2,
 (4420, 1327, 2): 9.2,
 (4420, 1328, 2): 10.0,
 (4420, 1332, 2): 7.9,
 (4420, 1333, 2): 7.8,
 (4420, 1334, 2): 8.7,
 (4420, 1336, 2): 24.8,
 (4420, 1337, 2): 23.2,
 (4420, 1338, 2): 20.6,
 (4420, 1339, 2): 15.4,
 (4420, 1340, 2): 13.4,
 (4420, 1342, 2): 29.0,
 (4420, 1343, 2): 23.3,
 (4420, 1344, 2): 24.0,
 (4420, 1346, 2): 19.5,
 (4420, 1347, 2): 18.5,
 (4420, 1348, 2): 17.0,
 (4420, 1349, 2): 13.5,
 (4420, 1351, 2): 19.1,
 (4420, 1352, 2): 23.3,
 (4420, 1353, 2): 15.9,
 (4420, 1354, 2): 16.0,
 (4420, 1357, 2): 23.2,
 (4420, 1358, 2): 21.0,
 (4420, 1360, 2): 19.2,
 (4420, 1361, 2): 26.1,
 (4420, 1363, 2): 20.6,
 (4420, 1364, 2): 25.5,
 (4420, 1365, 2): 27.5,
 (4420, 1366, 2): 17.4,
 (4420, 1367, 2): 17.1,
 (4420, 1368, 2): 21.4,
 (4420, 1370, 2): 29.2,
 (4420, 1372, 2): 15.7,
 (4420, 1373, 2): 11.2,
 (4420, 1375, 2): 12.2,
 (4420, 1376, 2): 15.4,
 (4420, 1377, 2): 9.2,
 (4420, 1379, 2): 22.3,
 (4420, 1380, 2): 11.8,
 (4420, 1381, 2): 10.3,
 (4420, 1382, 2): 13.8,
 (4420, 1384, 2): 21.7,
 (4420, 1385, 2): 25.3,
 (4420, 1386, 2): 26.0,
 (4420, 1388, 2): 29.0,
 (4420, 1390, 2): 3.6,
 (4420, 1391, 2): 10.0,
 (4420, 1392, 2): 19.4,
 (4420, 1393, 2): 17.6,
 (4420, 1394, 2): 15.6,
 (4420, 1395, 2): 11.8,
 (4420, 1396, 2): 12.0,
 (4420, 4406, 2): 10.5,
 (4420, 4411, 2): 13.0,
 (4420, 4414, 2): 17.6,
 (4420, 4415, 2): 23.1,
 (4420, 4417, 2): 22.9,
 (4420, 4418, 2): 12.9,
 (4420, 4419, 2): 2.5,
 (4420, 4421, 2): 16.4,
 (4420, 4423, 2): 15.7,
 (4420, 4425, 2): 11.6,
 (4421, 0, 2): 18.0,
 (4421, 1302, 2): 10.6,
 (4421, 1303, 2): 9.6,
 (4421, 1304, 2): 16.9,
 (4421, 1305, 2): 12.6,
 (4421, 1306, 2): 15.7,
 (4421, 1308, 2): 12.5,
 (4421, 1314, 2): 6.8,
 (4421, 1315, 2): 7.9,
 (4421, 1316, 2): 11.0,
 (4421, 1318, 2): 3.5,
 (4421, 1319, 2): 6.8,
 (4421, 1321, 2): 9.4,
 (4421, 1322, 2): 7.6,
 (4421, 1325, 2): 11.4,
 (4421, 1326, 2): 11.0,
 (4421, 1327, 2): 10.9,
 (4421, 1328, 2): 9.3,
 (4421, 1332, 2): 17.8,
 (4421, 1333, 2): 14.9,
 (4421, 1334, 2): 15.4,
 (4421, 1336, 2): 12.6,
 (4421, 1337, 2): 12.9,
 (4421, 1338, 2): 15.2,
 (4421, 1339, 2): 5.8,
 (4421, 1340, 2): 13.3,
 (4421, 1342, 2): 15.6,
 (4421, 1343, 2): 10.2,
 (4421, 1344, 2): 11.1,
 (4421, 1346, 2): 6.4,
 (4421, 1347, 2): 5.3,
 (4421, 1348, 2): 3.9,
 (4421, 1349, 2): 14.0,
 (4421, 1351, 2): 16.5,
 (4421, 1352, 2): 17.9,
 (4421, 1353, 2): 4.3,
 (4421, 1354, 2): 8.7,
 (4421, 1357, 2): 11.5,
 (4421, 1358, 2): 9.4,
 (4421, 1360, 2): 9.0,
 (4421, 1361, 2): 13.2,
 (4421, 1363, 2): 15.3,
 (4421, 1364, 2): 12.5,
 (4421, 1365, 2): 14.7,
 (4421, 1366, 2): 10.4,
 (4421, 1367, 2): 4.0,
 (4421, 1368, 2): 8.2,
 (4421, 1370, 2): 16.3,
 (4421, 1372, 2): 10.0,
 (4421, 1373, 2): 10.0,
 (4421, 1375, 2): 9.5,
 (4421, 1376, 2): 13.6,
 (4421, 1377, 2): 12.1,
 (4421, 1379, 2): 8.2,
 (4421, 1380, 2): 8.8,
 (4421, 1381, 2): 18.4,
 (4421, 1382, 2): 11.4,
 (4421, 1384, 2): 8.6,
 (4421, 1385, 2): 12.1,
 (4421, 1386, 2): 12.8,
 (4421, 1388, 2): 15.8,
 (4421, 1390, 2): 18.0,
 (4421, 1391, 2): 15.5,
 (4421, 1392, 2): 11.2,
 (4421, 1393, 2): 4.4,
 (4421, 1394, 2): 3.8,
 (4421, 1395, 2): 13.2,
 (4421, 1396, 2): 13.3,
 (4421, 4406, 2): 14.2,
 (4421, 4411, 2): 7.5,
 (4421, 4414, 2): 11.9,
 (4421, 4415, 2): 10.9,
 (4421, 4417, 2): 11.1,
 (4421, 4418, 2): 5.5,
 (4421, 4419, 2): 17.5,
 (4421, 4420, 2): 16.8,
 (4421, 4423, 2): 1.0,
 (4421, 4425, 2): 11.5,
 (4423, 0, 2): 19.2,
 (4423, 1302, 2): 9.9,
 (4423, 1303, 2): 8.9,
 (4423, 1304, 2): 16.2,
 (4423, 1305, 2): 12.0,
 (4423, 1306, 2): 15.0,
 (4423, 1308, 2): 13.6,
 (4423, 1314, 2): 6.2,
 (4423, 1315, 2): 7.2,
 (4423, 1316, 2): 10.3,
 (4423, 1318, 2): 5.9,
 (4423, 1319, 2): 8.0,
 (4423, 1321, 2): 10.6,
 (4423, 1322, 2): 8.8,
 (4423, 1325, 2): 12.6,
 (4423, 1326, 2): 12.2,
 (4423, 1327, 2): 12.1,
 (4423, 1328, 2): 10.5,
 (4423, 1332, 2): 19.0,
 (4423, 1333, 2): 16.1,
 (4423, 1334, 2): 16.6,
 (4423, 1336, 2): 12.0,
 (4423, 1337, 2): 12.2,
 (4423, 1338, 2): 13.2,
 (4423, 1339, 2): 8.4,
 (4423, 1340, 2): 15.8,
 (4423, 1342, 2): 13.6,
 (4423, 1343, 2): 8.2,
 (4423, 1344, 2): 9.0,
 (4423, 1346, 2): 6.6,
 (4423, 1347, 2): 5.0,
 (4423, 1348, 2): 3.2,
 (4423, 1349, 2): 17.4,
 (4423, 1351, 2): 14.1,
 (4423, 1352, 2): 15.6,
 (4423, 1353, 2): 4.4,
 (4423, 1354, 2): 10.0,
 (4423, 1357, 2): 10.5,
 (4423, 1358, 2): 8.6,
 (4423, 1360, 2): 7.0,
 (4423, 1361, 2): 11.6,
 (4423, 1363, 2): 13.0,
 (4423, 1364, 2): 10.8,
 (4423, 1365, 2): 13.2,
 (4423, 1366, 2): 12.3,
 (4423, 1367, 2): 3.3,
 (4423, 1368, 2): 7.4,
 (4423, 1370, 2): 14.8,
 (4423, 1372, 2): 11.2,
 (4423, 1373, 2): 11.5,
 (4423, 1375, 2): 12.2,
 (4423, 1376, 2): 14.8,
 (4423, 1377, 2): 14.0,
 (4423, 1379, 2): 7.1,
 (4423, 1380, 2): 10.3,
 (4423, 1381, 2): 19.9,
 (4423, 1382, 2): 15.2,
 (4423, 1384, 2): 6.6,
 (4423, 1385, 2): 9.7,
 (4423, 1386, 2): 10.5,
 (4423, 1388, 2): 13.1,
 (4423, 1390, 2): 19.5,
 (4423, 1391, 2): 16.8,
 (4423, 1392, 2): 10.6,
 (4423, 1393, 2): 3.7,
 (4423, 1394, 2): 3.2,
 (4423, 1395, 2): 15.5,
 (4423, 1396, 2): 15.6,
 (4423, 4406, 2): 17.9,
 (4423, 4411, 2): 10.0,
 (4423, 4414, 2): 16.8,
 (4423, 4415, 2): 11.4,
 (4423, 4417, 2): 10.5,
 (4423, 4418, 2): 6.8,
 (4423, 4419, 2): 19.0,
 (4423, 4420, 2): 18.3,
 (4423, 4421, 2): 0.9,
 (4423, 4425, 2): 14.0,
 (4425, 0, 2): 20.6,
 (4425, 1302, 2): 13.6,
 (4425, 1303, 2): 12.6,
 (4425, 1304, 2): 20.3,
 (4425, 1305, 2): 15.2,
 (4425, 1306, 2): 18.0,
 (4425, 1308, 2): 14.2,
 (4425, 1314, 2): 11.0,
 (4425, 1315, 2): 11.8,
 (4425, 1316, 2): 13.2,
 (4425, 1318, 2): 8.6,
 (4425, 1319, 2): 8.4,
 (4425, 1321, 2): 11.6,
 (4425, 1322, 2): 9.9,
 (4425, 1325, 2): 10.0,
 (4425, 1326, 2): 6.5,
 (4425, 1327, 2): 9.4,
 (4425, 1328, 2): 8.5,
 (4425, 1332, 2): 10.7,
 (4425, 1333, 2): 9.0,
 (4425, 1334, 2): 9.6,
 (4425, 1336, 2): 16.4,
 (4425, 1337, 2): 15.4,
 (4425, 1338, 2): 17.6,
 (4425, 1339, 2): 7.7,
 (4425, 1340, 2): 5.5,
 (4425, 1342, 2): 18.6,
 (4425, 1343, 2): 13.2,
 (4425, 1344, 2): 13.9,
 (4425, 1346, 2): 11.0,
 (4425, 1347, 2): 9.7,
 (4425, 1348, 2): 7.9,
 (4425, 1349, 2): 5.9,
 (4425, 1351, 2): 15.2,
 (4425, 1352, 2): 21.5,
 (4425, 1353, 2): 9.1,
 (4425, 1354, 2): 10.6,
 (4425, 1357, 2): 15.3,
 (4425, 1358, 2): 13.4,
 (4425, 1360, 2): 13.4,
 (4425, 1361, 2): 16.3,
 (4425, 1363, 2): 17.8,
 (4425, 1364, 2): 15.6,
 (4425, 1365, 2): 17.8,
 (4425, 1366, 2): 12.8,
 (4425, 1367, 2): 8.0,
 (4425, 1368, 2): 12.1,
 (4425, 1370, 2): 19.4,
 (4425, 1372, 2): 11.9,
 (4425, 1373, 2): 9.7,
 (4425, 1375, 2): 6.7,
 (4425, 1376, 2): 15.6,
 (4425, 1377, 2): 7.6,
 (4425, 1379, 2): 12.8,
 (4425, 1380, 2): 10.2,
 (4425, 1381, 2): 12.6,
 (4425, 1382, 2): 8.6,
 (4425, 1384, 2): 11.5,
 (4425, 1385, 2): 14.6,
 (4425, 1386, 2): 15.3,
 (4425, 1388, 2): 18.2,
 (4425, 1390, 2): 11.7,
 (4425, 1391, 2): 6.6,
 (4425, 1392, 2): 15.1,
 (4425, 1393, 2): 8.5,
 (4425, 1394, 2): 7.9,
 (4425, 1395, 2): 4.1,
 (4425, 1396, 2): 4.1,
 (4425, 4406, 2): 5.9,
 (4425, 4411, 2): 8.2,
 (4425, 4414, 2): 13.8,
 (4425, 4415, 2): 13.0,
 (4425, 4417, 2): 16.1,
 (4425, 4418, 2): 7.4,
 (4425, 4419, 2): 11.3,
 (4425, 4420, 2): 10.5,
 (4425, 4421, 2): 7.0,
 (4425, 4423, 2): 6.2}
# --- 이 코드로 G 생성 및 보완 셀을 완전히 교체하세요 ---

# 1. 원본 travel_time을 복사하여 G 생성
# (이 시점의 G는 t=2 데이터만 가지고 있다고 가정)
G = travel_time.copy()


# 2. [신규] 기간(period) 2 데이터를 기간 1로 복사하는 로직
# G.keys()를 리스트로 감싸야 반복 중 딕셔너리 변경 오류가 발생하지 않습니다.
for key in list(G.keys()):
    i, j, t = key
    # 키의 기간(t)이 2일 경우
    if t == 2:
        # 기간 1에 해당하는 키를 생성
        period_1_key = (i, j, 1)
        # 기간 1 키가 G에 존재하지 않는 경우에만 값을 복사
        if period_1_key not in G:
            G[period_1_key] = G[key] # G[(i, j, 2)]의 값을 G[(i, j, 1)]로 복사





### 휴리스틱 + LP

In [ ]:
import pulp
from pulp import *
import math
from datetime import datetime
from zoneinfo import ZoneInfo

# 실행 시점 확인 및 계획 기간 설정
now_kst = datetime.now(ZoneInfo('Asia/Seoul'))
current_hour = now_kst.hour

# 8시 ~ 19시(20시 전) 사이 실행 시 -> '야간(Period 2)' 운행 계획
if 8 <= current_hour < 20:
    Periods_to_plan = [2]
    print("="*70)
    print(f"⚙️ [야간 운행] 계획 수립을 시작합니다. (대상 Period: 2)")
    print("="*70)
# 20시 ~ 다음날 7시(8시 전) 사이 실행 시 -> '주간(Period 1)' 운행 계획
else:
    Periods_to_plan = [1]
    print("="*70)
    print(f"⚙️ [주간 운행] 계획 수립을 시작합니다. (대상 Period: 1)")
    print("="*70)


# 사용자 설정: 휴리스틱 파라미터

VEHICLE_CAPACITY = 30         # C: 차량 용량 (대)
MAX_CONSECUTIVE_ACTIONS = 2   # 연속 '배달' 또는 '수거' 최대 허용 횟수 (2회로 제한)

PERIOD_LENGTH_MINS = 180    # L: 계획 기간 길이 (분)
SERVICE_TIME_MINS = 5       # H_i: 대여소 평균 서비스 시간 (분)
BIG_M = 10000               # M: 충분히 큰 수 (제약 조건용)

# 최대 상/하차량 결과 제한 값 (Phase 2에서 사용)
MAX_LOAD_UNLOAD_CAP = 15

# 순수요 상/하한 제한 값
NET_DEMAND_MIN = -10
NET_DEMAND_MAX = 10

# 재고 상/하한 설정
USE_DEMAND_ADAPTIVE_BOUNDS = True
TARGET_INVENTORY_RATIO = 0.7      # 이상적 목표 재고 비율 (예: 70%)
INVENTORY_BUFFER_C = 10            # 평상시(t=1) 목표 재고 허용 범위 (±C)
INVENTORY_BUFFER_C_FINAL = 10      # 마지막 기간(t=2) 목표 재고 허용 범위 (±C)

# (USE_DEMAND_ADAPTIVE_BOUNDS = False 일 때 사용되는 정적 비율)
S_MIN_RATIO = 0.1
S_MAX_RATIO = 0.9
S_MIN_RATIO_FINAL = 0.1
S_MAX_RATIO_FINAL = 0.9

# 휴리스틱 실행 위한 데이터 준비 (From Script Variables)

Stations = list(station_ID) # 대여소 목록 (int)
Depot = 0
Nodes = Stations + [Depot] # 대여소 + 차고지
Periods = Periods_to_plan  # 동적으로 할당된 [1] 또는 [2]를 사용




if 'G' in locals() and 'Periods' in locals() and 'Depot' in locals():
    for t in Periods:
         G[(Depot, Depot, t)] = 0.0

else:
    print("   - ⚠️ 오류: G, Periods, Depot 변수 중 하나가 정의되지 않았습니다.")
print("="*70)

# S0_initial (Period 1 초기 재고) <- realtime_initial_stock (dict{int: int})
S0_initial = realtime_initial_stock
# rack_counts (대여소별 거치대 수) <- api_data_by_name (dict{int: (int, int)})
rack_counts = {sid: api_data_by_name.get(sid, (0, 10))[1] for sid in Stations} # 기본 10개 가정

# B (순수요) <- net_demand (dict{(int, int): float})
B = net_demand

# 순수요(B) 값 상/하한 제한
original_b_count = len(B)
capped_count = 0
for key, val in B.items():
    capped_value = max(NET_DEMAND_MIN, min(val, NET_DEMAND_MAX))
    if capped_value != val:
        capped_count += 1
    B[key] = capped_value



# 수요율 (E_it = B_it / L)
if PERIOD_LENGTH_MINS <= 0:
    print("🚨 오류: PERIOD_LENGTH_MINS는 0보다 커야 합니다. 기본값 180으로 설정합니다.")
    PERIOD_LENGTH_MINS = 180
E = {key: B.get(key, 0) / PERIOD_LENGTH_MINS for key in B if PERIOD_LENGTH_MINS > 0}

# 결과 저장을 위한 딕셔너리
solution_details = {}



# [Phase 2 함수 정의]
def solve_loading_for_route(fixed_route, t, S0, B, E, q, Smin, Smax, G):
    """
    고정된 경로(fixed_route)에 대한 최적 적재 계획 수립 함수 (PuLP 사용)

    """

    prob = LpProblem(f"Loading_Plan_Period_{t}", LpMinimize)
    route_stations = [i for i in fixed_route if i != Depot]

    if not route_stations:
        print("   - [Phase 2] 방문할 대여소가 없어 적재 계획을 생략합니다.")
        return {'status': 'Optimal', 'actions': {}, 'objective': 0, 'route': fixed_route, 'total_time': 0, 'final_inv': {}}

    # --- 1. 변수 정의 ---
    s = LpVariable.dicts("Inventory", [(i, t) for i in route_stations], lowBound=0, cat='Continuous')
    d = LpVariable.dicts("Delivery", [(i, t) for i in route_stations], lowBound=0, cat='Integer')
    p = LpVariable.dicts("Pickup", [(i, t) for i in route_stations], lowBound=0, cat='Integer')
    r = LpVariable.dicts("Imbalance", [(i, t) for i in route_stations], lowBound=0, cat='Continuous')

    load = LpVariable.dicts("VehicleLoad", range(len(fixed_route)), lowBound=0, upBound=VEHICLE_CAPACITY, cat='Continuous')
    initial_load = LpVariable("Initial_Load", lowBound=0, upBound=VEHICLE_CAPACITY, cat='Integer')

    # 제약 완화를 위한 벌금
    slack_min = LpVariable.dicts("Slack_Min", [(i, t) for i in route_stations], lowBound=0, cat='Continuous')
    slack_max = LpVariable.dicts("Slack_Max", [(i, t) for i in route_stations], lowBound=0, cat='Continuous')


    # --- 2. 도착 시간 계산 ---
    arrival_times = {}
    current_time = 0.0
    total_travel_time = 0.0
    total_service_time = 0.0

    for k in range(len(fixed_route) - 1):
        i = fixed_route[k]
        j = fixed_route[k+1]
        travel_time = G.get((i, j, t), BIG_M)
        if travel_time >= BIG_M:
             print(f"   ⚠️ 경고: 이동 시간 G[({i}, {j}, {t})] 값이 없습니다. BIG_M으로 대체합니다.")
        if i != Depot:
            current_time += SERVICE_TIME_MINS
            total_service_time += SERVICE_TIME_MINS
        current_time += travel_time
        total_travel_time += travel_time
        if j != Depot:
            arrival_times[j] = current_time
    total_time = total_travel_time + total_service_time

    # --- 3.  목적 함수   ---
    #  목적 1순위: 재고 상/하한 위반(벌금) 최소화
    PENALTY_COST = 1000
    penalty_objective = PENALTY_COST * (lpSum(slack_min[i, t] for i in route_stations) +
                                       lpSum(slack_max[i, t] for i in route_stations))

    # 목적 2순위: 불균형(r_it) 최소화
    primary_objective = lpSum(r[i, t] for i in route_stations)

    # 목적 3순위: 총 작업량(비용) 최소화
    COST_WEIGHT = 0.001
    secondary_objective = COST_WEIGHT * (lpSum(d[i, t] for i in route_stations) + lpSum(p[i, t] for i in route_stations))

    # 최종 목적 함수 = 벌금 최소화(1순위) + 불균형 최소화(2순위) + 비용 최소화(3순위)
    prob += penalty_objective + primary_objective + secondary_objective

    # --- 4.  제약 조건 ---
    prob += load[0] == initial_load
    for k in range(1, len(fixed_route)):
        i = fixed_route[k]
        if i == Depot: continue
        a_it = arrival_times[i]
        S0_i = S0.get(i, 0)
        E_it = E.get((i, t), 0)
        B_it = B.get((i, t), 0)
        q_it = q.get((i, t), PERIOD_LENGTH_MINS)
        Smin_i = Smin.get(i, 0)
        Smax_i = Smax.get(i, rack_counts.get(i, 10))
        prev_node_idx = k - 1

        prob += s[i, t] == S0_i + B_it + d[i, t] - p[i, t]


        # Smin_i 보다 모자라도 되면(slack_min > 0) 허용
        prob += s[i, t] + slack_min[i, t] >= Smin_i
        # Smax_i 보다 넘쳐도 되면(slack_max > 0) 허용
        prob += s[i, t] - slack_max[i, t] <= Smax_i

        # (기존 제약 조건들)
        available_at_arrival = S0_i + E_it * a_it
        prob += p[i, t] <= max(0.0, available_at_arrival)
        prob += r[i, t] >= (a_it - q_it) * abs(E_it)
        prob += load[k] == load[prev_node_idx] - d[i, t] + p[i, t]

    # --- 5. 솔버 실행 및 결과 반환 ---
    prob.solve(PULP_CBC_CMD(msg=False))

    status = LpStatus[prob.status]
    if status == 'Optimal':
        actions = {}
        final_inv = {}
        total_slack_used = 0

        for i in route_stations:
            delivered_suggested = d[i, t].varValue
            picked_up_suggested = p[i, t].varValue

            # MAX_LOAD_UNLOAD_CAP (10)로 상/하차량 제한
            delivered_capped = min(delivered_suggested, MAX_LOAD_UNLOAD_CAP)
            picked_up_capped = min(picked_up_suggested, MAX_LOAD_UNLOAD_CAP)

            if delivered_capped > 0.1 or picked_up_capped > 0.1:
                actions[i] = {'delivery': round(delivered_capped), 'pickup': round(picked_up_capped)}
            final_inv[i] = s[i, t].varValue


            total_slack_used += slack_min[i, t].varValue + slack_max[i, t].varValue




        return {
            'status': status,
            'objective': value(prob.objective),
            'route': fixed_route,
            'actions': actions,
            'total_time': total_time,
            'final_inv': final_inv,
            'initial_load': initial_load.varValue
        }
    else:

        print(f"   - ❗️ [Phase 2] 심각한 오류: Soft Constraint로도 해를 찾지 못했습니다. (Status: {status})")
        return {
            'status': status,
            'objective': 0,
            'route': fixed_route,
            'actions': {},
            'total_time': total_time,
            'initial_load': 0
        }




def estimate_station_action_quantity(station_id, S0, rack_counts, Smin, Smax, C, max_load_per_stop):
    """
    [Phase 1용] 현재 상태(S0)만을 기준으로 예상 상/하차량을 '시뮬레이션'합니다.
    """
    s0_i = S0.get(station_id, 0)
    rack = rack_counts.get(station_id, 10)
    s_min_i = Smin.get(station_id, 1)
    s_max_i = Smax.get(station_id, rack)

    # 목표 재고를 상/하한의 중간으로 단순 가정
    target_inv = (s_min_i + s_max_i) / 2

    p = 0.0
    d = 0.0
    action_type = 'neutral'

    # S0가 이미 목표 범위를 벗어났는지 확인
    if s0_i > s_max_i:
        # (현재 재고 - 목표 재고) 만큼 수거 시도
        p = s0_i - target_inv
        action_type = 'pickup'
    elif s0_i < s_min_i:
        # (목표 재고 - 현재 재고) 만큼 배달 시도
        d = target_inv - s0_i
        action_type = 'delivery'

    # 차량 전체 용량(C)과 정류소당 최대 작업량(max_load_per_stop) 중 작은 값으로 제한
    cap_limit = min(C, max_load_per_stop)
    p = min(p, cap_limit)
    d = min(d, cap_limit)

    return {'delivery': math.ceil(d), 'pickup': math.ceil(p), 'action_type': action_type}


# [Phase 1 함수 정의: 경로 구축]
def build_insertion_route(t, q, G, S0_current_period, rack_counts, Smin, Smax,
                          C, L_op, H, MAX_CONSECUTIVE, MAX_LOAD_PER_STOP):
    """
    Builds route using insertion heuristic based on:
    1. Urgency (q_it)
    2. Cheapest insertion cost (travel time)
    3. Time Constraint (L_op)
    4. Consecutive Action Constraint (MAX_CONSECUTIVE)
    5. Capacity Constraint (C)
    """
    print(f"   - [Phase 1] 방문 후보 선정 및 정렬 (q_it 기준)...")
    candidates = {}
    for i in Stations:
        q_it = q.get((i, t), L_op)
        if q_it < L_op: # 방문이 필요한 시급한 곳만
            candidates[i] = q_it

    sorted_candidates_by_qit = sorted(candidates.items(), key=lambda item: (item[1], item[0]))

    if not sorted_candidates_by_qit:
        print(f"   - [Phase 1] 기간 {t}에 방문할 '시급한' 후보 대여소가 없습니다. (경로 생성 즉시 종료)")
        return [Depot, Depot]

    print(f"   - [Phase 1] {len(sorted_candidates_by_qit)}개 후보 대상 경로 삽입 시도 (C={C}, L_op={L_op}, H={H}, MaxConsec={MAX_CONSECUTIVE})...")

    # [초기화]
    depot_action = {'delivery': 0, 'pickup': 0, 'action_type': 'depot'}
    route = [Depot, Depot]
    route_actions = [depot_action, depot_action] # 경로와 1:1 매칭

    current_total_travel_time = 0.0
    stations_in_route = 0
    skipped_stations = 0

    total_candidates = len(sorted_candidates_by_qit)
    print(f"      - [Phase 1] 총 {total_candidates}개 후보 검토 시작...")

    # [후보 순회]
    for idx, (station_k, q_k) in enumerate(sorted_candidates_by_qit):

        if idx == 0 or (idx + 1) % 5 == 0 or (idx + 1) == total_candidates:
            print(f"      - [Phase 1] 후보 검토 중... ({idx + 1} / {total_candidates}) -> {station_k}번 대여소")

        # [행동 예측]
        est_action_k = estimate_station_action_quantity(
            station_k, S0_current_period, rack_counts, Smin, Smax, C, MAX_LOAD_PER_STOP
        )
        action_type_k = est_action_k['action_type']

        best_insertion_cost = float('inf')
        best_position = -1

        # [1. 최적 위치 탐색 (시간 기준)]
        for pos in range(len(route) - 1):
            i = route[pos]
            j = route[pos+1]
            cost_i_j = G.get((i, j, t), BIG_M)
            cost_i_k = G.get((i, station_k, t), BIG_M)
            cost_k_j = G.get((station_k, j, t), BIG_M)

            if BIG_M in [cost_i_j, cost_i_k, cost_k_j]:
                 continue

            insertion_cost = (cost_i_k + cost_k_j) - cost_i_j
            if insertion_cost < best_insertion_cost:
                best_insertion_cost = insertion_cost
                best_position = pos + 1

        # [2. 제약 조건 검사]
        if best_position != -1:

            # --- [검사 2a] 시간 제약 (Time Constraint) ---
            new_total_travel_time = current_total_travel_time + best_insertion_cost
            new_total_service_time = (stations_in_route + 1) * H
            time_ok = (new_total_travel_time + new_total_service_time <= L_op)

            # --- [검사 2b] 연속 행동 제약 (Consecutive Action Constraint) ---
            consecutive_ok = True
            if action_type_k in ['pickup', 'delivery']:
                consecutive_match_count = 0

                if best_position > 0:
                    action_prev_1 = route_actions[best_position - 1]['action_type']
                    if action_prev_1 == action_type_k:
                        consecutive_match_count += 1
                        if best_position > 1:
                            action_prev_2 = route_actions[best_position - 2]['action_type']
                            if action_prev_2 == action_type_k:
                                consecutive_match_count += 1

                if consecutive_match_count >= MAX_CONSECUTIVE:
                    consecutive_ok = False

            # --- [검사 2c] 용량 제약 (Capacity Constraint) ---
            capacity_ok = True
            current_load = 0

            temp_route = route[:best_position] + [station_k] + route[best_position:]
            temp_actions = route_actions[:best_position] + [est_action_k] + route_actions[best_position:]

            for k_sim in range(1, len(temp_route) - 1):
                action = temp_actions[k_sim]
                current_load = current_load - action['delivery'] + action['pickup']

                if not (0 <= current_load <= C):
                    capacity_ok = False
                    break

            # [3. 최종 삽입]
            if time_ok and consecutive_ok and capacity_ok:
                route.insert(best_position, station_k)
                route_actions.insert(best_position, est_action_k)
                current_total_travel_time = new_total_travel_time
                stations_in_route += 1
            else:
                skipped_stations += 1
        else:
             skipped_stations += 1




    if not route: route = [Depot, Depot]
    if route[0] != Depot: route.insert(0, Depot)
    if route[-1] != Depot: route.append(Depot)
    if len(route) < 2: route = [Depot, Depot]

    return route


# 메인 로직 실행 (단일 Period 실행으로 수정됨)

print("\n" + "="*70)
print(f"⚙️ Heuristic (2-Phase, C={VEHICLE_CAPACITY}) 최적 재배치 계획 수립 시작")
print(f"   - 차량 용량(C): {VEHICLE_CAPACITY}, 계획 기간(L): {PERIOD_LENGTH_MINS}분, 서비스 시간(H): {SERVICE_TIME_MINS}분")
print(f"   - 경로 구축 전략: Capacity-Aware Insertion (연속 {MAX_CONSECUTIVE_ACTIONS}회 제한)")
print(f"   - 재고 상/하한 로직: {'동적 (순수요±C)' if USE_DEMAND_ADAPTIVE_BOUNDS else '정적 (비율 기반)'}")
print("="*70)

# 기간별 순차적 최적화 (t=1 또는 t=2 하나만 실행됨)
for t in Periods:
    print(f"\n>>> Period {t} 최적화 진행 중...")

    # 1. 현재 기간의 입력 데이터 준비 (S0_current_period)
    S0_current_period = S0_initial
    print(f"   - Period {t} 초기 재고: API 실시간 데이터 사용")


    # 2. Smin, Smax 계산 (현재 기간 t 기준)
    Smin = {}
    Smax = {}
    if USE_DEMAND_ADAPTIVE_BOUNDS:
        is_final_period = (t == 2)
        current_buffer = INVENTORY_BUFFER_C_FINAL if is_final_period else INVENTORY_BUFFER_C

        for i in Stations:
            rack = rack_counts.get(i, 10)
            s_target = TARGET_INVENTORY_RATIO * rack
            s_min_calc = s_target - current_buffer
            s_max_calc = s_target + current_buffer
            Smin[i] = max(1, min(math.floor(s_min_calc), rack))
            Smax[i] = max(1, min(math.floor(s_max_calc), rack))
            if Smin[i] >= Smax[i]:
                 Smax[i] = Smin[i] + 1
                 Smax[i] = min(Smax[i], rack)
                 if Smin[i] >= Smax[i]: Smin[i] = max(1, Smax[i] - 1)
    else:
        print(f"   - 정적 재고 상/하한 적용 (비율 기반)")
        is_final_period = (t == 2)
        s_min_ratio_t = S_MIN_RATIO_FINAL if is_final_period else S_MIN_RATIO
        s_max_ratio_t = S_MAX_RATIO_FINAL if is_final_period else S_MAX_RATIO
        Smin = {i: max(1, math.ceil(rack_counts.get(i, 10) * s_min_ratio_t)) for i in Stations}
        Smax = {i: math.floor(rack_counts.get(i, 10) * s_max_ratio_t) for i in Stations}
        for i in Stations:
             if Smin[i] >= Smax[i]:
                  Smax[i] = Smin[i] + 1
                  Smax[i] = min(Smax[i], rack_counts.get(i, 10))
                  if Smin[i] >= Smax[i]: Smin[i] = max(1, Smax[i] - 1)

    # 3. 불균형 시작 시각 (q_it) 계산 (논문 식 2-5)
    q = {}
    print(f"   - 불균형 시작 시각 (q_it) 계산 중...")
    for i in Stations:
        s0_i = S0_current_period.get(i, 0)
        e_it = E.get((i, t), 0)
        s_min_i = Smin.get(i, 1)
        s_max_i = Smax.get(i, 10)
        calc_q = PERIOD_LENGTH_MINS
        if abs(e_it) > 1e-9:
            if e_it < 0:
                if s0_i > s_min_i: calc_q = (s_min_i - s0_i) / e_it
                else: calc_q = 0
            elif e_it > 0:
                if s0_i < s_max_i: calc_q = (s_max_i - s0_i) / e_it
                else: calc_q = 0
        q[i, t] = max(0, min(PERIOD_LENGTH_MINS, calc_q))

    # --- [Phase 1] 경로 구축 ---
    print(f"   - [Phase 1] 'Capacity-Aware' 경로 구축 중...")

    fixed_route = build_insertion_route(
        t, q, G, S0_current_period, rack_counts, Smin, Smax,
        C=VEHICLE_CAPACITY,
        L_op=PERIOD_LENGTH_MINS,
        H=SERVICE_TIME_MINS,
        MAX_CONSECUTIVE=MAX_CONSECUTIVE_ACTIONS,
        MAX_LOAD_PER_STOP=MAX_LOAD_UNLOAD_CAP
    )
    route_str = " -> ".join(map(str, fixed_route))
    print(f"   - [Phase 1] 생성된 경로: {route_str}")
    print(f"   - [Phase 1] 방문 대여소 수: {len(fixed_route) - 2} / {len(Stations)}")

    # --- [Phase 2] 적재 계획 최적화 ---
    print(f"   - [Phase 2] 고정 경로 대상 최적 적재 계획 수립 중...")
    result = solve_loading_for_route(fixed_route, t, S0_current_period, B, E, q, Smin, Smax, G)

    solution_details[t] = result

    # --- 결과 처리 및 다음 기간 준비 ---
    status = result.get('status', 'Failed')
    if status == 'Optimal':
        print(f"   - [Phase 2] Solver Status: Optimal")
        print(f"   - 목적 함수 값 (Total Imbalance, r_it): {result.get('objective', 0):.4f}")
        print(f"   - 재배치 행동 (Delivery/Pickup): {result.get('actions', {})}")
        print(f"   - 경로 총 소요 시간: {result.get('total_time', 0):.2f} / {PERIOD_LENGTH_MINS} 분")

    else:
        print(f"   - ❗️ [Phase 2] 최적 적재 계획을 찾지 못했습니다. (Status: {status})")


# --- [7. 최종 결과 요약 출력] ---

print("\n" + "="*70)
print(f"✅ Heuristic (C={VEHICLE_CAPACITY}) 최적 재배치 계획 수립 완료")
print("="*70)

for t_result in Periods:
    if t_result in solution_details:
        result_data = solution_details[t_result]
        print(f"\n--- Period {t_result} 결과 요약 ---")
        print(f"Solver Status: {result_data['status']}")
        if result_data['status'] == 'Optimal':
            print(f"Objective (Total Imbalance): {result_data['objective']:.4f}")
            route_str_final = " -> ".join(map(str, result_data['route']))
            print(f"Route: {route_str_final}")
            actions_final = result_data.get('actions', {})
            if actions_final: print(f"Actions: {actions_final}")
            else: print(f"Actions: (재배치 없음)")
            print(f"Total Time Used: {result_data['total_time']:.2f} / {PERIOD_LENGTH_MINS} 분")
    else:
        print(f"\n--- Period {t_result} 결과 없음 ---")

⚙️ [주간 운행] 계획 수립을 시작합니다. (대상 Period: 1)

⚙️ Heuristic (2-Phase, C=30) 최적 재배치 계획 수립 시작
   - 차량 용량(C): 30, 계획 기간(L): 180분, 서비스 시간(H): 5분
   - 경로 구축 전략: Capacity-Aware Insertion (연속 2회 제한)
   - 재고 상/하한 로직: 동적 (순수요±C)

>>> Period 1 최적화 진행 중...
   - Period 1 초기 재고: API 실시간 데이터 사용
   - 불균형 시작 시각 (q_it) 계산 중...
   - [Phase 1] 'Capacity-Aware' 경로 구축 중...
   - [Phase 1] 방문 후보 선정 및 정렬 (q_it 기준)...
   - [Phase 1] 31개 후보 대상 경로 삽입 시도 (C=30, L_op=180, H=5, MaxConsec=2)...
      - [Phase 1] 총 31개 후보 검토 시작...
      - [Phase 1] 후보 검토 중... (1 / 31) -> 1304번 대여소
      - [Phase 1] 후보 검토 중... (5 / 31) -> 1347번 대여소
      - [Phase 1] 후보 검토 중... (10 / 31) -> 4421번 대여소
      - [Phase 1] 후보 검토 중... (15 / 31) -> 1336번 대여소
      - [Phase 1] 후보 검토 중... (20 / 31) -> 1348번 대여소
      - [Phase 1] 후보 검토 중... (25 / 31) -> 1373번 대여소
      - [Phase 1] 후보 검토 중... (30 / 31) -> 1303번 대여소
      - [Phase 1] 후보 검토 중... (31 / 31) -> 1344번 대여소
   - [Phase 1] 생성된 경로: 0 -> 1308 -> 1337 -> 1304 -> 1306 -> 1388 -> 4421 -> 1347 -> 441

### 동선 시각화 준비

In [20]:
#전체 정류소 불러오기
import pandas as pd

file_path = "/content/drive/MyDrive/DAB_따릉팡/공공자전거 대여소 정보(24.12월 기준).xlsx"

# 1️⃣ '대여소현황' 시트를 실제 헤더로 바로 읽기
df = pd.read_excel(file_path, sheet_name='대여소현황', header=0, engine='openpyxl')

# 2️⃣ 줄바꿈 문자(\n) 제거 및 컬럼명 정리
df.columns = df.columns.str.replace('\n', '', regex=False).str.strip()

# 3️⃣ 완전히 비어 있는 열은 제거
df = df.dropna(axis=1, how='all')

# 4️⃣ 결과 확인
print("✅ 컬럼명:")
print(df.columns.tolist())

print("\n✅ 상위 5행:")
print(df.head())

✅ 컬럼명:
['대여소번호', '보관소(대여소)명', '자치구', '상세주소', '위도', '경도', '설치시기', 'LCD', 'QR', '운영방식']

✅ 상위 5행:
   대여소번호     보관소(대여소)명  자치구                             상세주소         위도  \
0    NaN           NaN  NaN                              NaN        NaN   
1    NaN           NaN  NaN                              NaN        NaN   
2  301.0   경복궁역 7번출구 앞  종로구  서울특별시 종로구 사직로 지하130 경복궁역 7번출구 앞  37.575794   
3  302.0   경복궁역 4번출구 뒤  종로구  서울특별시 종로구 사직로 지하130 경복궁역 4번출구 뒤  37.575947   
4  303.0   광화문역 1번출구 앞  종로구       서울특별시 종로구 세종대로 지하189 세종로공원  37.571770   

           경도                설치시기     LCD      QR 운영방식  
0         NaN                 NaT  거치\n대수  거치\n대수  NaN  
1         NaN                 NaT     NaN     NaN  NaN  
2  126.971451 2015-10-07 12:03:46      20      20   QR  
3  126.974060 2015-10-07 12:04:22      12      12   QR  
4  126.974663 2015-10-07 00:00:00       8       8   QR  


In [21]:
# 성북구 정류소 불러오기
df_seongbuk = df[df['자치구'].str.contains('성북구', na=False)]
df_seongbuk

,대여소번호,보관소(대여소)명,자치구,상세주소,위도,경도,설치시기,LCD,QR,운영방식
620,1302.0,한성대입구역6번출구 뒤,성북구,서울특별시 성북구 동소문로 1 한성대입구역6번출구 뒤,37.588764,127.006310,2017-06-20 10:06:59,5,NaN,LCD
621,1303.0,돈암초교 입구,성북구,서울특별시 성북구 동소문로13길 38 돈암초교,37.591694,127.012497,2017-06-30 10:39:15,10,NaN,LCD
622,1304.0,만해공원,성북구,서울특별시 성북구 성북로 132-1 만해공원,37.594402,126.992310,2017-06-20 10:09:36,10,NaN,LCD
623,1305.0,성북구청,성북구,서울특별시 성북구 보문로 168 성북구청,37.589329,127.016434,2017-08-03 15:10:47,15,NaN,LCD
624,1306.0,한성대입구역2번출구,성북구,서울특별시 성북구 동소문로 2 성북구 동소문동2가 2-3,37.588448,127.006706,2017-06-22 10:24:21,10,10,QR
...,...,...,...,...,...,...,...,...,...,...
696,4420.0,성북구 한예종 캠퍼스 별관,성북구,서울특별시 성북구 화랑로32길 146-20 문화재청의릉지구관리소,37.602898,127.058479,2022-01-17 00:00:00,NaN,6,QR
697,4421.0,롯데캐슬클라시아 입구 앞,성북구,서울특별시 성북구 길음동 1289-3,37.606960,127.026070,2022-10-18 00:00:00,NaN,10,QR
698,4423.0,현대백화점 미아점,성북구,성북구 길음동 20-1,37.608849,127.028458,2023-01-05 00:00:00,NaN,10,QR
699,4425.0,하늘채코오롱아파트 앞,성북구,성북구 돌곶이로 220,37.619247,127.045586,2023-09-06 00:00:00,NaN,10,QR


In [22]:
# '대여소 번호'가 1362, 4408, 4427인 행의 인덱스를 찾습니다.
indexes_to_drop = df_seongbuk[df_seongbuk['대여소번호'].isin([1362, 4408, 4427])].index

# 해당 인덱스를 사용하여 행을 삭제합니다.
df_seongbuk = df_seongbuk.drop(indexes_to_drop)

print(df_seongbuk)

      대여소번호        보관소(대여소)명  자치구                                 상세주소  \
620  1302.0     한성대입구역6번출구 뒤  성북구        서울특별시 성북구 동소문로 1 한성대입구역6번출구 뒤   
621  1303.0          돈암초교 입구  성북구            서울특별시 성북구 동소문로13길 38 돈암초교   
622  1304.0             만해공원  성북구             서울특별시 성북구 성북로 132-1 만해공원   
623  1305.0             성북구청  성북구               서울특별시 성북구 보문로 168 성북구청   
624  1306.0       한성대입구역2번출구  성북구      서울특별시 성북구 동소문로 2 성북구 동소문동2가 2-3   
..      ...              ...  ...                                  ...   
695  4419.0   성북구 한예종 캠퍼스 본관  성북구     서울특별시 성북구 화랑로32길 146-37 한국예술종합학교   
696  4420.0   성북구 한예종 캠퍼스 별관  성북구  서울특별시 성북구 화랑로32길 146-20 문화재청의릉지구관리소   
697  4421.0    롯데캐슬클라시아 입구 앞  성북구                 서울특별시 성북구 길음동 1289-3   
698  4423.0        현대백화점 미아점  성북구                         성북구 길음동 20-1   
699  4425.0      하늘채코오롱아파트 앞  성북구                         성북구 돌곶이로 220   

            위도          경도                설치시기  LCD   QR 운영방식  
620  37.588764  127.006310 2017-06-20 10:06:59 

In [23]:
# 새로운 정류소 정보 [차고지] 넣기
new_station = {
    "대여소번호": '0000', # 기존 데이터와 겹치지 않는 번호 지정
    "보관소(대여소)명": "서울새활용플라자",
    "소재지(위치)": "성북구",
    "상세주소": "서울특별시 성동구 용답동 자동차시장길 49 서울새활용플라자",
    "위도": 37.558699,
    "경도": 127.058254,
    "설치시기": pd.Timestamp("2025-10-18 00:00:00"),
    "LCD": None,
    "QR": None,
    "운영방식": "QR"
}
# 1행짜리 데이터프레임으로 변환
new_df = pd.DataFrame([new_station])

# 기존 df_seongbuk에 추가 (new_df를 앞에 추가)
df_seongbuk = pd.concat([new_df, df_seongbuk], ignore_index=True)


# 인덱스를 다시 0부터 깔끔하게 정리
df_seongbuk.reset_index(drop=True, inplace=True)
# -----------------------------

# 확인
print(df_seongbuk)

     대여소번호        보관소(대여소)명 소재지(위치)                                 상세주소  \
0     0000         서울새활용플라자     성북구     서울특별시 성동구 용답동 자동차시장길 49 서울새활용플라자   
1   1302.0     한성대입구역6번출구 뒤     NaN        서울특별시 성북구 동소문로 1 한성대입구역6번출구 뒤   
2   1303.0          돈암초교 입구     NaN            서울특별시 성북구 동소문로13길 38 돈암초교   
3   1304.0             만해공원     NaN             서울특별시 성북구 성북로 132-1 만해공원   
4   1305.0             성북구청     NaN               서울특별시 성북구 보문로 168 성북구청   
..     ...              ...     ...                                  ...   
74  4419.0   성북구 한예종 캠퍼스 본관     NaN     서울특별시 성북구 화랑로32길 146-37 한국예술종합학교   
75  4420.0   성북구 한예종 캠퍼스 별관     NaN  서울특별시 성북구 화랑로32길 146-20 문화재청의릉지구관리소   
76  4421.0    롯데캐슬클라시아 입구 앞     NaN                 서울특별시 성북구 길음동 1289-3   
77  4423.0        현대백화점 미아점     NaN                         성북구 길음동 20-1   
78  4425.0      하늘채코오롱아파트 앞     NaN                         성북구 돌곶이로 220   

           위도          경도                설치시기   LCD    QR 운영방식  자치구  
0   37.558699  12

In [24]:
# 1. 시각화 라이브러리 설치 (이미 했다면 생략 가능)
!pip install folium

import folium
import pandas as pd
import sys
from folium.features import DivIcon # ⭐️ 커스텀 HTML 아이콘을 위해 import

# 2. ⭐️ 최적화 결과에서 동적으로 경로 및 작업내역 가져오기 ⭐️

try:
    current_period = Periods_to_plan[0]
    result_data = solution_details[current_period]
    dynamic_route = result_data['route']
    dynamic_actions = result_data.get('actions', {})

    print(f"✅ Period {current_period}의 최적화 경로를 성공적으로 가져왔습니다.")
    print(f"   -> 경로: {' -> '.join(map(str, dynamic_route))}")

except NameError as e:
    print(f"🚨 오류: '{e.name}' 변수를 찾을 수 없습니다.", file=sys.stderr)
    print("   -> 이 셀을 실행하기 전에, [7번 셀(최적화)]과 [마지막 셀(df_seongbuk 생성)]을 모두 실행했는지 확인해주세요.", file=sys.stderr)
    raise
except KeyError:
    print(f"🚨 오류: solution_details[{current_period}]에서 'route' 또는 'actions' 키를 찾을 수 없습니다.", file=sys.stderr)
    print("   -> 7번 셀(최적화)이 정상적으로 'Optimal' 상태로 완료되었는지 확인해주세요.", file=sys.stderr)
    raise
except Exception as e:
    print(f"🚨 알 수 없는 오류 발생: {e}", file=sys.stderr)
    raise

# 3. 'df_seongbuk'에서 좌표 정보 조회
try:
    df_seongbuk['대여소번호_int'] = df_seongbuk['대여소번호'].astype(int)
    coords_lookup = df_seongbuk.set_index('대여소번호_int')[['위도', '경도', '보관소(대여소)명']].to_dict(orient='index')
except NameError:
    print("🚨 오류: 'df_seongbuk' 변수를 찾을 수 없습니다.", file=sys.stderr)
    print("   -> 이 셀을 실행하기 전에, 엑셀 파일을 로드하는 마지막 3개 셀을 실행했는지 확인해주세요.", file=sys.stderr)
    raise

# 4. 경로 좌표 리스트 생성 (PolyLine용)
route_coords = []
missing_stations = []
for station_id in dynamic_route:
    if station_id in coords_lookup:
        route_coords.append((coords_lookup[station_id]['위도'], coords_lookup[station_id]['경도']))
    else:
        print(f"⚠️ 경고: {station_id}번 대여소의 좌표를 'df_seongbuk'에서 찾을 수 없습니다.")
        missing_stations.append(station_id)

if not route_coords:
    print("🚨 오류: 경로 좌표를 하나도 찾지 못했습니다. df_seongbuk 변수를 확인해주세요.")
else:
    # 5. Folium 지도 생성 (중심은 차고지)
    center_map = (coords_lookup[0]['위도'], coords_lookup[0]['경도'])
    m = folium.Map(location=center_map, zoom_start=14, tiles="cartodbdarkmatter") # 어두운 배경

    # 6.  경로 위에 마커 및 툴팁 추가
    for i, station_id in enumerate(dynamic_route):
        if station_id not in missing_stations:
            info = coords_lookup[station_id]
            coord = (info['위도'], info['경도'])

            # --- 1. 정보 정의 ---
            popup_text = f"<b>[{i}] {info['보관소(대여소)명']}</b><br>(ID: {station_id})"
            tooltip_text = "" # 클릭 없이 항상 보일 텍스트
            action_info = dynamic_actions.get(station_id)

            # --- 2. 아이콘 색상 및 툴팁/팝업 텍스트 결정 ---
            if station_id == 0:
                icon_bg_color = '#D33C40' # 빨간색
                tooltip_text = "출발" if i == 0 else "복귀"
            else:
                icon_bg_color = '#3388FF' # 파란색 (기본)
                tooltip_text = "경유" # 작업 없는 경우

                if action_info:
                    delivery_val = action_info.get('delivery', 0)
                    pickup_val = action_info.get('pickup', 0)

                    action_str = f"배달:{delivery_val} / 수거:{pickup_val}"

                    # 팝업(클릭 시)
                    popup_text += f"<br><b><span style='color:green;'>{action_str}</span></b>"
                    # 툴팁(항상 표시)
                    tooltip_text = action_str

                    if delivery_val > 0:
                        icon_bg_color = '#5CB85C' # 초록색 (배달)
                    elif pickup_val > 0:
                        icon_bg_color = '#F0AD4E' # 주황색 (수거)

            # --- 3. 방문 순서가 적힌 커스텀 아이콘 (DivIcon) 생성 ---
            icon_html = f"""
            <div style="
                font-family: Arial, sans-serif;
                font-size: 12px;
                font-weight: bold;
                color: white;
                background-color: {icon_bg_color};
                width: 24px;
                height: 24px;
                text-align: center;
                line-height: 24px;
                border-radius: 50%;
                border: 1px solid white;
            ">
                {i}
            </div>
            """

            # --- 4. 마커 + 툴팁 + 팝업 추가 ---
            folium.Marker(
                location=coord,
                # 팝업 (클릭 시)
                popup=folium.Popup(popup_text, max_width=300),

                # 툴팁 (클릭 없이 항상 표시)
                tooltip=folium.Tooltip(
                    tooltip_text,
                    permanent=True,
                    direction='right',
                    offset=[10, 0],
                    style="background-color: #FAFAFA; color: #333; border: 1px solid #CCC; padding: 5px; border-radius: 3px;"
                ),

                # 아이콘 (방문 순서 표기)
                icon=DivIcon(
                    icon_size=(24, 24),
                    icon_anchor=(12, 12), # 아이콘 중심
                    html=icon_html
                )
            ).add_to(m)

    # 7. 경로 선(PolyLine) 그리기
    folium.PolyLine(
        locations=route_coords,
        color='#3388FF',
        weight=4,
        opacity=0.9,
        tooltip=f"Period {current_period} 재배치 경로"
    ).add_to(m)



✅ Period 1의 최적화 경로를 성공적으로 가져왔습니다.
   -> 경로: 0 -> 1308 -> 1337 -> 1304 -> 1306 -> 1388 -> 4421 -> 1347 -> 4417 -> 4415 -> 4420 -> 1326 -> 1328 -> 0


### 동선 시각화 결과

In [25]:
# 1. 라이브러리 설치 (이미 했다면 생략)
!pip install osmnx geopandas networkx folium

import folium
import pandas as pd
import sys
from folium.features import DivIcon # 커스텀 HTML 아이콘

import osmnx as ox
import networkx as nx

# 2. 최적화 결과에서 동적으로 경로 및 작업내역 가져오기
try:
    current_period = Periods_to_plan[0]
    result_data = solution_details[current_period]
    dynamic_route = result_data['route']
    dynamic_actions = result_data.get('actions', {})

    print(f"✅ Period {current_period}의 최적화 경로를 성공적으로 가져왔습니다.")
    print(f"   -> 경로: {' -> '.join(map(str, dynamic_route))}")

except NameError as e:
    print(f"🚨 오류: '{e.name}' 변수를 찾을 수 없습니다.", file=sys.stderr)
    print("   -> 이 셀을 실행하기 전에, [7번 셀(최적화)]과 [마지막 셀(df_seongbuk 생성)]을 모두 실행했는지 확인해주세요.", file=sys.stderr)
    raise
except KeyError:
    print(f"🚨 오류: solution_details[{current_period}]에서 'route' 또는 'actions' 키를 찾을 수 없습니다.", file=sys.stderr)
    print("   -> 7번 셀(최적화)이 'Optimal' 상태로 완료되었는지, 혹은 'Infeasible' 대비 수정이 되었는지 확인해주세요.", file=sys.stderr)
    raise

# 3. 'df_seongbuk'에서 좌표 정보 조회
try:
    df_seongbuk['대여소번호_int'] = df_seongbuk['대여소번호'].astype(int)
    coords_lookup = df_seongbuk.set_index('대여소번호_int')[['위도', '경도', '보관소(대여소)명']].to_dict(orient='index')
except NameError:
    print("🚨 오류: 'df_seongbuk' 변수를 찾을 수 없습니다.", file=sys.stderr)
    print("   -> 이 셀을 실행하기 전에, 엑셀 파일을 로드하는 마지막 3개 셀을 실행했는지 확인해주세요.", file=sys.stderr)
    raise

# 4. Folium 지도 생성 및 도로망 데이터 다운로드
if 0 not in coords_lookup:
     print("🚨 오류: 차고지(ID: 0) 좌표를 coords_lookup에서 찾을 수 없습니다. df_seongbuk에 0번이 포함되었는지 확인하세요.")
else:
    center_map = (coords_lookup[0]['위도'], coords_lookup[0]['경도'])
    m = folium.Map(location=center_map, zoom_start=13, tiles="cartodbdarkmatter") # 어두운 배경


    G_road = ox.graph_from_point(center_map, dist=6000, network_type='drive', simplify=True)
    print("✅ 도로망 데이터 처리 완료.")

    # 5. 경로 위에 마커 및 툴팁 추가
    missing_stations = []
    for i, station_id in enumerate(dynamic_route):
        if station_id in coords_lookup:
            info = coords_lookup[station_id]
            coord = (info['위도'], info['경도'])

            # --- 1. 정보 정의 ---
            popup_text = f"<b>[{i}] {info['보관소(대여소)명']}</b><br>(ID: {station_id})"
            tooltip_text = ""
            action_info = dynamic_actions.get(station_id)

            # --- 2. 아이콘 색상 및 툴팁/팝업 텍스트 결정 ---
            if station_id == 0:
                icon_bg_color = '#D33C40' # 빨간색 (차고지)
                tooltip_text = "출발" if i == 0 else "복귀"
            else:
                icon_bg_color = '#3388FF' # 파란색 (기본 경유)
                tooltip_text = "경유"

                if action_info:
                    delivery_val = action_info.get('delivery', 0)
                    pickup_val = action_info.get('pickup', 0)
                    action_str = f"배달:{delivery_val} / 수거:{pickup_val}"

                    popup_text += f"<br><b><span style='color:green;'>{action_str}</span></b>"
                    tooltip_text = action_str

                    if delivery_val > 0:
                        icon_bg_color = '#5CB85C' # 초록색 (배달)
                    elif pickup_val > 0:
                        icon_bg_color = '#F0AD4E' # 주황색 (수거)

            # --- 3. 아이콘 크기 키우기 ---
            icon_size_px = 40  # 픽셀 (기존 24)
            font_size_px = 20  # 픽셀 (기존 12)

            icon_html = f"""
            <div style="
                font-family: Arial, sans-serif;
                font-size: {font_size_px}px;
                font-weight: bold;
                color: white;
                background-color: {icon_bg_color};
                width: {icon_size_px}px;
                height: {icon_size_px}px;
                text-align: center;
                line-height: {icon_size_px}px;
                border-radius: 50%;
                border: 1px solid white;
            ">
                {i}
            </div>
            """

            # --- 4. 마커 + 툴팁 + 팝업 추가 ---
            folium.Marker(
                location=coord,
                popup=folium.Popup(popup_text, max_width=300),
                tooltip=folium.Tooltip(
                    tooltip_text,
                    permanent=True, direction='right',
                    offset=[18, 0],
                    style="background-color: #FAFAFA; color: #333; border: 1px solid #CCC; padding: 5px; border-radius: 3px; font-size: 13px; font-weight: bold;"
                ),
                icon=DivIcon(
                    icon_size=(icon_size_px, icon_size_px),
                    icon_anchor=(icon_size_px // 2, icon_size_px // 2),
                    html=icon_html
                )
            ).add_to(m)
        else:
            print(f"⚠️ 경고: {station_id}번 대여소의 좌표를 'df_seongbuk'에서 찾을 수 없습니다. 마커를 생략합니다.")
            missing_stations.append(station_id)

    # 6.  실제 도로 경로 선그리기
    print("\n⏳ 실제 도로 경로를 계산하여 지도에 그리는 중입니다...")

    for k in range(len(dynamic_route) - 1):
        orig_id = dynamic_route[k]
        dest_id = dynamic_route[k+1]

        if orig_id in missing_stations or dest_id in missing_stations:
            continue

        orig_info = coords_lookup[orig_id]
        dest_info = coords_lookup[dest_id]
        orig_coord = (orig_info['위도'], orig_info['경도'])
        dest_coord = (dest_info['위도'], dest_info['경도'])

        try:
            orig_node = ox.nearest_nodes(G_road, X=orig_info['경도'], Y=orig_info['위도'])
            dest_node = ox.nearest_nodes(G_road, X=dest_info['경도'], Y=dest_info['위도'])
            shortest_path_nodes = nx.shortest_path(G_road, source=orig_node, target=dest_node, weight='length')
            path_coords = [(G_road.nodes[node]['y'], G_road.nodes[node]['x']) for node in shortest_path_nodes]

            folium.PolyLine(
                locations=path_coords,
                color='#3388FF',
                weight=9, #
                opacity=0.8
            ).add_to(m)

        except (nx.NetworkXNoPath, KeyError, Exception) as e:
            print(f"  ⚠️ {orig_id} -> {dest_id} 도로 경로 탐색 실패 (이유: {e}). 직선으로 대체합니다.")
            folium.PolyLine(
                locations=[orig_coord, dest_coord],
                color='#FF0000',
                weight=3, #
                opacity=0.8,
                dash_array='5, 5'
            ).add_to(m)

    # 7. 지도 표시

    display(m)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 5.0 MB/s eta 0:00:00
✅ Period 1의 최적화 경로를 성공적으로 가져왔습니다.
   -> 경로: 0 -> 1308 -> 1337 -> 1304 -> 1306 -> 1388 -> 4421 -> 1347 -> 4417 -> 4415 -> 4420 -> 1326 -> 1328 -> 0
✅ 도로망 데이터 처리 완료.

⏳ 실제 도로 경로를 계산하여 지도에 그리는 중입니다...
